In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import pandas as pd


# Read individual categories

In [2]:
import os

path = '/Users/hamzazair/Desktop/HamzazProjects/EU Projects/Open Maker/openmaker github/OpenMaker/Semantics/output/'
fn = os.listdir(path)

DF = []
CATEGORIES = []
for f in fn:
    print(f)
    df = pd.read_csv(path+f)
    DF.append(df)
    #CATEGORIES.append(f.split('_')[1].split('.')[0])
    CATEGORIES.append(f[9:].split('.')[0])
#need to remove.DS_Store

schwartz_Stimulation.csv
schwartz_Hedonism.csv
schwartz_Tradition.csv
.DS_Store
schwartz_Conformity.csv
schwartz_Self-direction.csv
schwartz_Benevolence.csv
schwartz_Power.csv
schwartz_Universalism.csv
schwartz_Achievement.csv
schwartz_Security.csv


In [3]:
CATEGORIES

['Stimulation',
 'Hedonism',
 'Tradition',
 '',
 'Conformity',
 'Self-direction',
 'Benevolence',
 'Power',
 'Universalism',
 'Achievement',
 'Security']

In [4]:
DF[0].Stem

0            explor
1               mid
2           fantasi
3             sport
4             citat
5           zealand
6          adventur
7            global
8             arriv
9            destin
10          fiction
11             nich
12            sport
13            olymp
14          iceland
15         particip
16            drama
17              non
18         thailand
19      documentari
20           explor
21           explor
22          overlap
23          shooter
24           featur
25         challeng
26           travel
27           explor
28          tourist
29         mytholog
           ...     
1329           sens
1330             go
1331           done
1332            two
1333            put
1334          thing
1335         someth
1336           last
1337           make
1338           away
1339           ever
1340          cours
1341            saw
1342           hous
1343           side
1344           give
1345            now
1346           good
1347          might


Retrieve a news item

In [5]:
"""import requests
import json

insightIP = 'http://178.62.229.16'
insightPort = '8484'
insightVersion = 'v1.0'

# parameters of news recommendation are twitter_ids, crm_ids
twitter_ids = []
crm_ids = [135574293]

insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion 
request = '/recommendation/news?' + 'twitter_ids=' + str(twitter_ids) + '&' + 'crm_ids=' + str(crm_ids)

# send a request
res = requests.get(insightSetting + request)

data = res.json()
"""
#data = "UN Secretary Guterres on Peacekeeping: I thank the Kingdom of the Netherlands for organizing this important debate and I thank you Prime Minister Mark Rutte for presiding it. I have addressed this Council several times over the past year on peacekeeping reform. It is now time to take action together.  At its best, UN peacekeeping is a remarkable enterprise of multilateralism and international solidarity. From Sierra Leone to Cambodia, Timor Leste, Namibia, El Salvador and elsewhere, United Nations peacekeeping has helped countries move from war to peace and supported the work of civil society activists—like Ms. Fatimata Touré who is with us today.  In West Africa, for example, Liberia, Sierra Leone and Côte d’Ivoire are now peaceful, thanks in part to the support of United Nations peacekeeping.  It is inspiring to see Côte d’Ivoire represented here today in the Council. Three UN peacekeeping missions have completed their mandates and left, after a job well done. That is our aim, for every mission.  But we all know that UN peacekeeping faces serious challenges, particularly in four of our largest deployments: Mali, the Democratic Republic of the Congo, the Central African Republic, and South Sudan. Put simply, peace operations cannot succeed if they are deployed instead of a political solution, rather than in support of one.  And UN peacekeeping missions now operate in far more dangerous, complex and high-risk environments. They are under threat from armed groups, criminals and terrorists, who have access to powerful modern weapons. UN peacekeepers are often under-equipped, under-prepared and unready for the dangerous environments in which they now operate.  There are gaps in command and control, in culture, in equipment and in training. Our peacekeepers are vulnerable, and they are targeted for attack.  Last year, we lost 59 peacekeepers through malicious acts – a sharp increase since 2016, when the figure was 34.  I pay tribute to the fallen; we will never forget their sacrifice. These figures are unacceptable and weigh on us all. That is why I will spend Peacekeepers’ Day this year in Mali, where our mission sustained the highest casualties last year.  Mr. President, We are damaging the instrument of peacekeeping, and indeed multilateralism itself, in creating unrealistic expectations. Lives and credibility are being lost.   These challenges require strong, collective action.   We should focus our efforts in three areas:   First, to refocus peacekeeping with realistic expectations; Second, to make peacekeeping missions stronger and safer; And, third, to mobilize greater support for political solutions and for well-structured, well-equipped, well-trained forces.  The United Nations Secretariat has set change in motion, based on the Security Council and General Assembly resolutions on sustaining peace, the Santos Cruz report, and other reviews and reports over recent years.  I will provide Member States today, and on a regular basis, with a summary of the concrete actions that have been implemented under the responsibility of my two Under-Secretaries General for Peacekeeping Operations and Field Support.  The document will be available at the end of the session. First, we are working to improve the safety and security of peacekeepers.  We have already started to implement measures to improve the preparedness and response of missions at high risk by strengthening training, reviewing medical support, and addressing performance issues.   Second, we are conducting independent reviews of our peacekeeping missions, aimed at refining their priorities and configuration, while assessing the viability of mandates and political processes.   The reviews are aimed at ensuring we have well-trained, well-equipped peacekeepers that are mobile and agile, and can be proactive in dealing with challenges and threats. Too often in the past, our troops have been reduced to waiting in a defensive posture, giving hostile forces time and space to plan attacks.   Third, I have launched a new approach to sexual exploitation and abuse, by ensuring that victims have a clear way to report allegations, and working with troop- and police-contributing countries to address allegations, end impunity and prevent future cases.   I am encouraged that 90 countries have joined the Compact on Preventing Sexual Exploitation and Abuse. I have appointed a system-wide Victims’ Rights Advocate, who is working closely with governments and civil society. Four more victims’ rights advocates are now working in the peacekeeping missions where there have been the highest number of allegations of sexual exploitation and abuse. And I have invited Heads of State and Government to join a Circle of Leadership – a visible and powerful symbol of our joint commitment to prevent and end this scourge.   Fourth, my reforms of the peace and security architecture should result in better analysis, stronger support to troop and police contributors, more effective political engagement, and enhanced accountability and transparency – as requested by many of you.  The overall goal of the reforms is to improve our capacities to prevent conflict and sustain peace.   Mr. President, These efforts are critical – but action by the Secretariat alone is not enough to meet the challenges we face. Our chances of success increase dramatically when we work together with Member States and share burdens, risks and responsibilities.   We urgently need a quantum leap in collective engagement. This is why I am launching a new initiative, ‘Action for Peacekeeping’, aimed at mobilizing all partners and stakeholders to support the great enterprise of United Nations peacekeeping.  As peacekeeping marks its 70th anniversary, I hope we can develop a set of mutually-agreed principles and commitments to create peacekeeping operations fit for the future.   These will be elaborated with our partners, including at a High-Level side event on the margins of the General Assembly in September. I hope we can reach a formal agreement by the end of the year.   I thank the Netherlands and Cote d’Ivoire for moving this effort forward, and I encourage all Council members and all our partners for peace to join.   As we build this agreement together, I have six immediate requests for Member States.   First, I urge Security Council members to sharpen and streamline mandates.   Please put an end to mandates that look like Christmas trees. Christmas is over, and the United Nations Mission in South Sudan cannot possibly implement 209 mandated tasks.   By attempting too much, we dilute our efforts and weaken our impact.   I hope our mission reviews will help to end this mandate inflation.  Second, I call on Member States to sustain your political engagement and push for political solutions and inclusive peace processes, including through bilateral diplomacy and sanctions if necessary.  A peacekeeping operation is not an army, or a counter-terrorist force, or a humanitarian agency. It is a tool to create the space for a nationally-owned political solution."
#data = "UN Secretary Guterres on Slavery: I am pleased to join you today to remember and commemorate the victims and survivors of the transatlantic slave trade. Lasting for over 400 years, the abominable buying and selling of human beings was the largest forced movement of people in history. It was inhumane.  It was shameful. And yet it was legally sanctioned – conducted and condoned by leaders and countries in Europe, the Americas and elsewhere.  Eleven years ago, the General Assembly established the International Day of Remembrance of the Victims of Slavery and the Transatlantic Slave Trade to acknowledge the horrific trafficking in human lives. On this Day we must also recognize the role played by many of our countries, including my own, Portugal, in robbing millions of people of their homes, families, dignity and lives and profiting from their misery. But this observance was established not only to acknowledge a dreadful chapter in human history, but also to shine a spotlight on the dangers of racism and prejudice today. Although the transatlantic slave trade was abolished in the 1800s, it continues to affect social, cultural and political interaction among people and countries. This tragic mass human suffering must be recounted to younger generations through education that offers an accurate reflection of historical accounts, including the many acts of bravery and resistance carried out by slaves. The UN’s Remember Slavery Programme and UNESCO’s Slave Route Project are among the initiatives that, through education and outreach, contribute to more inclusive societies. It is equally important to highlight the enormous contributions of people of African descent across the world. We see those contributions everywhere, in every area of human endeavour, in every realm of human experience.  From the sciences to the arts, from academia to sports, to politics, law, civil rights and international affairs.  One descendant of slaves made history at the United Nations itself: Ralph Bunche, the first African-American to win a Nobel Prize.  One of the most respected and celebrated international civil servants in the history of our Organization, he once said hearts are the strongest when they beat in response to noble ideals. Our work today builds on his achievements. The son of enslaved parents who had escaped to freedom, Lewis Howard Latimer is considered one of the greatest inventors of the United States, especially for his improvement of carbon filaments in light bulbs. Born in Saint Lucia, Sir William Arthur Lewis became the first person of African descent to win the Nobel Prize for Economics. The contributions of Martin Luther King, Jr. to human and civil rights are of such global renown and eternal value that they need no description. I commend to you all an exhibition currently on display in our visitor’s lobby, which highlights the work of contemporary architects of African descent in different parts of the world.  One of featured individuals is Elizabeth Kennedy, a descendant of slaves from Jamaica, who founded her own firm of landscape architects.  Graciela Dixon is the first woman of African Descent to serve as Chief Justice of the Supreme Court of Panama and whom we are delighted to host here today. The efforts of these and other people give life to the theme of this year’s observance, Triumphs and Struggles for Freedom and Equality, which is inspired by the efforts of survivors and their descendants to establish better lives for themselves and more just societies for all. It was precisely to ensure freedom and equality for all that the United Nations General Assembly adopted the Universal Declaration of Human Rights 70 years ago. Article 4 of the Declaration is emphatic: No one shall be held in slavery or servitude; slavery and the slave trade shall be prohibited in all their forms. Yet today we see modern manifestations of servitude and bondage, with millions of children among the victims. We know as well the depth of the gaps that exist in the realization of the rights of people of African descent, and the intensity of the racism and hatred they encounter daily. In this, the International Decade of People of African Descent, we must recognize the work that is still to be done.   And on this International Day of Remembrance, let us pledge to remain forever vigilant and to use this opportunity to lift all lives and fight against forced labour and other horrendous abuses that have no place in our world. Today we honour the memory of victims and survivors of the transatlantic slave trade by continuing our common struggle to ensure that all people live in dignity and justice.  I thank you for commemorating this day and for raising your voices for this noble cause."
#data = "UN Secretary Guterres on Technology: I thank Ambassador Bessho and the Permanent Mission of Japan for inviting me today and for hosting this gathering of experts on an important and fast-moving set of issues. We live in an era of unprecedented technological advances – one that many are calling the Fourth Industrial Revolution, with tectonic social and economic shifts. These have implications for all aspects of modern life, including how we work and the overall security environment.  It is clear that what we once considered frontier issues have moved rapidly to the front door. The advances of the Fourth Industrial Revolution, including those brought on by a combination of computing power, robotics, big data and artificial intelligence, are generating revolutions in healthcare, transport and manufacturing. I am convinced that these new capacities can help us to lift millions of people out of poverty, achieve the Sustainable Development Goals and enable developing countries to leapfrog into a better future. New technologies could also enhance the maintenance of peace and security, including disarmament and non-proliferation objectives, by providing new tools and augmenting existing ones.  For example, the use of shared ledger technology such as Blockchain in nuclear safeguards, or machine learning in multilateral disarmament verification – as the Comprehensive Nuclear-Test-Ban Treaty Organization is pioneering. The United Nations itself has been using unarmed, uninhabited aerial vehicles in our peacekeeping operations, helping to improve our situational awareness and to strengthen our ability to protect civilians. There is wide scope to use these and other innovations to build our defenses against vulnerabilities in our collective security. However, together with these clear benefits, there are also clear risks. Advances in technology are giving rise very quickly to new methods and means of warfare, with potentially undesirable or unclear consequences. Already, enabling technologies – such as information communication technologies, artificial intelligence and 3D printing – can be used for military purposes. Innovations intended for civilian applications, such as synthetic biology or facial recognition software, can be repurposed for harmful outcomes.  Of course, some technologies have purely military applications, such as precision-guided long-range weapons, including so-called hypersonic weapons. But in the long-term, deployment of new means and methods of warfare could potentially allow armed conflict to be waged from greater distances, at faster speeds and with enhanced destructive power. Such developments, that we are already witnessing to a certain extent, have unexamined consequences relating to, for example, escalation control and attribution. These new technologies pose unforeseen challenges to regional and global stability. The ‘democratization’ of many of these technologies also means that non-state actors, including terrorist groups, could acquire them. The links between emerging technology and weapons of mass destruction also poses challenges. For example, cyberattacks against nuclear command and control facilities could increase the possibility of accidents or miscalculation. Such developments would be worrying even in the most benign security environment, but today, we live in a climate characterized by fragmentation and an absence of confidence, namely among the big powers, that we can witness every day in the Security Council. There is a danger that military solutions could take precedence over dialogue and diplomacy. Military spending continues to rise while tens of thousands of civilians die in armed conflict. The threat posed by nuclear weapons is at a height not seen since the end of the Cold War, and tensions are running high and heated rhetoric is common. In this environment, extra care must be taken to avoid actions that could heighten tensions, increase the prospect of misunderstanding or precipitate dangerous outcomes such as arms races. In the broadest sense, our challenge is to maximize the benefits of the technological revolution while mitigating and preventing the dangers. And specifically on disarmament, the key question is this: how do we use these innovations to make progress on disarmament without losing control of these technologies and fuelling an arms race and conflict? The innovations we are talking about are at various stages of maturity. Some are already having an impact on international peace and security.  Others remain on the drawing board. Likewise, any response should also be layered. There is a spectrum of possible responses. One option could be industry self-regulation, such as codes of conduct, to promote responsible innovation. Another is the robust implementation of national measures such as legal weapons reviews pursuant to Article 36 of Additional Protocol I to the Geneva Conventions. A further possibility could be formal confidence-building measures such as political declarations on the use of weapons. And if necessary, international legally binding instruments might be part of the picture. A first step we can all take is a holistic attempt to improve global awareness and understanding of the implications of these inter-linked technological advances on international peace and security. After all, the long-term ramifications of much of this technology will affect everyone. The United Nations, as it was said, stands ready to assist however we can, but leadership on ideas and implementation must naturally come from the membership. Many of these issues are already being taken up in UN forums – including in the disarmament machinery. The Convention on Certain Conventional Weapons, which deals with lethal autonomous weapons systems, is a good example of the kind of multi-stakeholder dialogue that can be convened within the UN architecture. More broadly, with its mandate to discuss questions of international security and disarmament, the First Committee of the General Assembly is a natural home for multilateral discussions on these issues. While the First Committee has the mandate to discuss them, it would be important to involve other stakeholders. After all, civilian industry is responsible for the development of much of these technological gains.  Such stakeholders should be connected with governments to build understanding about mutual concerns. Similarly, the voices of civil society and academics working on these issues should also be taken into account. In Geneva last month, I announced my intention to launch a new initiative aimed at giving greater impetus and direction to the global disarmament agenda. That initiative will include consideration of the risks and challenges posed by the weapons of the future. Some consultations with member States have already taken place, and will continue. I have also been asked by the General Assembly to prepare a report on developments in science and technology and their potential impact on international security and disarmament. As requested, I have sought the views of Member States in preparing this report, and it will be issued for consideration at the 73rd session of the Assembly. Finally, I am working with colleagues throughout the entire UN system to determine how our organization can better harness the benefits and address the risks of new technologies, and how the United Nations itself can make better use of innovation. And I must tell you that there is still a long way to go in this regard. Throughout history, science and technology have overwhelmingly been forces for good. In view of the nature of the technologies we are looking at today, let us together consider what should constitute responsible state behavior and responsible innovation.  Let us take care that these technologies bolster, not weaken, our collective security. In this effort, I look forward to working with you all and with all those that must contribute to our collective security in the difficult times we are facing."
#data = "UN Secretary Guterres on Nowruz: It is indeed an enormous pleasure to be back in this wonderful celebration. I think we all recognize that nothing in our world is more important than the spirit of Nowruz. When we see all the conflicts around the world. When we see the plight of migrants and refugees. When we see the rise of xenophobia and racism in so many parts of the world. It is clear that we all need the spirit of Nowruz and the world needs the spirit of Nowruz in order to be able to allow for peace and for justice and for understanding, and for the capacity to have mutual respect and the capacity to bring together the population of this world around the most noble values. I thank the Ambassadors from many countries for bringing us together to celebrate Nowruz. I am delighted to say to all of you happy Nowruz! For more than 300 million people, Nowruz is about new beginnings. The beginning of a new year.  The arrival of spring.  The renewal of nature. Nowruz unites communities beyond borders. It promotes peace and solidarity within families and between generations. It encourages reconciliation and good neighbourliness. Its values echo those that we together promote at the United Nations every day and everywhere. The beginning of a new year brings with it a sense of fresh possibility. It gives us a chance to renew our commitment to peace, sustainable development and human rights. It is an opportunity to renew our pledge to human dignity and our promise to leave no one behind. It is an occasion to strengthen our new resolve to our path towards building a prosperous future for all while living in harmony with nature. Nowruz reminds us of what we have in common and the richness of our diversity.  I wish all of you a prosperous, happy, healthy and peaceful Nowruz. And may its spirit of friendship, harmony and respect for nature serve as an inspiration to us all."
#data = "UN Secretary Guterres on Water for Sustaınable Development: I am pleased to be with you on World Water Day to launch the International Decade for Action on Water for Sustainable Development I commend President Rahmon of Tajikistan for spearheading this effort at the General Assembly.I recall my trip to Tajikistan last year, when I had the opportunity to see the impact of receding glaciers in the Pamir mountains.During my visit, I also had the chance to attend the forum on the Sustainable Development Goals.And it is clear these 17 global Goals are inter-related, interdependent and mutually reinforcing.Safe water and adequate sanitation for all   the object of Sustainable Development Goal 6 -- are indispensable to achieve many other goals. Safe water and adequate sanitation underpin poverty reduction, economic growth and healthy ecosystems. They contribute to social well-being, inclusive growth and sustainable livelihoods. But, growing demands for water, coupled with poor water management, have increased water stress in many parts of the world. Climate change is adding to the pressure   and it is running faster than we are. With demand for freshwater projected to grow by more than 40 per cent by the middle of the century, and with climate change having a growing impact, water scarcity is an enormous concern. By 2050 at least one in four people will live in a country where the lack of fresh water will be chronic or recurrent. Without effective management of our water resources, we risk intensified disputes between communities and sectors and even increased tensions among nations. So far, water has historically proven to be a catalyst for cooperation not for conflict. From my own experience, the Albufeira Convention, agreed during my time as Prime Minister of Portugal, continues to promote good relations on water management between Spain and Portugal. And, there are many more examples of cooperation on water   between India and Pakistan, Bolivia and Peru, and several others. But we cannot take peace   or our precious and fragile water resources -- for granted. Quite simply, water is a matter of life and death. Our bodies are 60 per cent water. Our cities, our industries and our agriculture all depend on it. Yet, today, 40 per cent of the world’s people are affected by water scarcity; 80 per cent of wastewater is discharged untreated into the environment, and more than 90 per cent of disasters are water-related. More than 2 billion people lack access to safe water, and more than 4.5 billion people lack adequate sanitation services. What these numbers mean is a harsh daily reality for people in rural communities and urban slums in all regions of the world. Many of the most serious diseases in the developing world are directly related to unsafe drinking water, poor sanitation, and insufficient hygiene practices.  Today, I am using the launch of the Water Action Decade to make a global call to action for water, sanitation and hygiene   or WASH -- in all health care facilities. A recent survey of 100,000 facilities found that more than half lack simple necessities, such as running water and soap - and they are supposed to be healthcare facilities. The result is more infections, prolonged hospital stays and sometimes death. We must work to prevent the spread of disease. Improved water, sanitation and hygiene in health facilities is critical to this effort. Ladies and gentlemen, We cannot continue to take water for granted and expect to achieve the Sustainable Development Goals. Solutions exist and new technologies are in the pipeline to improve how we manage water for nations, communities and households. But often these solutions are inaccessible for those who need them most, perpetuating inequity within and among countries. As with most development challenges, women and girls suffer disproportionately.For example, women and girls in low-income countries spend some 40 billion hours a year collecting water. That is equivalent to the annual effort of the entire workforce of a country like France. The time spent could be much better invested in earning a livelihood or   in the case of girls   attending school It is time to change how we value and manage water.  Last week, the High-Level Panel on Water delivered its outcome report, Making every drop count: An agenda for water action. Their work is deep, serious and inspiring for us all. The United Nations stands ready to help countries to implement the Panel’s recommendations, including by promoting policy dialogue, exchanging best practices, raising awareness and forging partnerships. Member States have also asked me to prepare an Action Plan for the Water Decade, with the support of UN-Water   which I am determined to strengthen. My plan sets forth three core objectives. First, to transform our silo-based approach to water supply, sanitation, water management and disaster risk reduction to better tackle water stress, combat climate change and enhance resilience. Second, to align existing water and sanitation programmes and projects with the 2030 Agenda. Third, to generate the political will for strengthened cooperation and partnerships. I look forward to implementing this plan. The growing water crisis should be much higher on the world’s radar. Let us work collectively towards a more sustainable world, and an action-packed Decade of Water for Sustainable Development."
#data = "UN Secretary Guterres on International day of forests: It is a pleasure to be here to mark this day as part of the Netherlands’ Presidency of the Security Council. High-level debates in the Security Council may sometimes seem divorced from grassroots issues. Today, we are literally looking at grassroots. I was supposed to get my hands dirty, but circumstances did not allow it, so and that is not a metaphor. In our daily lives, in this busy United Nations complex, in the middle of this dynamic city, we all need regular reminders of our connection with nature and our planet. This act is fundamental. The health of our planet is essential to our own health and wellbeing, and to our future and trees are an essential part of it.   I welcome the Dutch Presidency’s focus on tackling the root causes of conflict, including climate change, which has very serious implications for peace and security in all regions. Trees and forests play an essential role in mitigating the impact of climate change. Planting trees is one of the most important things we can do to contribute to the health of the planet. Forests are the lungs of our planet, drawing in carbon dioxide and breathing out oxygen. And trees improve our lives both on a grand scale and at the local level. Strategic planting of trees can help save the energy used for heating in winter and for air conditioning in summer, both very necessary in New York. Trees in urban areas filter the air and remove harmful particles. They also help to filter and regulate water, preventing flooding and protecting watersheds.   They reduce noise pollution by shielding homes and offices from roads and industrial areas. They provide a hospitable place for animals to live and other plants to grow. And if any of you goes to Central Park knows how much trees can help to promote health and relaxation, improve quality of life and provide beautiful places for people to relax and socialize. Trees are mysterious, wonderful and ancient. Plants and forests are hundreds of millions of years old, while homo sapiens has been around for just 250,000 years.  As the geobiologist Hope Jahren has said: People don’t know how to make a leaf, but they know how to destroy one. Deforestation has slowed down significantly over the past 25 years. The goal of halting deforestation and restoring forests, as set out in the 2030 Agenda for Sustainable Development, is an attainable objective if we continue our efforts. Leaves, and trees, and all plants, are a vital accompaniment to human life. As we celebrate spring, let us celebrate them. I have a particular personal reason to do so. When I was born, my grandfather, the father of my mother, whose name is a family name, that I still keep, that is Oliveira, that is in Portuguese olive tree, the day I was born he planted an olive tree. And the olive tree is still there, probably in better health than myself. And I believe it will still be there many years after I no longer [will be] around. But there is one thing in trees that I find particularly important: it is how they are a symbol of dignity. It is not by chance that trees die standing. And so let us, with this symbolic gesture of today, remind ourselves that we need to do everything we can, not only for a healthy planet but [also] for human dignity to be safeguarded in an healthy planet."
#data = "UN Secretary Guterres on Racial Discrimination: The International Day for the Elimination of Racial Discrimination commemorates the Sharpeville massacre -- the horrific killing of 69 people peacefully demonstrating against apartheid in South Africa. The apartheid regime was based on institutionalized racial discrimination. It was ultimately   and thankfully   consigned to history on the release from prison and accession to the presidency of Nelson Mandela, whose centennial we mark this year. The memory of Sharpeville lives on in this annual UN observance, when we reaffirm our unequivocal rejection of all forms of racism, xenophobia and intolerance. Sadly, these attitudes persist in countries and among communities around the world. A stark and tragic example lies in the egregious treatment of Rohingya Muslims in Myanmar. It is time all nations and all people live up to the words of the Universal Declaration of Human Rights, which recognizes the inherent dignity and equal and inalienable rights of all members of the human race. This year marks the 70th anniversary of that landmark document.  We have made considerable progress since it was adopted. People around the world have gained greater freedoms and equality.  Conditions of profound economic misery and exploitation have been improved.  Women’s rights have advanced, along with the rights of children, victims of racial and religious discrimination, indigenous peoples and persons with disabilities.  And perpetrators of horrific human rights violations have been prosecuted by international criminal tribunals. But it is also plain that the words of the Universal Declaration are not yet matched by facts on the ground. In practice, people all over the world still endure constraints on -- or even total denial -- of their human rights. Gender inequality remains a pressing issue   with untold women and girls facing daily insecurity, violence and violation of their rights. We are also seeing an alarming rise in xenophobia, racism and intolerance, including anti-Semitism and anti-Muslim hatred. Far-right political parties and neo-Nazi viewpoints are seeing a resurgence. Refugees and migrants are systematically denied their rights and unjustly and falsely vilified as threats to the societies they seek to join, despite the proven benefits they bring. We still have a long way to go before we end the discriminatory attitudes, actions and practices that blight our world. So, on this international Day, let us all consider how we can better promote tolerance, inclusion and respect for diversity in all nations and among all communities. Let us work to eliminate messages of hatred   the concept of us and them; the false attitude that we can accept some and reject and exclude others simply for how they look, where they worship or who they love. And let us keep in mind the grave consequences of racist thinking   discrimination, slavery and genocide. We must always stand up to leaders who spread their toxic vison of racial superiority   especially when they couch it in sanitized language to denigrate migrants and foreigners. We have to protect our youth from these forces of intolerance and division. We cannot allow extremist ideologies to become normalized and legitimized in our societies. The answer is to preach and practice tolerance, inclusion and respect for diversity. This is achieved through greater debate and openness, and the exchange of different views, experiences and perspectives. And it is achieved through leadership   the kind of leadership admirably shown by Nelson Mandela. Leadership that is courageous enough and principled enough to counter intolerance, racism and discrimination in all its forms.  And that is what this Organization stands for."
#data = "UN Secretary Guterres on UNRWA donors: I thank Egypt, Jordan and Sweden for co-chairing this gathering. I also welcome the high-level presence of the League of Arab States, the Organization for Islamic Cooperation and the European Union. I would like to start by recalling a memorable encounter I had last year with students at an UNRWA school in Gaza. We all know the pressures that Palestinians in Gaza face -- the poverty and power shortages, the closures and restrictions, the periodic violence and fear that yet another conflict could erupt at any moment. Yet in that place of uncertainty, the young people with whom I met were composed in talking about their situation. They spoke with passion about their commitment to democracy and their thirst for human rights. They shared with me their desire to make a difference in the world.  In that place where hope is eroded daily, these students held on to their aspirations for the future. If only, I thought, the world could do more to respond to their plight, and more to translate their dreams into tangible improvements in their lives. That is precisely what UNRWA does every day with such steadfastness, not only in Gaza but in the camps, communities and countries across the region that host Palestinian refugees. We have gathered today to fortify that mission and to uphold our responsibility as an international community to support and protect the vulnerable.  We are here to help UNRWA overcome the worst financial crisis in its history. In the early days of the United Nations, the General Assembly created UN Relief and Works Agency to provide Palestine refugees with assistance, and to help them achieve their human potential. Today, UNRWA’s mission is just as crucial as it was sixty-eight years ago. But until a just and lasting solution to the Israeli-Palestinian conflict is achieved, the work of UNRWA remains vital. Vital to providing life-saving humanitarian relief and health care. Vital to building the future of Palestinian society through education. Vital to ensuring human security, rights and dignity for over five million Palestine refugees. UNRWA has a solid record of achievement. Its schools are among the best performing in the region. Its health, relief and social programmes are low-cost and high-impact. And by keeping half a million children in school and millions of people healthy and nourished, UNRWA is contributing to stability in the occupied Palestinian territory, as well as in Jordan and Lebanon -- and has  undertaken extraordinary efforts to support Palestinians who have suffered as a result of the tragedy in Syria. UNRWA is an asset to the international community that we must protect and support. Unfortunately, today’s financial crisis places all of UNRWA’s achievements at risk. Without a collective solution, UNRWA will soon run out of money. We must not allow this to happen. I have spoken to many of you about the situation. UNRWA’s Commissioner-General has also been seeking your support. We sense a very high level of concern. Now we need high-level action to translate concern into cash. I deeply appreciate the generosity of donors who support UNRWA, including those who have already pledged their contribution to its 2018 budget, including through multi-year agreements. We are also grateful to donors who have fast-tracked their contributions. Recognizing the urgent needs, the UN’s Central Emergency Response Fund is releasing an exceptional combined rapid response grant and loan allocation of $30 million.  I thank all donors to CERF who have made this possible. But we must also recognize that the financial crisis facing the Agency today is unlike those in the past.  It is far more grave, and threatens to cut programmes far more savagely. For 2018, with pledges reduced, UNRWA faces a shortfall not of millions, not even of tens of millions, but an estimated $446 million. Critical services could be reduced or eliminated entirely   from schools to sanitation, from medicine to microfinance to food security for some 1.7 million refugees in abject poverty or affected by conflict. This would have severe impacts   a cascade of problems that could push the suffering in disastrous and unpredictable directions. So today I ask you to bridge the Agency’s shortfall for 2018 with predictable, sustained and additional funding. I also appeal to you to increase support in the years ahead to ensure schooling, health care and food assistance.  Such spending is an investment with wide-ranging dividends   in the human development of the Palestinian people, in stability today and in a peaceful future in and beyond Palestine.  It can also address some of the despair and other factors that lead to radicalization. I commend UNRWA’s dedicated and highly skilled workforce -- the teachers, doctors, emergency responders and others who operate 700 schools and 140 clinics under often dire conditions. Let me also acknowledge the extraordinary solidarity of the host countries in accommodating Palestine refugees over seven decades. As you know, I have prioritized UN reform and I appreciate the advances made by UNRWA and its current leadership. This reflects our commitment to all stakeholders, including beneficiaries in our field operations, to be effective while constantly enhancing our efficiency. In that spirit, I appeal to all donors   each government and organization in this room today   to step forward and give UNRWA what you can. Truly, what you can. If each of you does that, we will be able to address UNRWA’s shortfall. More importantly, we will send a strong message to Palestine refugees that we are committed to their rights, their well-being, and meeting their daily needs. To those who may question the expense, let me echo UNRWA’s fundraising campaign: Dignity is priceless. As a matter of human solidarity, and as a matter of smart steps for peace, let us give UNRWA our full and generous support."
#data = "Politician Sanders on Foreign Policy: Let me begin by thanking Westminster College, which year after year invites political leaders to discuss the important issue of foreign policy and America’s role in the world. I am honored to be here today and I thank you very much for the invitation. One of the reasons I accepted the invitation to speak here is that I strongly believe that not only do we need to begin a more vigorous debate about foreign policy, we also need to broaden our understanding of what foreign policy is. So let me be clear: Foreign policy is directly related to military policy and has everything to do with almost seven thousand young Americans being killed in Iraq and Afghanistan, and tens of thousands coming home wounded in body and spirit from a war we should never have started. That’s foreign policy. And foreign policy is about hundreds of thousands of people in Iraq and Afghanistan dying in that same war. Foreign policy is about U.S. government budget priorities. At a time when we already spend more on defense than the next 12 nations combined, foreign policy is about authorizing a defense budget of some $700 billion, including a $50 billion increase passed just last week. Meanwhile, at the exact same time as the President and many of my Republican colleagues want to substantially increase military spending, they want to throw 32 million Americans off of the health insurance they currently have because, supposedly, they are worried about the budget deficit. While greatly increasing military spending they also want to cut education, environmental protection and the needs of children and seniors. Foreign policy, therefore, is remembering what Dwight D. Eisenhower said as he left office: In the councils of government, we must guard against the acquisition of unwarranted influence, whether sought or unsought, by the military industrial complex. The potential for the disastrous rise of misplaced power exists and will persist. And he also reminded us that; Every gun that is made, every warship launched, every rocket fired signifies, in the final sense, a theft from those who hunger and are not fed, those who are cold and are not clothed. This world in arms is not spending money alone. It is spending the sweat of its laborers, the genius of its scientists, the hopes of its children. The cost of one modern heavy bomber is this: a modern brick school in more than 30 cities. It is two electric power plants, each serving a town of 60,000 population. It is two fine, fully equipped hospitals. It is some 50 miles of concrete highway . What Eisenhower said over 50 years ago is even more true today. Foreign policy is about whether we continue to champion the values of freedom, democracy and justice, values which have been a beacon of hope for people throughout the world, or whether we support undemocratic, repressive regimes, which torture, jail and deny basic rights to their citizens. What foreign policy also means is that if we are going to expound the virtues of democracy and justice abroad, and be taken seriously, we need to practice those values here at home. That means continuing the struggle to end racism, sexism, xenophobia and homophobia here in the United States and making it clear that when people in America march on our streets as neo-nazis or white supremacists, we have no ambiguity in condemning everything they stand for. There are no two sides on that issue. Foreign policy is not just tied into military affairs, it is directly connected to economics. Foreign policy must take into account the outrageous income and wealth inequality that exists globally and in our own country. This planet will not be secure or peaceful when so few have so much, and so many have so little   and when we advance day after day into an oligarchic form of society where a small number of extraordinarily powerful special interests exert enormous influence over the economic and political life of the world. There is no moral or economic justification for the six wealthiest people in the world having as much wealth as the bottom half of the world’s population   3.7 billion people. There is no justification for the incredible power and dominance that Wall Street, giant multi-national corporations and international financial institutions have over the affairs of sovereign countries throughout the world. At a time when climate change is causing devastating problems here in America and around the world, foreign policy is about whether we work with the international community   with China, Russia, India and countries around the world - to transform our energy systems away from fossil fuel to energy efficiency and sustainable energy. Sensible foreign policy understands that climate change is a real threat to every country on earth, that it is not a hoax, and that no country alone can effectively combat it. It is an issue for the entire international community, and an issue that the United States should be leading in, not ignoring or denying. My point is that we need to look at foreign policy as more than just the crisis of the day. That is important, but we need a more expansive view. Almost 70 years ago, former British Prime Minister Winston Churchill stood on this stage and gave an historic address, known as the Iron Curtain speech, in which he framed a conception of world affairs that endured through the 20th century, until the collapse of the Soviet Union. In that speech, he defined his strategic concept as quote nothing less than the safety and welfare, the freedom and progress, of all the homes and families of all the men and women in all the lands. To give security to these countless homes, he said, they must be shielded from the two giant marauders, war and tyranny. How do we meet that challenge today? How do we fight for the freedom and progress that Churchill talked about in the year 2017? At a time of exploding technology and wealth, how do we move away from a world of war, terrorism and massive levels of poverty into a world of peace and economic security for all. How do we move toward a global community in which people have the decent jobs, food, clean water, education, health care and housing they need? These are, admittedly, not easy issues to deal with, but they are questions we cannot afford to ignore. At the outset, I think it is important to recognize that the world of today is very, very different from the world of Winston Churchill of 1946. Back then we faced a superpower adversary with a huge standing army, with an arsenal of nuclear weapons, with allies around the world, and with expansionist aims. Today the Soviet Union no longer exists. Today we face threats of a different sort. We will never forget 9/11. We are cognizant of the terrible attacks that have taken place in capitals all over the world. We are more than aware of the brutality of ISIS, Al Qaeda, and similar groups. We also face the threat of these groups obtaining weapons of mass destruction, and preventing that must be a priority. In recent years, we are increasingly confronted by the isolated dictatorship of North Korea, which is making rapid progress in nuclear weaponry and intercontinental ballistic missiles. Yes, we face real and very serious threats to our security, which I will discuss, but they are very different than what we have seen in the past and our response must be equally different. But before I talk about some of these other threats, let me say a few words about a very insidious challenge that undermines our ability to meet these other crises, and indeed could undermine our very way of life. A great concern that I have today is that many in our country are losing faith in our common future and in our democratic values. For far too many of our people, here in the United States and people all over the world, the promises of self-government -- of government by the people, for the people, and of the people -- have not been kept. And people are losing faith. In the United States and other countries, a majority of people are working longer hours for lower wages than they used to. They see big money buying elections, and they see a political and economic elite growing wealthier, even as their own children’s future grows dimmer. So when we talk about foreign policy, and our belief in democracy, at the very top of our list of concerns is the need to revitalize American democracy to ensure that governmental decisions reflect the interests of a majority of our people, and not just the few   whether that few is Wall Street, the military industrial complex, or the fossil fuel industry. We cannot convincingly promote democracy abroad if we do not live it vigorously here at home. Maybe it's because I come from the small state of Vermont, a state that prides itself on town meetings and grassroots democracy, that I strongly agree with Winston Churchill when he stated his belief that democracy is the worst form of government, except for all those other forms. In both Europe and the United States, the international order which the United States helped establish over the past 70 years, one which put great emphasis on democracy and human rights, and promoted greater trade and economic development, is under great strain. Many Europeans are questioning the value of the European Union. Many Americans are questioning the value of the United Nations, of the transatlantic alliance, and other multilateral organizations. We also see a rise in authoritarianism and right wing extremism   both domestic and foreign -- which further weakens this order by exploiting and amplifying resentments, stoking intolerance and fanning ethnic and racial hatreds among those in our societies who are struggling. We saw this anti-democratic effort take place in the 2016 election right here in the United States, where we now know that the Russian government was engaged in a massive effort to undermine one of our greatest strengths: The integrity of our elections, and our faith in our own democracy. I found it incredible, by the way, that when the President of the United States spoke before the United Nations on Monday, he did not even mention that outrage. Well, I will. Today I say to Mr. Putin: we will not allow you to undermine American democracy or democracies around the world. In fact, our goal is to not only strengthen American democracy, but to work in solidarity with supporters of democracy around the globe, including in Russia. In the struggle of democracy versus authoritarianism, we intend to win. When we talk about foreign policy it is clear that there are some who believe that the United States would be best served by withdrawing from the global community. I disagree. As the wealthiest and most powerful nation on earth, we have got to help lead the struggle to defend and expand a rules-based international order in which law, not might, makes right. We must offer people a vision that one day, maybe not in our lifetimes, but one day in the future human beings on this planet will live in a world where international conflicts will be resolved peacefully, not by mass murder. How tragic it is that today, while hundreds of millions of people live in abysmal poverty, the arms merchants of the world grow increasingly rich as governments spend trillions of dollars on weapons of destruction. I am not naïve or unmindful of history. Many of the conflicts that plague our world are longstanding and complex. But we must never lose our vision of a world in which, to quote the Prophet Isaiah, they shall beat their swords into plowshares, and their spears into pruning hooks: nation shall not lift up sword against nation, neither shall they learn war any more. One of the most important organizations for promoting a vision of a different world is the United Nations. Former First Lady Eleanor Roosevelt, who helped create the UN, called it our greatest hope for future peace. Alone we cannot keep the peace of the world, but in cooperation with others we have to achieve this much longed-for security. It has become fashionable to bash the UN. And yes, the UN needs to be reformed. It can be ineffective, bureaucratic, too slow or unwilling to act, even in the face of massive atrocities, as we are seeing in Syria right now. But to see only its weaknesses is to overlook the enormously important work the UN does in promoting global health, aiding refugees, monitoring elections, and doing international peacekeeping missions, among other things. All of these activities contribute to reduced conflict, to wars that don’t have to be ended because they never start. At the end of the day, it is obvious that it makes far more sense to have a forum in which countries can debate their concerns, work out compromises and agreements. Dialogue and debate are far preferable to bombs, poison gas, and war. Dialogue however cannot only be take place between foreign ministers or diplomats at the United Nations. It should be taking place between people throughout the world at the grassroots level. I was mayor of the city of Burlington, Vermont, in the 1980’s, when the Soviet Union was our enemy. We established a sister city program with the Russian city of Yaroslavl, a program which still exists today. I will never forget seeing Russian boys and girls visiting Vermont, getting to know American kids, and becoming good friends. Hatred and wars are often based on fear and ignorance. The way to defeat this ignorance and diminish this fear is through meeting with others and understanding the way they see the world. Good foreign policy means building people to people relationships. We should welcome young people from all over the world and all walks of life to spend time with our kids in American classrooms, while our kids, from all income levels, do the same abroad. Some in Washington continue to argue that benevolent global hegemony should be the goal of our foreign policy, that the US, by virtue of its extraordinary military power, should stand astride the world and reshape it to its liking. I would argue that the events of the past two decades   particularly the disastrous Iraq war and the instability and destruction it has brought to the region   have utterly discredited that vision. The goal is not for the United States to dominate the world. Nor, on the other hand, is our goal to withdraw from the international community and shirk our responsibilities under the banner of America First. Our goal should be global engagement based on partnership, rather than dominance. This is better for our security, better for global stability, and better for facilitating the international cooperation necessary to meet shared challenges. Here’s a truth that you don’t often hear about too often in the newspapers, on the television, or in the halls of Congress. But it’s a truth we must face. Far too often, American intervention and the use of American military power has produced unintended consequences which have caused incalculable harm. Yes, it is reasonably easy to engineer the overthrow of a government. It is far harder, however, to know the long term impact that that action will have. Let me give you some examples: In 1953 the United States, on behalf of Western oil interests, supported the overthrow of Iran’s elected Prime Minister Mohammed Mossadegh, and the re-installation of the Shah of Iran, who led a corrupt, brutal and unpopular government. In 1979, the Shah was overthrown by revolutionaries led by Ayatollah Khomeini, and the Islamic Republic of Iran was created. What would Iran look like today if their democratic government had not been overthrown? What impact did that American-led coup have on the entire region? What consequences are we still living with today? In 1973, the United States supported the coup against the democratically elected president of Chile Salvador Allende which was led by General Augusto Pinochet. The result was almost 20 years of authoritarian military rule and the disappearance and torture of thousands of Chileans   and the intensification of anti-Americanism in Latin America. Elsewhere in Latin America, the logic of the Cold War led the United States to support murderous regimes in El Salvador and Guatemala, which resulted in brutal and long-lasting civil wars that killed hundreds of thousands of innocent men, women and children. In Vietnam, based on a discredited domino theory, the United States replaced the French in intervening in a civil war, which resulted in the deaths of millions of Vietnamese in support of a corrupt, repressive South Vietnamese government. We must never forget that over 58,000 thousand Americans also died in that war. More recently, in Iraq, based on a similarly mistaken analysis of the threat posed by Saddam Hussein’s regime, the United States invaded and occupied a country in the heart of the Middle East. In doing so, we upended the regional order of the Middle East and unleashed forces across the region and the world that we’ll be dealing with for decades to come. These are just a few examples of American foreign policy and interventionism which proved to be counter-productive. Now let me give you an example of an incredibly bold and ambitious American initiative which proved to be enormously successful in which not one bullet was fired   something that we must learn from. Shortly after Churchill was right here in Westminster College, the United States developed an extremely radical foreign policy initiative called the Marshall Plan. Think about it for a moment: historically, when countries won terrible wars, they exacted retribution on the vanquished. But in 1948, the United States government did something absolutely unprecedented. After losing hundreds of thousands of soldiers in the most brutal war in history to defeat the barbarity of Nazi Germany and Japanese imperialism, the government of the United States decided not to punish and humiliate the losers. Rather, we helped rebuild their economies, spending the equivalent of $130 billion just to reconstruct Western Europe after World War II. We also provided them support to reconstruct democratic societies. That program was an amazing success. Today Germany, the country of the Holocaust, the country of Hitler’s dictatorship, is now a strong democracy and the economic engine of Europe. Despite centuries of hostility, there has not been a major European war since World War II. That is an extraordinary foreign policy success that we have every right to be very proud of. Unfortunately, today we still have examples of the United States supporting policies that I believe will come back to haunt us. One is the ongoing Saudi war in Yemen. While we rightly condemn Russian and Iranian support for Bashar al-Assad’s slaughter in Syria, the United States continues to support Saudi Arabia’s destructive intervention in Yemen, which has killed many thousands of civilians and created a humanitarian crisis in one of the region’s poorest countries. Such policies dramatically undermine America’s ability to advance a human rights agenda around the world, and empowers authoritarian leaders who insist that our support for those rights and values is not serious. Let me say a word about some of the shared global challenges that we face today. First, I would mention climate change. Friends, it is time to get serious on this: Climate change is real and must be addressed with the full weight of American power, attention and resources. The scientific community is virtually unanimous in telling us that climate change is real, climate change is caused by human activity, and climate change is already causing devastating harm throughout the world. Further, what the scientists tell us is that if we do not act boldly to address the climate crisis, this planet will see more drought, more floods   the recent devastation by Hurricanes Harvey and Irma are good examples   more extreme weather disturbances, more acidification of the ocean, more rising sea levels, and, as a result of mass migrations, there will be more threats to global stability and security. President Trump’s decision to withdraw from the Paris agreement was not only incredibly foolish and short-sighted, but it will also end up hurting the American economy. The threat of climate change is a very clear example of where American leadership can make a difference. Europe can’t do it alone, China can’t do it alone, and the United States can’t do it alone. This is a crisis that calls out for strong international cooperation if we are to leave our children and grandchildren a planet that is healthy and habitable. American leadership   the economic and scientific advantages and incentives that only America can offer   is hugely important for facilitating this cooperation. Another challenge that we and the entire world face is growing wealth and income inequality, and the movement toward international oligarchy   a system in which a small number of billionaires and corporate interests have control over our economic life, our political life, and our media. This movement toward oligarchy is not just an American issue. It is an international issue. Globally, the top 1 percent now owns more wealth than the bottom 99% of the world's population. In other words, while the very, very rich become much richer, thousands of children die every week in poor countries around the world from easily prevented diseases, and hundreds of millions live in incredible squalor. Inequality, corruption, oligarchy and authoritarianism are inseparable. They must be understood as part of the same system, and fought in the same way. Around the world we have witnessed the rise of demagogues who once in power use their positions to loot the state of its resources. These kleptocrats, like Putin in Russia, use divisiveness and abuse as a tool for enriching themselves and those loyal to them. But economic inequality is not the only form of inequality that we must face. As we seek to renew America's commitment to promote human rights and human dignity around the world we must be a living example here at home. We must reject the divisive attacks based on a person's religion, race, gender, sexual orientation or identity, country of origin, or class. And when we see demonstrations of neo naziism and white supremacism as we recently did in Charlottesville, Virginia, we must be unequivocal in our condemnation, as our president shamefully was not. And as we saw here so clearly in St. Louis in the past week we need serious reforms in policing and the criminal justice system so that the life of every person is equally valued and protected. We cannot speak with the moral authority the world needs if we do not struggle to achieve the ideal we are holding out for others. One of the places we have fallen short in upholding these ideas is in the war on terrorism. Here I want to be clear: terrorism is a very real threat, as we learned so tragically on September 11, 2001, and many other countries knew already too well. But, I also want to be clear about something else: As an organizing framework, the Global War on Terror has been a disaster for the American people and for American leadership. Orienting US national security strategy around terrorism essentially allowed a few thousand violent extremists to dictate policy for the most powerful nation on earth. It responds to terrorists by giving them exactly what they want. In addition to draining our resources and distorting our vision, the war on terror has caused us to undermine our own moral standards regarding torture, indefinite detention, and the use of force around the world, using drone strikes and other airstrikes that often result in high civilian casualties. A heavy-handed military approach, with little transparency or accountability, doesn’t enhance our security. It makes the problem worse. We must rethink the old Washington mindset that judges seriousness according to the willingness to use force. One of the key misapprehensions of this mindset is the idea that military force is decisive in a way that diplomacy is not. Yes, military force is sometimes necessary, but always   always   as the last resort. And blustery threats of force, while they might make a few columnists happy, can often signal weakness as much as strength, diminishing US deterrence, credibility and security in the process. To illustrate this, I would contrast two recent US foreign policy initiatives: The Iraq war and the Iran nuclear agreement. Today it is now broadly acknowledged that the war in Iraq, which I opposed, was a foreign policy blunder of enormous magnitude. In addition to the many thousands killed, it created a cascade of instability around the region that we are still dealing with today in Syria and elsewhere, and will be for many years to come. Indeed, had it not been for the Iraq War, ISIS would almost certainly not exist. The Iraq war, as I said before, had unintended consequences. It was intended as a demonstration of the extent of American power. It ended up demonstrating only its limits. In contrast, the Iran nuclear deal advanced the security of the US and its partners, and it did this at a cost of no blood and zero treasure. For many years, leaders across the world had become increasingly concerned about the possibility of an Iranian nuclear weapon. What the Obama administration and our European allies were able to do was to get an agreement that froze and dismantled large parts of that nuclear program, put it under the most intensive inspections regime in history, and removed the prospect of an Iranian nuclear weapon from the list of global threats. That is real leadership. That is real power. Just yesterday, the top general of US Strategic Command, General John Hyden, said The facts are that Iran is operating under the agreements the we signed up for. We now have a four-year record of Iran’s compliance, going back to the 2013 interim deal. I call on my colleagues in the Congress, and all Americans: We must protect this deal. President Trump has signaled his intention to walk away from it, as he did the Paris agreement, regardless of the evidence that it is working. That would be a mistake. Not only would this potentially free Iran from the limits placed on its nuclear program, it would irreparably harm America’s ability to negotiate future nonproliferation agreements. Why would any country in the world sign such an agreement with the United States if they knew that a reckless president and an irresponsible Congress might simply discard that agreement a few years later? If we are genuinely concerned with Iran’s behavior in the region, as I am, the worst possible thing we could do is break the nuclear deal. It would make all of these other problems harder. Another problem it would make harder is that of North Korea. Let’s understand: North Korea is ruled by one of the worst regimes in the world. For many years, its leadership has sacrificed the well-being of its own people in order to develop nuclear weapons and missile programs in order to protect the Kim family’s regime. Their continued development of nuclear weapons and missile capability is a growing threat to the US and our allies. Despite past efforts they have repeatedly shown their determination to move forward with these programs in defiance of virtually unanimous international opposition and condemnation. As we saw with the 2015 nuclear agreement with Iran, real US leadership is shown by our ability to develop consensus around shared problems, and mobilize that consensus toward a solution. That is the model we should be pursuing with North Korea. As we did with Iran, if North Korea continues to refuse to negotiate seriously, we should look for ways to tighten international sanctions. This will involve working closely with other countries, particularly China, on whom North Korea relies for some 80 percent of its trade. But we should also continue to make clear that this is a shared problem, not to be solved by any one country alone but by the international community working together. An approach that really uses all the tools of our power   political, economic, civil society   to encourage other states to adopt more inclusive governance will ultimately make us safer. Development aid is not charity, it advances our national security. It’s worth noting that the U.S. military is a stalwart supporter of non-defense diplomacy and development aid. Starving diplomacy and aid now will result in greater defense needs later on. US foreign aid should be accompanied by stronger emphasis on helping people gain their political and civil rights to hold oppressive governments accountable to the people. Ultimately, governments that are accountable to the needs of their people will make more dependable partners. Here is the bottom line: In my view, the United States must seek partnerships not just between governments, but between peoples. A sensible and effective foreign policy recognizes that our safety and welfare is bound up with the safety and welfare of others around the world, with all the homes and families of all the men and women in all the lands, as Churchill said right here, 70 years ago. In my view, every person on this planet shares a common humanity. We all want our children to grow up healthy, to have a good education, have decent jobs, drink clean water and breathe clean air, and to live in peace. That’s what being human is about. Our job is to build on that common humanity and do everything that we can to oppose all of the forces, whether unaccountable government power or unaccountable corporate power, who try to divide us up and set us against each other. As Eleanor Roosevelt reminded us, The world of the future is in our making. Tomorrow is now. My friends, let us go forward and build that tomorrow."
#data = "Honda CEO Takahiro Hachigo on Future of Honda: What Honda has achieved in the past two years and the direction Honda will take in the future Since I became president, we have set and pursued the following two themes: Development of challenging products unique to Honda Advancement of our six-region global operation structure 1. Strengthening global models and regional models We have been strengthening our global and regional models, which have long been strengths of Honda. Global models Civic: Sales of the all-new Civic with a refined design and driving experience are going strong everywhere. The announcement of this all-new Civic in Japan is planned in late July 2017. CR-V: Sales of the all-new CR-V started in the U.S. With the addition of a hybrid version, we are going to further grow CR-V as a strong global model. The next-generation Accord: Accord will undergo a full model change this year starting from the U.S. market, with a further advanced design and driving experience. Regional models Highly-acclaimed and continuously-growing regional models include the BR-V (Asia), the Ridgeline and Odyssey (North America), the Avancier and UR-V (China). N-BOX mini-vehicle model (Japan) will undergo a full model change this fall. 2. Design and driving experience Design: We are going to launch our new design direction at an auto show this fall. Driving experience: Technologies to realise what Honda strives for -- further evolved driving experience that performs at the will of the driver   are currently under development. 3. Mutually complementary system for Honda’s global operations Balancing supply and demand in global production and sales: We have been working to establish a flexible production system and mutually complementary production among six regions. We began seeing positive results from our efforts.  In North America, in order to enable us to accommodate the recent increase in demand for SUVs, we are in the process of establishing a production system where we can handle the production of light truck models such as the Honda CR-V and Pilot and the Acura MDX more flexibly. As for Civic Hatchback, supply from the U.K. to North America has begun and export to Japan is being planned.  The WR-V, a regional model developed mainly by the Honda R&D centre in Brazil, is now being produced and sold in India as well. We are facilitating inter-regional co-ordination to evolve our models more efficiently. 4. Enhancing the introduction of electrification technologies Automobiles We strive to electrify two-thirds of global automobile unit sales in 2030. Our development will put a central focus on hybrid-based models utilising a high-efficiency plug-in hybrid system unique to Honda. As for zero-emission vehicles (ZEV), we will strengthen the development of electric vehicles (battery EV) in addition to fuel cell vehicles (FCV). Battery EV: In addition to a China-exclusive model scheduled to go on sale in 2018, a dedicated EV model for other regions are also currently under development. We will introduce this model at an auto show this fall. In order to further increase development speed, we are strengthening our system and capability for the development of electrified vehicles. In October of last year, we established within Honda R&D an Electric Vehicle Development Division, a specialised team which will be in charge of developing the entire vehicle including the powertrain and body. Motorcycles We are striving to promote the electrification of commuter models, and we are planning to introduce some new models including an electric scooter in 2018. We are currently working on the research and development of a highly-convenient system for electric commuters, which feature a detachable mobile battery that is easy to replace and/or recharge. We are considering to conduct demonstration testing of this mobile battery in collaboration with the Japan Post Co., Ltd. in Japan. 5. Initiative toward the introduction of advanced safety technologies Increasing the number of vehicles that feature Honda SENSING technologies In Japan, starting with the all-new N-BOX, we will make Honda SENSING standard equipment on all new models we introduce in Japan from here forward, including mini-vehicles. We will expand application of Honda SENSING for all-new models in other regions as well, including North America, China, and Europe. Development of automated driving technologies Through its automated driving technologies, Honda is striving to serve people worldwide with the joy and freedom of collision-free mobility. There are three values we want to realise. Realisation of a society where people do not get involved in accidents Offering mobility products which enable all people to always enjoy the freedom of mobility. Creating travel time and cabin space that make mobility fun for people. Concept of Honda’s automated driving. We strive to provide our customers with a sense of confidence and trust by offering automated driving that will keep vehicles away from any dangerous situation and also not make people around the vehicle feel unsafe. By featuring smooth and natural driving characteristics, our automated driving vehicle will realise occupant comfort that gives the driver complete trust and provides a fun of mobility that makes people naturally want to go driving. We are going to realise automated driving technologies for use on highways in 2020. And then, we will make them usable in a broader area including regular roads. As for automated driving on highways, we are striving to realise an automated lane-changing function which enables the vehicle to drive in multiple lanes without any command from the driver. And we also are going to realise an automated driving function, which will free the driver from the need to monitor their surroundings while the vehicle is experiencing traffic congestion. Moreover, we will strive to achieve the technological establishment of level 4 automated driving for personal car use by around 2025. 2030 Vision 1. Concept behind formulating the 2030 Vision In order to quickly respond and overcome rapid changes in our business environment, we set the direction we will take next and establish it as our vision. For Honda to continue to be a company society wants to exist even in the year 2050, when Honda will be more than 100 years old, we envisioned what we want Honda to look like in such a future. And based on that, we determined what we want to look like in the year 2030, and we summarised that as our 2030 Vision. Constitution of the 2030 Vision 1) Vision statement 2) Corporate attitude   3) Direction of our initiatives toward the fulfillment of the 2030 Vision 4) Business viewpoint to focus on 2. Introduction of the 2030 Vision Statement We first clarified the two key elements of the Vision   the universal passion of Honda and the strengths of Honda. Universal passion of Honda Value proposition: To provide products and services that expand people’s dreams and potential. Corporate attitude: Our approach to taking on new challenges while being driven by a strong passion to realise our value proposition. Strengths of Honda Existing strengths of Honda: 1) Strength in manufacturing a broad range of products including motorcycles, automobiles and power products and 2) earning 28 million customers per year in markets around the world. New strengths: Our ability to create solutions, which includes both Mono-zukuri, the art of making things, and Koto-zukuri   which is creating new experiences through brand storytelling about the art of making things. Through this integration of further advanced Mono-zukuri and Koto-zukuri, we will continue creating the strengths of Honda. 2030 Vision Statement Two areas where we are going to create new value: 1) advancement of mobility and 2) value creation for people’s daily lives. In these areas, we will bring the universal passion of Honda to serve people worldwide with the joy of expanding their life’s potential. And toward this end, we will lead the advancement of mobility and enable people everywhere in the world to improve their daily lives. This is our 2030 Vision Statement. 3. Direction of our initiatives toward the fulfillment of the 2030 Vision 1) Creating value for mobility and daily lives We will focus on three areas, namely mobility, robotics, with a focus on AI, and energy solutions in order to provide people with the joy and freedom of mobility and the joy of making their lives better. 2) Accommodate the different characteristics of people and society We will strive to further expand the joy of people by offering products and services that are optimised based on Honda’s desire to utilise technology to help people, while putting people at the core.   3) Toward a clean and safe/secure society Striving to become No. 1 in the areas of the environment and safety, we will invest more resources in these areas. And we will strive to become a company that leads the effort to realise a carbon-free and collision-free society. 4. Business viewpoint to focus on Toward the fulfillment of this Vision, we will transform and evolve our existing businesses and also create new value through efficient use of finite corporate resources. To this end, we identified three areas we will focus on, and we will utilise our corporate resources effectively and strengthen our business foundation.  1) Inter-regional coordination and collaboration We will further enhance our inter-regional coordination and collaboration and increase the efficiency of our operations from a global perspective. Toward this end, we will grow our global models strong, so that they will be well-received anywhere in the world. As for regional models, with consideration of customers needs in each region, multiple regions with similar customer needs will work together to realise common regional models with a high level of product strength. Through this initiative, we will strive to achieve more efficient business operations in each region. 2) Collaboration for Open Innovation with third parties We will clarify what we should do on our own, and we will focus on that. At the same time, we will be proactive in engaging in Open Innovation with external partners. Since its foundation, Honda’s strengths have been advantages in technologies and products centering around the powertrain and packaging, which includes internal combustion engines as well as more recent electrified vehicles. In addition, our business foundation established all around the world also has been one of our strengths. Identifying powertrain and packaging as core Honda technologies, we would like to strengthen partnerships from the perspective of both Mono-zukuri (the art of making things) and Koto-zukuri (new experiences drawn from the art of making things) with our customers worldwide. 3) Strengthening foundation of existing businesses We are going to strengthen our business foundation and ensure sustainable operations, especially with our automobile business, in order to solidify existing businesses. We are currently moving forward with a project that will significantly advance Honda’s development system and capability. The special merit of Honda’s automobile development is our SED development system, where teams from development (R&D), production and sales work as one and form a project team to pursue product development. We will further advance this development system and introduce a new development process for Honda’s art of making automobiles, which will enable us to more efficiently create both products and services that appeal to people’s emotions. As the first step of this initiative, we created a new planning division within Honda R&D called the Product and Perceived Quality Planning Division in October of last year. This is a planning team that ensures that our products feature perceived quality which is not easy to quantify. In order to ensure that all of our automobile models consistently offer such perceived value to our customers, we are currently considering the introduction of modular strategies which include the sharing of parts and units. We will newly establish an Automobile Cost Planning Division that will be in charge of comprehensive cost reduction by working closely among the divisions of Research & Development, Purchasing, and Production in addition to strengthening our capability in the area of cost planning. Through those activities, we will aim for creating the attractive automobiles and achieving the comprehensive cost reduction by making both developing and producing new automobiles more efficient. We will begin applying these initiatives starting from automobiles scheduled to go on sale in 2019. 5. Corporate attitude For the fulfillment of our 2030 Vision, we will set the corporate attitude to pursue growth through the pursuit of quality. We will increase the quality of Honda products and services, and at the same time, we will increase the quality of our corporate activities, which we will carry out based on the strong principle that we want to please our customers."
#data = "FB CEO Zuckerberg on Harvard 2017 Commencement: President Faust, Board of Overseers, faculty, alumni, friends, proud parents, members of the ad board, and graduates of the greatest university in the world. I’m honored to be with you today because, let’s face it, you accomplished something I never could. If I get through this speech, it’ll be the first time I actually finish something at Harvard. Class of 2017, congratulations! I’m an unlikely speaker, not just because I dropped out, but because we’re technically in the same generation. We walked this yard less than a decade apart, studied the same ideas and slept through the same Ec10 lectures. We may have taken different paths to get here, especially if you came all the way from the Quad, but today I want to share what I’ve learned about our generation and the world we’re building together. But first, the last couple of days have brought back a lot of good memories. How many of you remember exactly what you were doing when you got that email telling you that you got into Harvard? I was playing Civilization and I ran downstairs, got my dad, and for some reason, his reaction was to video me opening the email. That could have been a really sad video. I swear getting into Harvard is still the thing my parents are most proud of me for. What about your first lecture at Harvard? Mine was Computer Science 121 with the incredible Harry Lewis. I was late so I threw on a t-shirt and didn’t realize until afterwards it was inside out and backwards with my tag sticking out the front. I couldn’t figure out why no one would talk to me except one guy, KX Jin, he just went with it. We ended up doing our problem sets together, and now he runs a big part of Facebook. And that, Class of 2017, is why you should be nice to people. But my best memory from Harvard was meeting Priscilla. I had just launched this prank website Facemash, and the ad board wanted to see me. Everyone thought I was going to get kicked out. My parents came to help me pack. My friends threw me a going-away party. As luck would have it, Priscilla was at that party with her friend. We met in line for the bathroom in the Pfoho Belltower, and in what must be one of the all-time romantic lines, I said: I’m going to get kicked out in three days, so we need to go on a date quickly. Actually, any of you graduating can use that line. I didn’t end up getting kicked out I did that to myself. Priscilla and I started dating. And, you know, that movie made it seem like Facemash was so important to creating Facebook. It wasn’t. But without Facemash I wouldn’t have met Priscilla, and she’s the most important person in my life, so you could say it was the most important thing I built in my time here. We’ve all started lifelong friendships here, and some of us even families. That’s why I’m so grateful to this place. Thanks, Harvard. Today I want to talk about purpose. But I’m not here to give you the standard commencement about finding your purpose. We’re millennials. We’ll try to do that instinctively. Instead, I’m here to tell you finding your purpose isn’t enough. The challenge for our generation is creating a world where everyone has a sense of purpose. One of my favorite stories is when John F. Kennedy visited the NASA space center, he saw a janitor carrying a broom and he walked over and asked what he was doing. The janitor responded: Mr. President, I’m helping put a man on the moon. Purpose is that sense that we are part of something bigger than ourselves, that we are needed, that we have something better ahead to work for. Purpose is what creates true happiness. You’re graduating at a time when this is especially important. When our parents graduated, purpose reliably came from your job, your church, your community. But today, technology and automation are eliminating many jobs. Membership in communities is declining. Many people feel disconnected and depressed, and are trying to fill a void. As I’ve traveled around, I’ve sat with children in juvenile detention and opioid addicts, who told me their lives could have turned out differently if they just had something to do, an after-school program or somewhere to go. I’ve met factory workers who know their old jobs aren’t coming back and are trying to find their place. To keep our society moving forward, we have a generational challenge to not only create new jobs, but create a renewed sense of purpose. I remember the night I launched Facebook from my little dorm in Kirkland House. I went to Noch’s with my friend KX. I remember telling him I was excited to connect the Harvard community, but one day someone would connect the whole world. The thing is, it never even occurred to me that someone might be us. We were just college kids. We didn’t know anything about that. There were all these big technology companies with resources. I just assumed one of them would do it. But this idea was so clear to us that all people want to connect. So we just kept moving forward, day by day. I know a lot of you will have your own stories just like this. A change in the world that seems so clear you’re sure someone else will do it. But they won’t. You will. But it’s not enough to have purpose yourself. You have to create a sense of purpose for others. I found that out the hard way. You see, my hope was never to build a company, but to make an impact. And as all these people started joining us, I just assumed that’s what they cared about too, so I never explained what I hoped we’d build. A couple years in, some big companies wanted to buy us. I didn’t want to sell. I wanted to see if we could connect more people. We were building the first News Feed, and I thought if we could just launch this, it could change how we learn about the world. Nearly everyone else wanted to sell. Without a sense of higher purpose, this was the startup dream come true. It tore our company apart. After one tense argument, an advisor told me if I didn’t agree to sell, I would regret the decision for the rest of my life. Relationships were so frayed that within a year or so every single person on the management team was gone. That was my hardest time leading Facebook. I believed in what we were doing, but I felt alone. And worse, it was my fault. I wondered if I was just wrong, an imposter, a 22 year-old kid who had no idea how the world worked. Now, years later, I understand that is how things work with no sense of higher purpose. It’s up to us to create it so we can all keep moving forward together. Today I want to talk about three ways to create a world where everyone has a sense of purpose: by taking on big meaningful projects together, by redefining equality so everyone has the freedom to pursue purpose, and by building community across the world. First, let’s take on big meaningful projects. Our generation will have to deal with tens of millions of jobs replaced by automation like self-driving cars and trucks. But we have the potential to do so much more together. Every generation has its defining works. More than 300,000 people worked to put a man on the moon including that janitor. Millions of volunteers immunized children around the world against polio. Millions of more people built the Hoover Dam and other great projects. These projects didn’t just provide purpose for the people doing those jobs, they gave our whole country a sense of pride that we could do great things. Now it’s our turn to do great things. I know, you’re probably thinking: I don’t know how to build a dam, or get a million people involved in anything. But let me tell you a secret: No one does when they begin. Ideas don’t come out fully formed. They only become clear as you work on them. You just have to get started. If I had to understand everything about connecting people before I began, I never would have started Facebook. Movies and pop culture get this all wrong. The idea of a single eureka moment is a dangerous lie. It makes us feel inadequate since we haven’t had ours. It prevents people with seeds of good ideas from getting started. Oh, you know what else movies get wrong about innovation? No one writes math formulas on glass. That’s not a thing. It’s good to be idealistic. But be prepared to be misunderstood. Anyone working on a big vision will get called crazy, even if you end up right. Anyone working on a complex problem will get blamed for not fully understanding the challenge, even though it’s impossible to know everything upfront. Anyone taking initiative will get criticized for moving too fast, because there’s always someone who wants to slow you down. In our society, we often don’t do big things because we’re so afraid of making mistakes that we ignore all the things wrong today if we do nothing. The reality is, anything we do will have issues in the future. But that can’t keep us from starting. So what are we waiting for? It’s time for our generation-defining public works. How about stopping climate change before we destroy the planet, and getting millions of people involved manufacturing and installing solar panels? How about curing all diseases and asking volunteers to track their health data and share their genomes? Today we spend 50x more treating people who are sick than we spend finding cures so people don’t get sick in the first place. That makes no sense. We can fix this. How about modernizing democracy so everyone can vote online, and personalizing education so everyone can learn? These achievements are within our reach. Let’s do them all in a way that gives everyone in our society a role. Let’s do big things, not only to create progress, but to create purpose. So taking on big meaningful projects is the first thing we can do to create a world where everyone has a sense of purpose. The second is redefining equality to give everyone the freedom they need to pursue purpose. Many of our parents had stable jobs throughout their careers. Now we’re all entrepreneurial, whether we’re starting projects or finding or role. And that’s great. Our culture of entrepreneurship is how we create so much progress. Now, an entrepreneurial culture thrives when it’s easy to try lots of new ideas. Facebook wasn’t the first thing I built. I also built games, chat systems, study tools and music players. I’m not alone. JK Rowling got rejected 12 times before publishing Harry Potter. Even Beyonce had to make hundreds of songs to get Halo. The greatest successes come from having the freedom to fail. But today, we have a level of wealth inequality that hurts everyone. When you don’t have the freedom to take your idea and turn it into a historic enterprise, we all lose. Right now our society is way over-indexed on rewarding success and we don’t do nearly enough to make it easy for everyone to take lots of shots. Let’s face it. There is something wrong with our system when I can leave here and make billions of dollars in 10 years while millions of students can’t afford to pay off their loans, let alone start a business. Look, I know a lot of entrepreneurs, and I don’t know a single person who gave up on starting a business because they might not make enough money. But I know lots of people who haven’t pursued dreams because they didn’t have a cushion to fall back on if they failed. We all know we don’t succeed just by having a good idea or working hard. We succeed by being lucky too. If I had to support my family growing up instead of having time to code, if I didn’t know I’d be fine if Facebook didn’t work out, I wouldn’t be standing here today. If we’re honest, we all know how much luck we’ve had. Every generation expands its definition of equality. Previous generations fought for the vote and civil rights. They had the New Deal and Great Society. Now it’s our time to define a new social contract for our generation. We should have a society that measures progress not just by economic metrics like GDP, but by how many of us have a role we find meaningful. We should explore ideas like universal basic income to give everyone a cushion to try new things. We’re going to change jobs many times, so we need affordable child care to get to work and health care that aren’t tied to one company. We’re all going to make mistakes, so we need a society that focuses less on locking us up or stigmatizing us. And as technology keeps changing, we need to focus more on continuous education throughout our lives. And yes, giving everyone the freedom to pursue purpose isn’t free. People like me should pay for it. Many of you will do well and you should too. That’s why Priscilla and I started the Chan Zuckerberg Initiative and committed our wealth to promoting equal opportunity. These are the values of our generation. It was never a question of if we were going to do this. The only question was when. Millennials are already one of the most charitable generations in history. In one year, three of four US millennials made a donation and seven out of ten raised money for charity. But it’s not just about money. You can also give time. I promise you, if you take an hour or two a week that’s all it takes to give someone a hand, to help them reach their potential. Maybe you think that’s too much time. I used to. When Priscilla graduated from Harvard she became a teacher, and before she’d do education work with me, she told me I needed to teach a class. I complained: Well, I’m kind of busy. I’m running this company. But she insisted, so I taught a middle-school program on entrepreneurship at the local Boys and Girls Club. I taught them lessons on product development and marketing, and they taught me what it’s like feeling targeted for your race and having a family member in prison. I shared stories from my time in school, and they shared their hope of one day going to college too. For five years now, I’ve been having dinner with those kids every month. One of them threw me and Priscilla our first baby shower. And next year they’re going to college. Every one of them. First in their families. We can all make time to give someone a hand. Let’s give everyone the freedom to pursue their purpose not only because it’s the right thing to do, but because when more people can turn their dreams into something great, we’re all better for it. Purpose doesn’t only come from work. The third way we can create a sense of purpose for everyone is by building community. And when our generation says everyone, we mean everyone in the world. Quick show of hands: How many of you are from another country? Now, how many of you are friends with one of these folks? Now we’re talking. We have grown up connected. In a survey asking millennials around the world what defines our identity, the most popular answer wasn’t nationality, religion or ethnicity, it was citizen of the world. That’s a big deal. Every generation expands the circle of people we consider one of us. For us, it now encompasses the entire world. We understand the great arc of human history bends towards people coming together in ever greater numbers from tribes to cities to nations to achieve things we couldn’t on our own. We get that our greatest opportunities are now global we can be the generation that ends poverty, that ends disease. We get that our greatest challenges need global responses too no country can fight climate change alone or prevent pandemics. Progress now requires coming together not just as cities or nations, but also as a global community. But we live in an unstable time. There are people left behind by globalization across the world. It’s hard to care about people in other places if we don’t feel good about our lives here at home. There’s pressure to turn inwards. This is the struggle of our time. The forces of freedom, openness and global community against the forces of authoritarianism, isolationism, and nationalism. Forces for the flow of knowledge, trade and immigration against those who would slow them down. This is not a battle of nations, it’s a battle of ideas. There are people in every country for global connection and good people against it. This isn’t going to be decided at the UN either. It’s going to happen at the local level, when enough of us feel a sense of purpose and stability in our own lives that we can open up and start caring about everyone. The best way to do that is to start building local communities right now. We all get meaning from our communities. Whether our communities are houses or sports teams, churches or music groups, they give us that sense we are part of something bigger, that we are not alone; they give us the strength to expand our horizons. That’s why it’s so striking that for decades, membership in all kinds of groups has declined as much as one-quarter. That’s a lot of people who now need to find purpose somewhere else. But I know we can rebuild our communities and start new ones because many of you already are. I met Agnes Igoye, who’s graduating today. Where are you, Agnes? She spent her childhood navigating conflict zones in Uganda, and now she trains thousands of law-enforcement officers to keep communities safe. I met Kayla Oakley and Niha Jain, graduating today, too. Stand up. Kayla and Niha started a nonprofit that connects people suffering from illnesses with people in their communities willing to help. I met David Razu Aznar, graduating from the Kennedy School today. David, stand up. He’s a former city councilor who successfully led the battle to make Mexico City the first Latin American city to pass marriage equality even before San Francisco. This is my story too. A student in a dorm room, connecting one community at a time, and keeping at it until one day we connect the whole world. Change starts local. Even global changes start small with people like us. In our generation, the struggle of whether we connect more, whether we achieve our biggest opportunities, comes down to this your ability to build communities and create a world where every single person has a sense of purpose. Class of 2017, you are graduating into a world that needs purpose. It’s up to you to create it. Now, you may be thinking: Can I really do this? Remember when I told you about that class I taught at the Boys and Girls Club? One day after class I was talking to them about college, and one of my top students raised his hand and said he wasn’t sure he could go because he’s undocumented. He didn’t know if they’d let him in. Last year I took him out to breakfast for his birthday. I wanted to get him a present, so I asked him and he started talking about students he saw struggling and said, You know, I’d really just like a book on social justice. I was blown away. Here’s a young guy who has every reason to be cynical. He didn’t know if the country he calls home the only one he’s known would deny him his dream of going to college. But he wasn’t feeling sorry for himself. He wasn’t even thinking of himself. He has a greater sense of purpose, and he’s going to bring people along with him. It says something about our current situation that I can’t even say his name because I don’t want to put him at risk. But if a high-school senior who doesn’t know what the future holds can do his part to move the world forward, then we owe it to the world to do our part too. Before you walk out those gates one last time, as we sit in front of Memorial Church, I am reminded of a prayer, Mi Shebeirach, that I say whenever I face a challenge, that I sing to my daughter thinking about her future when I tuck her into bed. It goes:May the source of strength, who blessed the ones before us, help us find the courage to make our lives a blessing. I hope you find the courage to make your life a blessing. Congratulations, Class of ’17! Good luck out there."
#data = "Apple CEO Tim Cook on MIT 2017 Commencement: Hello, MIT! Thank you. Congratulations class of ’17. I especially want to thank Chairman Millard, President Reif, distinguished faculty, trustees, and the members of the class of 1967. It is a privilege to be here today with your families and your friends on such on amazing and important day. MIT and Apple share so much. We both love hard problems. We love the search for new ideas, and we especially love finding those ideas, the really big ones, the ones that can change the world. I know MIT has a proud tradition of pranks or as you would call them, hacks. And you have have pulled off some pretty great ones over the years. I’ll never figure out how MIT students sent that Mars rover to the Kresge Oval, or put a propeller beanie on the great dome, or how you’ve obviously taken over the president’s Twitter account. I can tell college students are behind because most of the Tweets happen at 3:00 a.m. I’m really happy to be here. Today is about celebration. And you have so much to be proud of. As you leave here to start the next leg of your journey in life, there will be days where you ask yourself, ‘Where is this all going?’ ‘What is the purpose?’ ‘What is my purpose?’ I will be honest, I asked myself that same question and it took nearly 15 years to answer it. Maybe by talking about my journey today, I can save you some time. The struggle for me started early on. In high school, I thought I discovered my life’s purpose when I could answer that age-old question, ‘What do you want to be when you grow up?’ Nope. In college I thought I’d discover it when I could answer, ‘What’s your major?’ Not quite. I thought that maybe I’d discovered it when I found a good job. Then I thought I just needed to get a few promotions. That didn’t work either. I kept convincing myself that it was just over the horizon, around the next corner. Nothing worked. And it was really tearing me apart. Part of me kept pushing ahead to the next achievement. And the other part kept asking, ‘Is this all there is?’ I went to grad school at Duke looking for the answer. I tried meditation. I sought guidance in religion. I read great philosophers and authors. And in a moment of youthful indiscretion, I might even have experimented with a Windows PC, and obviously that didn’t work. After countless twists and turns, at last, 20 years ago, my search brought me to Apple. At the time, the company was struggling to survive. Steve Jobs had just returned to Apple, and had launched the ‘Think Different’ campaign. He wanted to empower the crazy ones the misfits, the rebels and the troublemakers, the round pegs, and the square holes to do the best work. If we could just do that, Steve knew we could really change the world. Before that moment, I had never met a leader with such passion or encountered a company with such a clear and compelling purpose: to serve humanity. It was just that simple. Serve humanity. And it was in that moment, after 15 years of searching, something clicked. I finally felt aligned. Aligned with a company that brought together challenging, cutting edge work with a higher purpose. Aligned with a leader who believed that technology which didn’t exist yet could reinvent tomorrow’s world. Aligned with myself and my own deep need to serve something greater. Of course, at that moment I don’t know all of that. I was just grateful to have psychological burden lifted. But with the help of hindsight, my breakthrough makes a lot more sense. I was never going to find my purpose working some place without a clear sense of purpose of its own. Steve and Apple freed me to throw my whole self into my work, to embrace their mission and make it my own. How can I serve humanity? This is life’s biggest and most important question. When you work towards something greater than yourself, you find meaning, you find purpose. So the question I hope you will carry forward from here is how will you serve humanity? The good news is since you are here today you are on a great track. At MIT you have learned how much power that science and technology have to change the world for the better. Thanks to discoveries made right here, billions of people are leading healthier, more productive and more fulfilling lives. And if we’re ever going to solve some of the hardest problems facing the world today, everything from cancer to climate change to educational inequality, then technology will help us to do it. But technology alone isn’t the solution. And sometimes it’s even part of the problem. Last year I had the chance to meet with Pope Francis. It was the most incredible meeting of my life. This is a man who has spent more time comforting the inflicted in slums than with heads of state. This may surprise you, but he knew an unbelievable amount about technology. It was obvious to me that he had thought deeply about it. Its opportunity. Its risks. Its morality. What he said to me at that meeting, what he preached, really, was on a topic that we care a lot about at Apple. But he expressed a shared concern in a powerful new way: Never has humanity had such power over itself, yet nothing ensures it will be used wisely, he has said. Technology today is integral to almost all aspects of our lives and most of the time it’s a force for good. And yet the potential adverse consequences are spreading faster and cutting deeper. The threats to security, threats to privacy, fake news, and social media that becomes antisocial. Sometimes the very technology that is meant to connect us divides us. Technology is capable of doing great things. But it doesn’t want to do great things. It doesn’t want anything. That part takes all of us. It takes our values and our commitment to our families and our neighbors and our communities, our love of beauty and belief that all of our faiths are interconnected, our decency, our kindness. I’m not worried about artificial intelligence giving computers the ability to think like humans. I’m more concerned about people thinking like computers without values or compassion, without concern for consequences. That is what we need you to help us guard against. Because if science is a search in the darkness, then the humanities are a candle that shows us where we’ve been and the danger that lies ahead. As Steve once said, technology alone is not enough. It is technology married with the liberal arts married with the humanities that make our hearts sing. When you keep people at the center of what you do, it can have an enormous impact. It means an iPhone that allows the blind person to run a marathon. It means an Apple Watch that catches a heart condition before it becomes a heart attack. It means an iPad that helps a child with autism connect with his or her world. In short, it means technology infused with your values, making progress possible for everyone. Whatever you do in your life, and whatever we do at Apple, we must infuse it with the humanity that each of us is born with. That responsibility is immense, but so is the opportunity. I’m optimistic because I believe in your generation, your passion, your journey to serve humanity. We are all counting on you. There is so much out there conspiring to make you cynical. The internet has enabled so much and empowered so many, but it can also be a place where basic rules of decency are suspended and pettiness and negativity thrive. Don’t let that noise knock you off course. Don’t get caught up in the trivial aspects of life. Don’t listen to trolls and for God’s sake don’t become one. Measure your impact in humanity not in the likes, but the lives you touch; not in popularity, but in the people you serve. I found that my life got bigger when I stopped carrying about what other people thought about me. You will find yours will too. Stay focused on what really matters. There will be times when your resolve to serve humanity will be tested. Be prepared. People will try to convince you that you should keep your empathy out of your career. Don’t accept this false premise. At a shareholders meeting a few years back, someone questioned Apple’s investment and focus on the environment. He asked me to pledge that Apple would only invest in green initiatives that could be justified with a return on investment. I tried to be diplomatic. I pointed out that Apple does many things, like accessibility features for those with disabilities that don’t rely on an ROI. We do the things because they are the right thing to d, and protecting the environment is a critical example. He wouldn’t let it go and I got my blood up. So I told him, If you can’t accept our position, you shouldn’t own Apple stock. When you are convinced that your cause is right, have the courage to take a stand. If you see a problem or an injustice, recognize that no one will fix it but you. As you go forward today, use your minds and hands and your hearts to build something bigger than yourselves. Always remember there is no idea bigger than this. As Dr. Martin Luther King said, All life is interrelated. We are all bound together into a single garment of destiny. If you keep that idea at the forefront of all that you do, if you choose to live your lives at that intersection between technology and the people it serves, if you strive to create the best, give the best, do the best for everyone, not just for some, then today all of humanity has good cause for hope. Thank you very much and congratulations class of 2017"
#data = "King Salman on Extremists: I am pleased to inaugurate the works of the second year of the seventh session of the Shura Council today, praying to Allah Almighty to make our works sincere and help us for the interests of the homeland and its citizens, appreciating the Council's efforts and works, wishing you all success. The Kingdom since its foundation by late King Abdulaziz has applied Sharia, adhered to the Islamic religion and justice in all matters, and adopted the principle of Shura (consultation). We praise Allah for all blessings, and thank Him for honoring us with the service of the Grand Holy Mosque, the Prophet's Holy Mosque, and Allah's guests of pilgrims, Umrah performers and visitors. As the Kingdom's vision 2030 includes developmental plans and programs aiming at preparing the Kingdom for the promising future and for achieving the vision's objectives, some government agencies have been restructured and a number of decisions have been taken to serve the interests of the society, strengthen the homeland's security, combat corruption, and increase the participation of male and female citizens in the national development. We appreciate the role of the private sector as an important partner in development, its support for the national economy, expanding in employing the homeland's male and female youths, and localizing the industry and will continue to empower the private sector and stimulate it to achieve further growth and development. I have directed ministers and officials to facilitate procedures, provide further services of high quality for male and female citizens, expand in a number of programs affecting main needs of citizens, notably the most important one is the housing program. Corruption, in all its kinds and forms, is a severe lesion that undermines societies and prevents their development and growth, and we are determined to confront it in a fair and firm way so that our country enjoys the renaissance and development hoped for by every citizen. In this context we ordered to form a supreme committee for public corruption issues under the chairmanship of His Royal Highness Crown Prince, and we praise Allah that those corrupted are few. And what they have made does not prejudice the integrity of honorable citizens of this country of princes, ministers, businessmen, employees, and workers at all levels, and in their various positions of responsibility in public and private sectors, as well as residents of workers and investors, whom we are proud and wish them all success. Your country seeks to develop its present, build its future and move forward on the track of growth, modernization, and continuous development, in a way that does not contradict our principles, adhering to moderation as a way and approach just as Allah commanded us to do so, we are proud of our values and constant fundamentals. Our message to all is that there is no place between us for an extremist who sees moderation as degeneration and exploits our tolerant religion to achieve his goals, and there is no place between us for a degenerated one who sees our war on extremism as a means of disseminating decadence and exploits the facilitations of the religion to achieve his goals. We will hold accountable anyone who exceeds that. We are the protectors of the religion and honored by Allah Almighty with the service Islam and Muslims, praying to Allah Almighty to crown us with all success. The Kingdom plays an effective role at regional and international organizations and enjoys a global and regional respect that enabled the timely convening of historical summits in which many leaders of the brotherly and friendly countries participated and established a joint action aimed at achieving security and stability in the region and the world. The Kingdom has continued its pioneering and effective role in combating phenomenon of terrorism and drying up its sources. The Kingdom called for a political solution for the crises in the region and resolve its issues, topped by the Palestinian Cause and restoration of the Palestinian people's legitimate rights, including the right to establish their independent state which East Jerusalem as its capital. On this occasion, I would like to reiterate the condemnation of the Kingdom and its deep regret of the US decision on Jerusalem which represents a blatant bias against the historic rights of the Palestinian people in Jerusalem, which have been guaranteed by the relevant international resolutions and have been recognized and supported by the international community. Meanwhile, the Kingdom is working with its allies to confront the tendency of interference in the countries’ internal affairs to fuel sectarian strife and undermine regional security and stability. The Kingdom seeks to consolidate the values of tolerance and coexistence along with its effort for lifting the suffering of peoples. King Salman opened the seventh session of the Shoura Council’s second year at the headquarters of the council in Riyadh. King Salman was received by his special advisor Prince Abdullah bin Abdulaziz Al Saud, the Prince of Riyadh Faisal bin Bandar bin Abdulaziz, Crown Prince Mohammed bin Salman, Deputy Prince of Riyadh Mohammed bin Abdul Rahman bin Abdul Aziz, the Minister of Interior Prince Abdul Aziz bin Saud bin Nayef bin Abdul Aziz, the Kingdom's Mufti Chairman of the Supreme Council of Scholars and the Standing Committee for Scientific Research and Ifta Sheikh Abdulaziz bin Abdullah Al-Sheikh, the Chairman of the Shoura Council Dr. Sheikh Abdullah bin Mohammed Al-Sheikh, and senior officials of the Shoura Council and the heads of the committees. "
#data = "King of Spain at WEF: I would like to express my deepest gratitude to His Majesty King Abdullah and to the World Economic Forum for their kind invitation to participate in this highly relevant and reputed forum. It is truly a great honor to be with you all here today. His Majesty knows well −and is a great part− of the long, friendly and fruitful relations that bring the kingdom of Spain and the Hashemite Kingdom of Jordan so close; as close as to consider that our respective royal families are tied by a strong and rewarding familiar bond. So I feel privileged to be here, beside Your Majesty and Queen Rania, and to address the opening ceremony of this WEF meeting you host, here in the Jordan Valley, by the Dead Sea. Allow me to congratulate you on your leadership in making this  once again  a key moment for the ongoing debates focused on the Middle East and North Africa that take place worldwide. This great meeting also owes tribute, of course, to the hard and good work of President Schwab’s team in partnership with the KAFD (King Abdullah II Fund for Development), so I’m quite sure of its success. It is also a great opportunity to accompany all the Heads of state, government, ministers and high representatives of International Organizations that are present, and also to meet many business and civil society leaders from all the region. But allow me to highlight the fact that in this WEF edition for the MENA region, and under the engaging and timely title: Enabling a Generational Transformation, we have just had the special opportunity of hearing HRH Prince Hussein Bin Abdullah speak at this morning’s opening ceremony. Indeed I think we all appreciate the important meaning of young Prince Hussein’s participation, representing the next generation, the one we all need to work for ─and with─ today, to help provide them with greater hope and real opportunities for a better future in the region. Thank you, Your Highness, for your words. And thank you, Your Majesties, for making us witnesses of Crown Prince Hussein’s strong commitment. One of the constant ambitions of Spanish foreign policy has been to foster the cooperation between Europe and both Middle Eastern and North African countries. My presence here today bears testimony of how much we cherish this Forum as a unique opportunity to advance this cooperation. In 2011, it was my father, King Juan Carlos, who came and took part in this great Forum, on the Jordanian shores of the Dead Sea. Since then, so many things have indeed changed in our world and especially in this region. We are facing very serious crisis that challenge the very foundations of a world order, that is ─with no doubt─ in need of improvements and transformations, but that is also based on highly valued principles: the sovereignty and independence of states, the respect of human rights and the benefits of free trade and the mobility of people of persons. Also in my country, many things have happened. We have overcome the worst of a serious economic crisis. Thanks to the shared efforts of our countrymen and women and of our institutions, the Spanish economy is now growing at a good and solid pace while steadily creating employment. So I am pleased to say that we are now ready and willing to strengthen our cooperation with all our partners in the Middle East, in the quest for a better future for Humanity. So I am fully convinced, that if we join forces with determination  especially those of us who share this vast world around the Mediterranean, in the west and in the east  and understand our true potential and common stakes, we can finally look ahead with well-founded optimism and honestly say to our youth: yes it is possible and worth the effort, yes there is hope, yes we are taking-on our responsibility, and yes we are doing the best we can. And Jordan, my dear friends, is clearly setting an example and showing us the way: First, by behaving with generosity, in hosting such a huge number of refugees, once again, that have been tragically obliged to leave their homes because of war and despair. Also, by acting with bravery, leading the fight against those whose only argument is terror and represent the common enemy for us all. And finally, by persevering in its message of peace, dialogue and the pursuit of an inclusive and sustainable development. Your Majesty, we thank you deeply for continuing this tradition of the Hashemite Monarchy: a tradition of bravery, generosity and tolerance. Furthermore, Jordan is also an example of constructive engagement with the whole International community. In Spain we know it well: we highly appreciate our strong bilateral ties and our shared determination to enhance our fruitful economic cooperation. Spanish companies are here to stay, wishing to be part of a bright future for this country. They are, I am pleased to say, already present in Jordan. Certainly in the energy sector, but our companies also have great interest in other areas, such as desalination, water treatment and transport infrastructures. This Forum provides us with the opportunity to share views and experiences, to work together ─governments, civil society, and the private sector─ to create the economic and social conditions for a wider and stronger shared prosperity. In our world today, security is a shared need, and we are constantly struggling to find the way to be more effective in providing it. We cannot isolate the threats nor the challenges we face, nor can we deal with them on our own, since they are more pervasive and agile than ever, travel faster and ignore all national or regional boundaries. So, likewise, we should really move ahead and find more powerful and adequate ways of also sharing our prosperity, to cooperate more efficiently in economic and social development. And that, of course, will be the best way to work for our long term greater common security. And the opportunities are right in front of us. There is clearly a new revolution emerging from information and technological advances, which indeed can drive reforms and creativity. There is, too, a new generation, well prepared, more open to the world, ready to work hard locally and globally for a better future. Let us accept the challenge of conceiving the Mediterranean and the larger Middle East region as a space of prosperity. There is certainly still a tough path ahead to tread, a dire need to do it, and of course many obstacles that we should remove: inequalities, ignorance and extremism are some of them. But we are entitled to look further. We owe it to a new generation that will not accept the past as a barrier on their way to the future. Millions of youngsters are not only dreaming but looking for a way to make their projects come true in a wide range of fields: from renewable energies to education technologies, from bio-medical research to sustainable tourism. As we meet here, a young man or woman -in Barcelona, in Amman, in Cairo, in Dubai or Casablanca- is thinking on how to build his or her start-up, where to complete his or her post-graduate studies, how to establish his or her small business or to improve his or her professional skills. They cannot wait for all the problems created by previous generations to be solved. One thing we know: the crucial role of education. It is clear that youth unemployment and lack of opportunities fuel frustration and anger. Finding the right policies to foster youth empowerment is not a simple task. This is why this Forum, widely recognized as a hub for sharing ideas and best practices, can be so useful. Schools, universities and research institutions should unleash the potential of our people. They need to be at the core of our policy-making in order for us to extend the limits of what we can achieve. International cooperation to favor student mobility between our countries is also essential. Development of effective policies in this area will not only increase the employability of young people but will also create a stronger social contract for all. The future is embodied in these young boys and girls I have been referring to. Every time two or more of them meet -no matter if it is in the field of science, business or leisure- something happens: a new building block is set to sustain the future we want for us and for our children. A new argument arises to support our conviction that the broad Mediterranean area, including all the Middle East, may indeed become  already today: not tomorrow- one of the more dynamic poles of the world. Your Royal Highness, you represent this hope. You have the benefit of the extraordinary legacy of the Hashemite Kings, and with your initiatives, you have shown your will to follow that path. In August 2015, under your patronage, a Global Forum on Youth, Peace and Security was held in Amman, following the debate that you presided in the Security Council in April of that same year. What an admirable task you are promoting: to partner with young people to build lasting peace, prevent radicalism and implement sustainable development goals. Much is as stake, and I wish you the very best in this endeavor and in your life. Your Majesty, Your Excellencies, ladies and gentlemen. My country, Spain, with a long history behind, looks to the future with optimism. We want to continue building a stronger and better European Union, based on the respect for human dignity as the ‘principle of principles’, and based also on the conviction that united we are stronger. We want to work together with the whole world, and very especially with our friends in the Middle East and North Africa, to rise up to the challenges and seize the opportunities. Our youth is ready, our companies have accelerated their international expansion in the recent past, so they are eager and well prepared. Let us join forces, defend our values and together build coalitions for peace and for inclusive development. I wish all of you a very fruitful Forum. Thank you Jordan, and thank you Your Majesties."
#data = "King of Morocco at African Union: Praise be to God May peace and blessings be upon the Prophet, His Kith and Kin His Excellency President Alpha Condé, Chairman of the 28th AU Summit Distinguished Heads of State and Government, Honourable Chairperson of the Commission, Your Excellencies, Ladies and Gentlemen, It is so good to be back home, after having been away for too long! It is a good day when you can show your affection for your beloved home! Africa is my continent, and my home. I am home at last and happily reunited with you. I have missed you all. That is why, My Dear Brothers, Heads of State, I wanted to make this trip and to address you, without waiting for the protocol and legal procedure for the Kingdom to take its place again within the Organization to be finalized. The massive, outspoken support Morocco has received is proof of the solid bonds that unite us. It was necessary to withdraw from the OAU; it has enabled Morocco’s action to be refocused in Africa to show how indispensable Africa is to Morocco and how indispensable Morocco is to Africa. We have thought it through carefully and it is now so obvious! It is time to return home; at a time when the Kingdom is among the most developed African nations and when a majority of Member States looks forward to our return, we have decided to join our family again. A family we had not really left! In fact, despite having been absent from the AU institutions for so many years, our links, which have never been severed, have remained strong and African sister nations have always been able to rely on us. Strong bilateral relations have thus been significantly developed. Since 2000, Morocco has signed nearly a thousand agreements with African countries, in various fields of cooperation. By way of comparison, do you know that between 1956 and 1999, 515 agreements were signed, whereas 949 agreements have been signed since 2000   in other words, almost twice as many! During this period I, personally, was keen to give fresh impetus to this action, by making more visits to various African sub-regions. On each of the 46 visits I paid to 25 African countries, numerous agreements were signed involving the public as well as the private sector. My action has been particularly geared towards the field of training, which is at the heart of my country’s cooperation with sister nations. This has enabled a number of African students to continue their higher education in Morocco, thanks to the thousands of scholarships given to them. Furthermore, major strategic projects were set up during my visits to these countries: Firstly, I had the pleasure of launching the Africa Atlantic Gas Pipeline project with my brother, His Excellency Mr. Muhammadu Buhari, President of the Federal Republic of Nigeria. This project will of course allow natural gas to be transported from gas-producing countries to Europe. But more than that, it will benefit the whole of West Africa. It will, indeed, contribute to creating a regional electricity market and be a substantial source of energy which will help develop industry, improve economic competitiveness and speed up social development. The project will thus create wealth for neighboring countries and populations, generating crucial momentum that will stimulate the emergence and the development of parallel projects. Moreover, it will help build more peaceful bilateral and multilateral relations and thus create an environment conducive to development and growth. Secondly, as part of projects aimed at improving agricultural productivity and promoting food security and rural development, fertilizer production plants have been set up with both Ethiopia and Nigeria. These projects will benefit the continent as a whole. As we know, basic food needs cannot be met with gas or oil. But is not food security the major challenge facing Africa? This is the objective of the initiative for the Adaptation of African Agriculture, or Triple A Initiative, which we promoted during the COP22. It is an innovative and extremely concrete response to the common challenges posed by climate change. As soon as it was launched, the initiative was backed by some thirty African countries. The Triple A Initiative is aimed at providing more significant funding for the Adaptation of small-scale African Agriculture; it will also support the structuring and acceleration of agricultural projects in Africa through four programs:  Rational management of soils;  Sustainable management of agricultural water; Climate-related risk management; and Solidarity-based funding for promoters of small projects. The initiative was also one of the main axes of the Africa Action Summit, which I had the privilege of chairing last November in Marrakesh. Finally, our ties have also remained strong as far as security and peace are concerned. Do we need to point out that we have always been present when the stability of the Continent is at stake? Since its independence, Morocco has contributed to six UN peace-keeping missions in Africa, engaging thousands of troops in various theaters of operation. Moroccan forces are still present today in CAR and DRC. Morocco has also conducted a number of mediations which helped achieve substantial progress towards peace, namely in Libya and the Mano River region.mYour Excellencies, Ladies and Gentlemen,  My vision of South-South cooperation is clear and constant: my country shares what it has, without ostentation. Within the framework of a clear-sighted collaboration, Morocco   which is a major economic player in Africa   will become a catalyst for shared expansion. In my country, sub-Saharan citizens are received according to the conditions previously announced: several regularization operations have been launched; more than 25000 people benefited from the first phase. The second phase was successfully launched just a few weeks ago, in the same spirit of solidarity and humanism. We are proud of these actions. They were necessary, vital for these men and women who have suffered too long due to their life in hiding. We are acting to stop these people from living on the fringes of society, with no work, no healthcare, nowhere to live and no access to education. We are acting so couples, particularly those from mixed marriages   between Moroccans and sub-Saharans   will not be parted. All this constructive action to help migrants has bolstered Morocco’s image and strengthened the bonds we had already forged. Some say that, through this commitment, Morocco is seeking to gain leadership in Africa. I tell them that it is to Africa that the Kingdom is seeking to give the leadership. Your Excellencies, Ladies and Gentlemen,  We know that we do not have unanimous backing from this prestigious assembly. Far be it from us to spark off a sterile debate! We have absolutely no intention of causing division, as some would like to insinuate! You will see: as soon as the Kingdom becomes a member and is able to contribute to the agenda of activities, its action will, on the contrary, help bring about unity and progress. We participated in the creation of this beautiful pan-African edifice and we naturally look forward to regaining the place that is ours within it. During all these years and without natural resources, Morocco became an emerging economy, with acknowledged expertise; today it is one of the most prosperous nations in Africa. Morocco has always considered that its strength comes primarily from the integration of the Maghreb sub-region. It is however clear that the flame of the Arab Maghreb Union has faded, because faith in a common interest has vanished! The mobilizing momentum of the Maghreb ideal, advocated by the pioneers in the 1950s, has been betrayed. Today, we regret to see that the Maghreb Union is the least integrated region in the African continent, if not in the whole world. Intra-regional trade has reached 10% between ECOWAS countries and 19% between SADC countries, while it is still stagnating at less than 3% between Maghreb countries. Similarly, while West African Economic Community countries are moving forward in ambitious integration projects and ECOWAS is offering a reliable space for free movement of persons, goods and capital, economic cooperation between Maghreb countries is at a low level. Our fellow citizens in the Maghreb find this situation hard to understand. If we do not act, by following the example of neighboring African sub- regions, the Maghreb Union will crumble in its chronic incapacity to live up to the ambitions of the Marrakesh Treaty, which gave birth to it 28 years ago. Your Excellencies, Ladies and Gentlemen,  All this confirms that Morocco is right to choose Africa. By doing so, my country has opted to share and transfer its know-how; in concrete terms, it is offering to build a safe, solidarity-based future. We are proud to see history has proved us right. Morocco is not returning to the African Union through the back door, but by the main gate. This is shown by the warm welcome extended to us today by our African brothers. We enthusiastically invite African nations to join our country’s dynamism and to give new impetus to the whole of our continent. Your Excellencies, Ladies and Gentlemen,  It is time for Africa to benefit from Africa’s wealth. We must work to enable our land, after decades of looting, to enter an era of prosperity. Admittedly, colonialism is not the sole cause of Africa’s problems. However, its negative impact persists. For a long time, we have looked elsewhere to seek help in making a decision, a commitment. Is it not time for this tropism to be stopped? Is it not time to look towards our continent? To consider its cultural wealth, its human potential? Africa should be proud of its resources, its cultural heritage, its spiritual values, and the future should strongly support this natural pride. Africa can and must validate, on its own, its elections and thus endorse its citizens’ free choice. It has regulatory tools and legal institutions, such as Constitutional Councils and Supreme Courts, which can settle electoral disputes and appeals. These institutions could be reinforced, if need be. But they exist! They are operational! Otherwise, what is the use of having them? Africa is governed today by a new generation of uninhibited leaders. They are working for the stability, political openness, economic development and social progress of their peoples. They are working with determination, resolve and conviction, without caring about being graded or assessed by the West. For several decades, the growth rates achieved in some countries in the North have not exceeded those in some African countries. The failure of their opinion polls shows the extent to which they have lost their capacity to understand their peoples’ aspirations. And yet, these countries with an ailing economy, a poor social situation and a weakening leadership, assume the right to impose their development model on us! I repeat: I consider the notion of Third-worldism to be outdated! Such practices bear signs of economic opportunism: the respect and benevolence shown to a country should no longer depend on its natural resources and the profit expected! Your Excellencies, Ladies and Gentlemen, This is the path to solidarity, peace and union chosen by my country. We reaffirm our commitment to the development and prosperity of African citizens. We, peoples of Africa, have the means and the genius; together, we can fulfill the aspirations of our peoples. Thank you for your kind attention. Wassalamu alaikum warahmatullah wabarakatuh."
#data = "King of Sweden on Christmas: Dear people of Sweden, both at home and abroad, everyone in Sweden! A child is born this day. So starts one of our most well-known Christmas hymns, with a text by the reformer Martin Luther. Christmas is a Christian festival. We celebrate in memory of the birth of Jesus. However, Christmas's message - of peace, joy and fellowship - is universal and can be embraced by everyone. Regardless of where you come from or what you believe in. This also applies to the challenge the Bible directs at us: Carry each other's burdens! Let us not become weary in doing good. I know that there are many of you in our country who devote a lot of time and energy to precisely this: carrying the burdens of others. Helping to the best of your ability. We see it time after time with different crises in society. After the terrorist act on Drottninggatan many people showed what it means to be a fellow human being. Who acted with courage, consideration and energy. This desire to help also exists in everyday life. The hand that is extended in support, that knocks on the door of a sick neighbour, that invites the lonely into the community; that hand carries an enormous force, far stronger than hatred, violence and distrust. 2017 will inevitability be associated with the terror attack in Stockholm in April. Five people lost their lives. My family and I would like to direct our warmest sympathies to their families and those close to them on this difficult first Christmas. This year has also represented the start of an important conversation between all of us. How are we to treat each other with respect? How are we to show each other consideration? How are we to strengthen responsibility and moral courage? These are questions which have always been important. But which have become particularly relevant after the autumn's many testimonies to unacceptable violations. How we answer these questions will shape the society in which our children and young people grow up. I therefore hope that this is something that we continue to talk about, now and in the future. When I look back on the year that has passed, there are a number of events in particular that I will remember: Aurora, the biggest defence exercise in our country since the early 1990s, took place in September, with over 20,000 participants from eight countries and 40 or so civil authorities. I personally had the privilege, together with Prince Carl Philip, to visit both Gotland and Sörmland to get a picture of the exercise in situ. I met both professional members of the military and volunteers on the ground, in the air and at sea. Men and women with different backgrounds who are ready to defend Sweden, our freedom and our democracy. I feel respect and gratitude for their commitment. The climate meeting in Bonn in November was a confirmation that the important agreement from Paris 2015 is holding fast. Rising global emissions are a cause of concern. But there are constructive forces that are ready to take responsibility for the future. Sweden is a relatively small country, but a large forest nation. We have been developing valuable knowledge over many years about how the forest can be managed in a way that is sustainable in the long-term. I am convinced that this knowledge will enable us to make an important contribution to the task of addressing the world's climate challenges. In the evening of the sixth of December, on Finland's independence day, blue and white candles were lit in many windows in our neighbouring country to the east. A tradition that had additional significance in that Finland was celebrating one hundred years as an independent nation. In June I personally took part in a splendid ceremony together with other Nordic Heads of State at the Hanaholmen Cultural Centre outside Helsinki. Many Swedes have their roots in Finland. Others have connections through personal ties. The Nordic community is strong. Not just historically. We are neighbours and colleagues. We cooperate and share the values of democracy and the equal worth of all people. In business and sport we are sometimes competitors. But above all we are family and friends. Sweden's relations to the Nordic countries are important. As are our relations to other countries in the world. We will continue to cultivate them. 2017 has been an eventful year for my family too: Crown Princess Victoria celebrated her 40th birthday. A joyful celebration, with a fantastic gathering both in Stockholm and on Öland. The Crown Princess has also commenced a series of walking tours that will proceed through all of Sweden's provinces. I have always had a strong commitment to nature and the environment, and is it gratifying to share this interest with my children. It is my hope that these walks through the provinces will contribute to more people opening their eyes to the fantastic natural environment to which all of us in Sweden have access. During the year my family has expanded with a new member: Prince Gabriel. He was christened just a few weeks ago in the wintry Royal Chapel at Drottningholm, with water from our own spring on Öland. And we are now looking forward to welcoming another small child in the spring. I would like to take the opportunity to say thank you for all the congratulations that we have received during the year and for the warm reception that we receive on our trips and visits to different activities. My family and I value this consideration highly. I started this Christmas greeting by talking about carrying each other's burdens. People don't like to intrude. We don't like to place obligations on others. It can require courage, both to offer help and to receive it. My wish for the coming year is that we will all find this courage. To be not just people, but also fellow human beings. With this thought, my family and I would like to wish everybody in our country a continued Merry Christmas with peace, joy and fellowship - and subsequently a Happy New Year for 2018."
#data = "PM Lee Hsien Loong of Singapore on National Day: Good evening again. My fellow Singaporeans, we have had an eventful year. We have been busy guarding against terrorism and strengthening our racial harmony, making friends and cooperating with other countries big and small. Last year, I spoke about our economy: how we were working to develop skills, build capabilities, promote entrepreneurship, and take the economy to the next level and I am happy to report good progress. We expect growth around 2.5 per cent this year, higher than last year. Wages have been rising, gradually but steadily, and most encouragingly, productivity is improving. Last year, productivity went up by one per cent, after several years of almost zero growth. This year, we should do even better. This is important because productivity is the key to our prosperity, and to higher wages. We still have work to do. Heng Swee Keat and the Future Economy Council are hard at it. Unions and employers are fully on board. And together, we are implementing Industry Transformation Maps (ITMs), industry by industry. We are also pressing on with SkillsFuture, preparing everyone for the jobs of the future, helping workers and professionals with the problems of today when they are displaced from their jobs, retraining and reskilling them to take up new jobs. If we can keep up this path, we can create better jobs and new opportunities for our workers and businesses. The immediate priorities are important and we will continue to focus on them but tonight I want to take a step back from these immediate priorities. Instead, I would like to discuss three longer term issues that are important to the success and wellbeing of Singapore, for the current generation and also for future generations. One, building up our preschools so that every child, regardless of his family background, starts well and has a bright future. Two, fighting diabetes, because many Singaporeans suffer from it, not only the old, but increasingly younger people too, and three, making Singapore a Smart Nation, by using IT to the full, to create jobs and opportunities for all Singaporeans. Preschool, Diabetes, and Smart Nation   these are the things we must do now, work on now, to build our future so that Singaporeans can start right, stay healthy and live smart at every age. Start Right   Preschool Let me start with preschool. Here, we are talking about infants to six year olds. Nowadays even two month old babies are enrolled in infant care, and that is part of preschool.Preschool is important to give our children a good start and the best chance to succeed in life. In the past, we started at Primary 1, when the child is seven. But now we know we have to begin much earlier, not to give children a head-start in Primary 1, but to build sound foundations for them for life. At an early age, there are specific windows in a child’s development and you must catch that window or you miss it. For example, a child picks up language skills most easily and rapidly in his first three years. You miss that and it becomes much harder later on. In fact, it is best if they hear the language sounds as infants, before they are eight months old. Social and emotional skills need to be learnt early too: how to play with other children, how to be patient and wait your turn, how to behave when you do not get what you want. Cognitively, young children need stimulation. Parents and teachers need to encourage kids to talk, not to tell them to keep quiet. Kids are naturally curious. Encourage them to wonder about the world around them, to ask questions, to explore, to express themselves. Of course, parents and family members are the first and best teachers for children, because they look after them, play with them, and talk to them most intensively. But a good preschool will help parents to make the most of these formative windows of a child’s development. Preschool is also a great practical help to working couples because nowadays, both parents are likely to be working. Not all grandparents are available to become babysitters! They like to play with their grandchildren; they do not want to be responsible for their grandchildren. Hence, five years ago, the Government moved decisively, to build up preschools, and transform the whole sector. We created almost 50,000 more childcare and kindergarten places, especially in young estates like Punggol and Sengkang. We increased subsidies to make preschools more affordable. We raised the standards of Anchor Operators. PCF Kindergartens have become Sparkletots, with better curriculum, and better trained teachers. NTUC’s My First Skool has also developed engaging programmes. Last month, I visited My First Skool’s campus in Punggol, built on top of a multi-storey carpark. It has an arts programme, coordinated by the National Arts Council. Artists visit the kindergarten to work with the kids and the kids were enjoying all sorts of art   painting, making things, doodling with coloured chalk on a blackboard. I joined them, some of them were doing better than me because I did not do art when I was in kindergarten. The Ministry of Education (MOE) has also started to run MOE Kindergartens, for children of ages five and six. They have developed a good curriculum. They have hired well-trained teachers, including mother tongue teachers   Malay, Chinese and Tamil. There are now 15 of these MOE Kindergartens. I visited one in Punggol. The kids are happy, and so their parents are happy too. Every preschool a good preschool. As you can see, we have made good progress and a big reason is that we set up the Early Childhood Development Agency (ECDA). It oversees the whole sector. It brings together all the ministries involved with preschool. It works with the operators to make sure everything goes well. Now to take the preschools to the next level, we will roll out three more initiatives. First, we will create more places. As I said earlier, in the last five years, we have already created 50,000 more preschool places. But in the young towns, we are still short. In the next five years, we will create an additional 40,000 new places. This will bring the total number of preschool places to about 200,000, which will be almost double what we had. Here is one centre, coming up in Sengkang West. The shortage of preschool places is mainly for the younger ones, ages zero to four, and those early years are critical for the child’s development. So Anchor Operators will create more places just for this group. For a start, they will build Early Years Centres in new HDB developments. We will call them Early Years Centres for zero to four. Each Early Years Centre will partner a nearby MOE Kindergarten. From zero to four, your child can go to an Early Years Centre and when he reaches five and six, he or she will have a place reserved in that MOE Kindergarten, if you want to take it up. Second, we will improve the quality of preschool education for ages five and six, in other words the K1s and the K2s. We have enough places already for this group. Now, we want to raise their standards further and MOE will lead the way. MOE’s 15 Kindergartens have helped to establish a good standard at a reasonable price and parents know they can trust the MOE brand. MOE will scale up to 50 kindergartens over the next five years. And this way, MOE can make a bigger impact beyond its own MOE Kindergartens to influence and uplift the quality of the whole sector. Thirdly, we will upgrade the preschool profession in order to attract good teachers and carers because teachers and carers make all the difference to our children. As a comparison, let us look at what we do with MOE teachers for primary, secondary and junior college. We train our teachers well. We have a National Institute of Education (NIE). Trainee teachers attend NIE programmes before they can start teaching. NIE does research and keeps up-to-date on the latest teaching methods. During their careers, periodically teachers return to NIE to upgrade themselves, and prepare for more senior roles. The MOE teachers do fulfilling work; they are paid competitively and they have good career paths. You can be a master teacher   showing other teachers how to teach. You can be a subject specialist   developing new curricula. Or you can be a school leader   like a Vice-Principal, or a Principal, or a Cluster Superintendent. Because teachers are trained well, paid well and have good career prospects, good people take up teaching and we have a good education system. We will take the same approach with preschool teachers: train them well, reward them well, and attract good, passionate people. I am glad that since we started upgrading and expanding preschools, people have been switching careers, like Ms Marie Luo. I met Marie when I visited the MOE Kindergarten at Punggol View. Marie used to work at the Ministry of Trade and Industry (MTI), overseeing productivity. But Marie loved to take care of young children, so when MOE Kindergartens started up, she gave it a try. She has been teaching for more than a year, and can look forward to a good career. To encourage more people like Marie to join the preschool sector, we will raise the standing of the profession. We will set up a new institute to train teachers well. Today, there is no centralised institute for early childhood development. Instead we have many different preschool training programmes   in Temasek Polytechnic, in Ngee Ann Polytechnic, ITE and the SEED Institute. We will now bring all these different programmes together, under a new centralised institute and we will call it the National Institute of Early Childhood Development (NIEC). The NIEC will be like NIE but it will be for preschool teachers and carers. It will provide the full range of diploma and certificate programmes for preschool professionals. It will also have the scale to develop curricula, with different specialisations, like music, art, mother tongue or special education. Within a larger fraternity, the faculty will have more opportunities for professional development and progression. We will provide training awards for students of the new institute. The awards will cover fees plus an allowance, so preschool teachers can make a strong start to their careers. We will also work with employers to ensure good career prospects and competitive pay. Over the last five years, as preschools expanded and upgraded, salaries have gone up. As we upgrade the profession, salaries need to improve further. And in particular, salaries must match career progression. Salaries have to be competitive not just for teachers, but also for the more senior specialists, supervisors and principals. Then you will have a career which is attractive, then good people will join. This will take a while but we must do it and we must get there. Only then can we upgrade the preschool sector and get good people to join and make it a career. Only then we can build a strong preschool system. So if you are passionate about kids, I hope you will consider this as a career. In summary, we are doing three things to build up preschools and give young children a good start: More places for zero to four, better quality for five to six, raising the standing of teachers and carers. These changes will benefit all preschool kids. In addition, we are making a special effort for kids from low income and vulnerable families. These kids need more support, starting even earlier. We have a programme for them: it is called KidSTART, which we piloted last year. Tan Chuan-Jin started this and is taking a close personal interest in it. Currently 400 families are on KidSTART. The mother gets onto the programme when she is expecting - ideally, in the first trimester. Even before the baby is born, trained officers visit the families at home, to support the expecting mother and impart practical skills and knowledge on health, nutrition and child development. After the baby arrives, the officers continue to coach the parents and they carry on doing so all the way through preschool. The early feedback on KidSTART has been promising. Parents say that their children are more talkative and expressive, and interact much better. And if these results are confirmed, we will scale up KidSTART to benefit more children. I have described what we are doing to develop our children in practical ways but actually, we are emphasising preschools to achieve a broader social purpose because access to affordable, quality preschools will help level the playing field for young children. Today, every child goes to a good school. We want every child to go to a good preschool so that all children, regardless of family background, have the best possible start in life. We must do this because every child counts and if we get this right, we will foster social mobility and sustain a fair and just society. So it is a practical thing that we are doing but it is a strategic goal which we are aiming for. And you heard or read ESM Goh at his National Day Dinner last night talking about this   social mobility   how important it is to make sure that we help all the kids, particularly those who do not have an advantaged background, to have a good start in life, not to be at a disadvantage, to be able to have equal opportunities just like everybody else. And we have to start early with KidSTART. We are starting before the baby is born, with preschools. And that is a strategic objective, and that is why I am talking about this subject at tonight’s National Day Rally. That is also why the Government is investing heavily in young children. We have doubled our annual spending on preschools in the last five years. In 2012, we have spent S$360 million. In 2017, this year we are spending S$840 million. So it is actually more than double. And in the next five years, we will double spending again. So by 2022, five years from now, we will be spending S$1,700 million. It is a heavy investment but worthwhile and necessary. But all this will be for nought unless young couples do your part. Please have more babies! The champions are Sengkang West families. Sengkang West’s Lam Pin Min told me very proudly he is the advisor with the most babies in his ward   1,300 last year. PA organises Embracing PArenthood events for young parents and babies. Sengkang West has so many babies that Lam Pin Min had to organise multiple Embracing PArenthood events. I did just one. So here are some of his pictures - the first group, a second group, a third group, and the fourth. I think the only person who appears in more than one picture is Lam Pin Min. I assure you all the babies are different. So please go forth and do likewise! Stay Healthy   Diabetes My second topic tonight is our health, and specifically diabetes. You may not think diabetes is a major problem, but in fact, it is very serious in Singapore, particularly so for older people but increasingly for younger Singaporeans. Generally, Singaporeans think of ourselves as being fairly healthy. After all, we live quite long but it is not just about how many years you live, the quality of life matters greatly. Let us take a close look with a live poll. You have got your Pigeonholes all set up. I would like to ask you two questions. If you are in the live audience tonight, please take out your phones and look at them and answer the questions. Are you ready? Let us begin. Question one   what is the life expectancy of Singaporeans? What do you think? Is it 80 years old, 82 years old or 84 years old? Make a guess. Well, most of you think it is 84. In fact, it is 82 but for women it is 84; for men it is 80. But at 82 years, it is among one of the longest in the world. But this is life expectancy and life expectancy includes both periods of good health and ill health. And you know from your own experience with ageing parents that when people grow old, they suffer from many ailments and frailties, often dragging on for years. Let me ask question two   on average, how many years of ill health do Singaporeans have in old age? What do you think? Four years, six years, eight years? I think most of you got it right. The answer is actually eight years   a bit shorter for men, a bit longer for women, but on average eight years. On average, we live to 82 years and out of these 82 years in old age, we experience eight years of ill health. Eight years is a long time to be in ill health. It is tough on the old folks, and on their families too, particularly the caregivers. What causes this ill health? One big reason is diabetes. And unfortunately, here compared to other developed countries, Singapore is almost world champion   just behind the US. Overall, one in nine Singaporeans have diabetes. But the prevalence increases as you age. If you are my age, over 60, three in ten Singaporeans have diabetes. And since most Singaporeans will reach 60, that is a lot of us. In fact, if you look at yourself, and at the person sitting to your left, person sitting to your right, most probably at least one of you will have diabetes one day! If you break it down by race, among the Chinese, 2.5 in 10 have diabetes; among the Malays, five in 10 have diabetes; and among the Indians, six in 10 have diabetes. These people age 60 and above. Half of Malays in my age group will have diabetes, and more than half of Indian in my age group will have diabetes, already have diabetes. This is a very serious problem, and especially for the Malays and Indians, is actually a health crisis. It is not an exaggeration. The challenge with diabetes is that in the early stages, it is an invisible disease. You do not feel sick; there are hardly any symptoms. You may not even know that you have it. But if it is not treated, over time it can become very serious. If you look at the top causes of death in Singapore, diabetes does not appear there but actually, many common causes of death can be traced back to diabetes   heart attack, stroke, kidney failure   underlying causes, pre-disposing them to diabetes because diabetes damages all these organs. You may also lose your limbs, or go blind, and for the men, if nothing else worries you, diabetes can make you impotent! And that should cause you to sit up. So what can we do? Typically, people will immediately ask: what is the Government doing? But with diabetes, the more important question is: What can I myself do, to change my lifestyle and diet, to minimise my risk? Personal choice and responsibility make all the difference to whether we get diabetes or not, and if we already have diabetes, whether we keep it under control. So what can each one of us do? Let me offer four suggestions. First, please get regular medical check-ups. Find out whether you have diabetes or are at risk. Do not take the attitude that it is better not to know. You must want to know, because if you know your condition, then you can do something about it. Take advantage of the health checks at community events or MOH’s subsidised screenings under Screen for Life programme and do not stop there. Health checks only tell you how healthy or unhealthy you are. They do not cure your illness, or make you healthier, unless you do something with the results. So if your health check shows something is amiss, please see a doctor. Take the medicine that he prescribes. Follow his advice and surely, he will ask you to exercise more, which is also my second suggestion: please exercise more. Exercise is good for you. It helps with your blood sugar, you blood pressure. It brings down your weight. It makes you feel better. But if you prefer something more fun, join a group activity. PA organises all sorts of activities every weekend   Zumba, line dancing, brisk walking, taiji. Sign up to enjoy yourself with family and friends, and keep fit and healthy at the same time. The easiest way to get more exercise is just to walk a little bit more every day. The doctor says to aim for 10,000 steps a day. But I know myself that 10,000 steps is not so easy to hit. I brisk walk for 40 minutes every morning. I walked this morning and today I checked and I am still only at 7,000 steps. The reason is because the rest of the day, I am sedentary   sitting down at the computer, either at work or at home. And many Singaporeans are like me. Only when I have community events or walkabouts, or go on jalan jalan in the evenings, then I reach the target. But let us all make an effort to walk a bit more and work it into your daily routine. For example, walk to the MRT station, rather than take the feeder bus. Some people get off the train one stop early and walk the rest of the way home, or climb the stairs instead of taking the lift. I do that: when I go to the office every morning, I climb up the stairs, and if I go out and come back I climb up the stairs again. Coming down I take the lift. Let us do it together. To motivate you, the Health Promotion Board is giving everybody in the audience today a free gift. It is a step tracker; I am wearing one. I am taking part in the 520 Million Step Challenge. Please collect yours on your way out and start walking tonight. My third piece of advice is to eat less and eat healthily. I was looking at some of my old school photos recently. And it reminded me that when I was in school, the children were not as tall, and some of us were quite scrawny. Perhaps we were not eating enough and were a little bit under-nourished. So there were school milk programmes to top up our nutrition. Today, nutrition has improved and our children are taller and better built. Some are too well-built! I cannot show you a photograph, you know what I mean. And already among 5-year olds, one in 10 are overweight. And unfortunately, often baby fat does not go away, but carries into adulthood. One reason we are putting on weight is because we are eating more. 20 years ago, we ate on average 2,100 calories every day, which is about the right amount if you are not in a very active physical life. This is 1998. Six years later, by 2004, it had gone up to 2,400 calories   that is 300 calories more and that is equivalent to two extra scoops of ice cream every day. By 2010, six years later, it had gone up to 2,600 calories   in other words, another 200 calories, which means another extra scoop of ice cream, plus topping! I have not done the latest survey yet   one should be coming in this year but I fear the trend is still up. Now, to burn off these extra 500 calories, these three extra scoops of ice cream, you need to run more than an hour every day! How many of you run more than one hour every day? Most of us do not, so obesity has gone up. The key is to be disciplined, and to make the right choices in food. Mr Lee Kuan Yew was very disciplined about his food intake. Charles Chong told me a story: Mr Lee once invited him to lunch. He was the only guest. When the main course was served, the plate came up, he looked at the plate, one and a half pieces of steak   one small steak and one half, tiny piece, one and a half portions for two people. Charles Chong was the guest, so he was served first. He did not know what to do. So out of respect for Mr Lee, he took the tiny, half piece. Mr Lee stopped him. He said, That is my piece. You take the other one! So like Mr Lee, we all can decide to eat less, and watch our weight. Unfortunately, not all of us do that. In fact, we are not just eating more, we are also eating less healthily. Lifestyles have changed. In the past, fewer wives worked. They could prepare healthy, home-cooked meals. Now, increasingly both the husband and the wife work. So families eat out more often. They buy char kway teow or roti prata at hawker centres, or increasingly fast food. But actually, even if you eat out, you can still choose healthier dishes like yong tau foo, sliced fish soup or soto ayam. There are such choices. More hawkers serve healthy alternatives   less oil, less sugar, less salt, but tasting almost as good. In fact, if you get used to it, tasting better, give it a try! If you do cook at home, make small changes, like replacing white rice with brown or mixed grain rice. I know many of us are used to eating white rice but white rice is bad for diabetes because it has very high glycaemic index (GI), which means white rice may not taste sweet. But the effect of eating white rice is almost like eating sugar and when you eat white rice, your blood sugar will shoot up. If you have always eaten white rice, brown rice may take some getting used to. Every week, my Ministers and I meet over lunch before our Cabinet meeting. We call it our PreCab Lunch. Some years ago when Khaw Boon Wan was Minister for Health, he suggested to me that we switch to brown rice because it is healthier. So I polled the Ministers for their views; everyone agreed. At least nobody objected. Ever since, I have been serving brown rice at PreCab lunches. But only very, very recently, I found out that some Ministers do not like brown rice. So when we come to PreCab Lunch, they do not have rice and they go home and they eat white rice for dinner! Now I will need another very serious Cabinet discussion to decide what we should serve at PreCab lunch. As a compromise, I am thinking of trying white rice mixed together with brown rice. It is not quite as healthy but it is better tasting than all brown rice and it is healthier than all-white rice. So at the reception later, I am going to get you to try first. I am serving mixed grain fried rice   white and brown mixed together! Let me know what you think, and if you like it, please tell my Ministers. I told Seah Kian Peng about our plans and he said that NTUC FairPrice will help me to encourage Singaporeans to eat more brown rice and healthy food. From tomorrow till the end of August, FairPrice will have special discounts and offers on healthy food, including brown rice, so do not hesitate. Let us start eating healthier today! My fourth piece of advice is to cut down on soft drinks. Soft drinks contain a lot of refined sugar, which is very bad for you. Just one can of soft drink can contain eight cubes of sugar and that is more than you need for one whole day. If you drink soft drinks every day, you are overloading your system with sugar, and significantly increasing your risk of diabetes. Our children are most at risk because soft drinks are part of their lifestyle. When a young man takes his girlfriend to the cinema, how can they not buy popcorn and soft drinks! And we are seeing more diabetes cases among young people, Type 2 Diabetes, including children, and that means big trouble for their health. That is why in our schools, tuckshops can only sell drinks containing six per cent or less of sugar. But that does not really solve the problem because the kids then just go outside the school compound and buy the full-strength sweetened drinks at the convenience store. We are scouting around for solutions. Some countries   several European ones, Mexico, Brunei   they have tried a sugar tax. UK and Chile also placed warning labels on drinks with high sugar content. But it is not clear yet if these measures work. Others have tried to limit the size of soft drinks   no more Big Gulp. But not sure that will work either because people may just buy two smaller gulps instead. Nobody has found an ideal solution yet but we are scanning the horizon and if someone comes up with one that works, we will study it and we will implement it. Meanwhile, we must do something about this problem of soft drinks and sugar. As a first step, we have got the soft drink producers to agree to reduce the sugar in all their soft drinks sold in Singapore. This will help. But ultimately, what to drink is a personal choice. The best is to drink plain water. Better still, drink PUB water. I just described four simple things each one of us can do: get a check-up, exercise more, watch your diet, cut down on sugar. It requires commitment, adjustments to our habits, our lifestyles and diet. But the payoff is large and can be done. I give you an example, Mr Song Hee Pheow, who is a taxi driver. Mr Song got a check-up through a programme started by HPB for taxi drivers   Check car, Check body. Send your taxi for servicing. While you wait, send your own body for health screening. Kill two birds with one stone! Mr Song did that and discovered he had diabetes. His health coach told him to change his lifestyle. So he decided to do something about it. Instead of taking the lift to his flat, he climbed the five floors up every day. He also started exercising regularly. He ate less fried food, and cut his sugar intake. In six months, he lost 4kg. And when he saw his doctor at the polyclinic recently, the doctor said   Your blood sugar is good, well done! If Mr Song can do it, each one of us can do it too. I am sharing this with you hoping that you will take it to heart but actually I am also preaching to myself because I also have to watch my own condition, because I have a family history of diabetes. My paternal grandmother had diabetes. Several of my uncles had diabetes. My father did not but that is probably because he watched his diet and his weight very carefully and he was extremely disciplined about exercising. And for diabetes, genes play a part but your choices make the difference, and that is why I have to be careful.I do a test for fasting blood sugar twice a year. If the reading is below six, you are okay. If it is above six, well you need to investigate further. So far, I am okay. I am below six but I am not very far below six, so I know I have to be careful. And to stay in the safe zone I weigh myself every day, and I adjust my food intake. I exercise daily. I watch what I eat and drink   wholemeal bread instead of white bread, Teh O Kosong instead of Teh. Teh Gah Dai is completely out of the question. But if the dessert is chendol, it cannot be helped; I will just take a little bit. It takes effort and discipline but it can be done. So I hope you will join me. Start today, so that we can live healthily and live well. Live Smart   Smart Nation My third topic tonight is a Smart Nation. What is Smart Nation about? Some think it is about each person owning two handphones, or learning to play Pokemon Go, or having the fastest internet connection. Others talk about e-commerce, the Internet of Things, self-driving cars, artificial intelligence and big data. Those are all part of it, but not the whole story. Smart Nation is about Singapore taking full advantage of IT. Using IT comprehensively to create new jobs, new business opportunities, to make our economy more productive, to make our lives more convenient. To make this an outstanding city in which to live, work and play. We have a natural advantage. We are compact. We are highly connected. Our people are digitally literate. Our schools are teaching students basic computing and robotics. But while we have the right ingredients, we lag behind other cities in several areas. For example, electronic payments. China has gone the furthest with e-payments. Lim Swee Say told me how he found out about this. He was in Shanghai a few years ago, queuing to buy gao lak, chestnuts, at a roadside hawker. He saw people in front of him waving their handphones before taking their chestnuts and leaving, without paying any cash. So being Singaporean, he thought there must be some special offer. When it came his turn, he confidently said, I do not need the special offer, I will pay the full price in cash! The hawker did not say anything   just gave him a quizzical look and pointed him to the QR code. And that was then he realised the QR code was for WeChat Pay and he was the suaku one! Indeed, in major Chinese cities, cash has become obsolete. Even debit and credit cards are becoming rare. Everyone is using WeChat Pay or AliPay and these apps are linked to your bank account. To pay someone money, just pick up the phone, scan his QR Code. And you can use these apps for nearly all payments. You can buy snacks from a roadside stall like Swee Say. You can see the QR code here, there is another one here. You can pay for a taxi ride. You can even tip the waiter at a restaurant. So when visitors from China find that they have to use cash here, they ask: how can Singapore be so backward? In Singapore, we too have e-payments, but we have too many different schemes and systems that do not talk to one another. So people have to carry multiple cards, and businesses have to install multiple readers. It is inconvenient for consumers, it is costly for businesses. And the result is, most of us still prefer cash and cheques   6 in 10 transactions are cash and cheques. We must simplify and integrate our systems. MAS has been working hard at this   integrating the different systems into one   so now at last we have one single unified terminal that can read different cards. MAS and the banks have also rolled out a new service, called PayNow. PayNow links your mobile phone number to your bank account. So you can pay and receive money using your mobile banking app. You can pay money to somebody’s mobile number, if he is on PayNow. And if you are on PayNow, other people can pay you just using your mobile number. So you use your app you send $20, you know his phone number, you send it to him, it pops up on his account. It is done. Different bank, notwithstanding, money goes across. Does not matter if the bank is different. In fact, you do not have to know his bank account number, or which bank he is using. Soon you will be able to use QR codes too. It is convenient, it is cheap. It is safe. There is no credit card fee. So next time I am at a hawker centre, I look forward to paying for my meal with PayNow. Then I will know it is fully working. Another area where IT can help is public safety and security. I chair the Research, Innovation and Enterprise Council. The Council met recently. The members, some Singaporeans, some international experts. I was struck by what one of the Council members said   that Singapore had a reputation as one of the safest cities in the world, so she was surprised to find that when it came to using IT to make a city safe, other cities were ahead of Singapore. I told her she was quite right. Many cities already have comprehensive CCTV and sensor networks. And they also can integrate the inputs from all the sources, analyse and make sense of the information, and respond promptly if there is an incident or an emergency. Take for example the Boston Bombings. During the Boston Marathon in April 2013, about four years ago. Terrorists detonated two bombs near the finish line. There were many casualties. The Boston Police pulled together a huge amount of information from CCTVs, social media, license plate readers, plus video and photos contributed by the public. They combed through all this data. Within three days, the police identified the two bombers, and released footage of them. And this prevented the bombers from moving around freely. The police soon tracked them down, killed one, and arrested the other. In Singapore we started building our own network of sensors, especially CCTV cameras, some time ago. The Police installed CCTVs at HDB void decks and lift landings to deal with loan sharks. And it has worked. We see fewer cases of owe-money-pay-money now. They do not paint your door but I just read in the newspapers the owe-money-pay-money has gone online now. But at least the physical harassment is down. Residents are greatly relieved. Other agencies have different sensors. PUB has sensors to detect water levels in drains to know whether it is likely to flood or not. LTA has cameras to monitor traffic conditions and deter illegal parking. The camera is quite small, and the sign is very big. Hotels, shopping centres and office buildings also have their own CCTV systems. But we have not brought all the different systems together. When the Little India riot happened in December 2013, we were caught a little flat-footed. There were too few CCTV cameras monitoring Little India. We had to rely on footage posted by the public on social media. Since then we have made progress. We are building an integrated national sensor network. We are making every lamp-post a smart lamp-post, meaning it can mount different types of sensors on any of the lampposts. We are installing more CCTV cameras in public places. We are combining inputs from different sources   police, LTA, hotels and commercial buildings, even handphones, which are effectively sensors on the ground. And we are learning to analyse this combined data, for example, using artificial intelligence to automatically flag when something unusual is happening. So if I have 10,000 cameras, I do not need 1,000 people watching those cameras. I just need maybe just 10 people. Each person can watch 1,000 cameras and if the AI detects that something funny is happening, it will pop up and the man can pay attention and a response can be directed. So one day if we have an incident like the Boston Bombings, then the Home Team can assess the situation quickly and respond promptly, or even pre-empt it from happening. This is a major project. Meanwhile, we are working on smaller projects too, to solve daily problems. Like paying for parking. We have ERP, we have mobile phones, and yet we are still tearing parking coupons, which we have been doing for 40 years. Dozens of other cities have parking apps. Why do we not? So I pushed HDB and URA to go digital. And I am glad that with GovTech’s help, we have now developed an app for street parking   and it is called parking.sg. Actually they should call it parking lady lai liao. You only pay for however long you park. If it is 18 minutes, you pay for 18 minutes, and the app will alert you when your session is about to end. If you want to park longer, you can extend your session remotely. No need to rush back to your car to add coupons before the parking lady puts a summons on you. We will launch parking.sg by October. And it will save motorists a lot of unnecessary parking hassles. These small projects are instructive. They show what can be done. They help us learn and adapt to new technology in our daily lives and they prepare us to make bigger adjustments which are necessary later on. I give you another example from the retail business. Traditional retail is being transformed by technology. People are shopping online. Delivery services like Redmart, HonestBee and Amazon Prime now have become popular. Online retailers are also setting up brick-and-mortar stores, with a twist. Like in Seattle, Amazon has just opened a drive-up grocery store. You order your items online. You choose a pick-up time, maybe when you are on your way home from work. At your allotted time, you drive to the store to do the pickup. A licence plate reader will register your arrival. Your groceries are brought out to your car. It is not available here yet, but perhaps it is not so far away. This means retail stores will not disappear. But traditional stores and businesses must adapt and reinvent themselves. Use technology to offer customers more efficient, more convenient service. I am glad to see that our businesses are responding. Supermarkets have had self-service counters for some time now, which are popular with shoppers. FairPrice has taken it further. They have opened an unmanned, cashless Cheers store at Nanyang Polytechnic. There is no cashier in this convenience store, no staff at all. To buy an item, you simply take it off the shelf, you do a self-checkout. There is a back-end system which tracks the inventory and automatically restocks when the stocks run down. It saves manpower and costs for Cheers, offers convenience and savings for customers. I hear Cheers is going to push out another store like this soon. We have got continue to push such projects. The world is changing. Unless we change with it, we will fall behind. Singapore must always stay with the leaders to attract talent and business, to live up to our own expectations of what we ought to be and can be. To do such Smart Nation projects, big or small, we need engineers, programmers, data analysts, technicians. We need people with the skills. We need managers with the understanding. We need leaders with the dare and the courage and the organisational ability to make it happen. There is a worldwide shortage of such talent and skills. But we must urgently build up our talent pool. The Government is offering scholarships and sponsorships for engineering. The SkillsFuture and Professional Conversion Programmes will help Singaporeans build up and upgrade their skills. This is one direct way a Smart Nation will create new jobs and opportunities for our people. When we started out with economic development, we put a lot of emphasis on engineering and science. In fact, when we gave scholarships, almost all the scholarships were for engineering but in the last decade, two decades, the trend has shifted. First it was balanced, from engineering to economics and liberal arts, which was a good sign. Then it went all the way and nowadays most of the students going on scholarship are not doing engineering and sciences. They are doing liberal arts, which are good subjects but I need a balance. We really must encourage our best students, a good proportion of them, to take up engineering, sciences and computer sciences. You come back, there are many things to do where your skills will be very valuable. Smart Nation is for all of us, young and old. We are making every effort to bring everyone along - through our schools, through SkillsFuture, and through PA. Many Singaporeans are using IT to create jobs or solve social problems. In my Malay speech, I described how Syafiq grew his company, and now Riverwood employs 120 workers. In my Chinese speech, you saw how Gillian Tee’s company, Homage, gave many seniors and their families’ peace of mind, by providing on-demand caregiving services. If you know about IT, you can share your knowledge with others, to help bring them up to speed, like Mr Tariam Singh. Tariam is 70. He does not look it but he is. He is a volunteer Silver Infocomm Wellness Ambassador. He helps fellow seniors learn new IT skills. Here he is in action. Tariam speaks many languages. He teaches in Hokkien, Mandarin, Malay, and Punjabi. He shows seniors how to use messaging apps and social media, to connect with family and friends. He brings people together to learn, and make friends with one another. And this is how we will become a Smart Nation together   by taking the initiative to improve ourselves; by helping others and bettering all our lives; by looking towards the future and making Singapore a happening place where people want to live. Conclusion Tonight, I have spoken about three things we are doing to build our future. Making our preschools better because we want every child to start well and have the best chance in life. Declaring war on diabetes so that we can stay well, live healthily, and enjoy the fruits of our labours. Building a Smart Nation to create opportunities for all of us and keep Singapore a leading city in the world. Why are we so preoccupied with the future? Whom are we doing this for? Not just for ourselves, but for our children and future generations. In January, I presented Edusave Awards to residents of Teck Ghee. It was a reward and an encouragement for the students who have done well and a proud moment for their parents who worked hard to support them and who encouraged them to excel. One family I met moved me. This is Adam, receiving his Edusave Merit Award. You can see the smile on his face. After the ceremony, I met Adam’s father, Aziz. He showed me an old picture. It was a photo of himself when he was a schoolboy, receiving an SBC House Union Bursary from me   we were both much younger   in 1986! So, I have presented awards to father and son, 31 years apart. It was a special moment for Aziz, for Adam, also for me. Recently, I met the grandfather, Ahmad. Ahmad was a gardener with SBC, the old Mediacorp. This is Ahmad down here, Aziz and Adam. And that is why Aziz qualified for the Bursary all those years ago! It is my Government’s duty to build for our future, so that every family can be like Ahmad, Aziz and Adam. It has ever been so, and it must always be so. In the beginning when we had little else, we had faith in our future. We believed every family should have the chance to work hard and do well, and improve their lives. We wanted every generation to outdo their parents. We strove mightily to make this happen. This family’s story started with Ahmad, who belonged to the pioneer generation. When Aziz was born, he could go to a good school and receive every help to further his education. In 1986, when Aziz received his bursary, Singapore was going through a difficult time. We were just coming out of a severe recession. We had taken drastic measures to revive the economy, including a sharp and painful CPF cut. But even as we dealt with the crisis, we never stopped building for the future. We continued to improve our schools, expand our polytechnics and universities, and upgrade the old Vocational and Industrial Training Board (VITB)   this is the Ang Mo Kio VITB, just across the road from where we are now   to build today’s ITE. Aziz could work hard and do well. He graduated with a Diploma in Mechanical Engineering from Singapore Polytechnic. He became an engineer working for semiconductor multinationals. He upgraded himself along the way and earned a Master’s Degree in Engineering Management. As the new bio-medical sciences industry took off, Aziz was headhunted to be a senior maintenance engineer with GSK, a pharmaceutical multinational. Now it is his son, Adam’s turn. Adam is growing up in a completely different world. He will need different skills to compete in the future economy. He will have to study and work hard like his father   there is no escaping that. But like his father, he too will have the opportunity to thrive. Adam is now studying in Teck Ghee Primary School. He and all his TGPS schoolmates will learn how to code and programme, and if they are keen, to do robotics and digital media. We are giving children like Adam strong foundations so that when they start working, they can take up good jobs and seize the opportunities and make this a Smart Nation. Perhaps one day Adam may launch his own startup! Why not? This is the Singapore of the last half century: Ahmad, a gardener; Aziz, a pharmaceutical engineer; and Adam, a bright future ahead of him. Every generation striving and building for the next; keeping our eyes on tomorrow and investing in our children; undaunted by challenges and disruptions, instead working together to overcome every obstacle, seize every opportunity and realise a bright future for all of us."
#data = "PM Theresa May at Davos on Brexit: Thank you Professor Schwab for that introduction, and thank you for inviting me to speak here at the World Economic Forum this morning. This is an organisation that is, as it says in the very first line of your mission statement, committed to ‘improving the state of the world’. Those of us who meet here are all   by instinct and outlook   optimists who believe in the power of public and private co-operation to make the world of tomorrow better than the world of today. And we are all united in our belief that that world will be built on the foundations of free trade, partnership and globalisation. Yet beyond the confines of this hall, those forces for good that we so often take for granted are being called into question. The forces of liberalism, free trade and globalisation that have had   and continue to have   such an overwhelmingly positive impact on our world, that have harnessed unprecedented levels of wealth and opportunity, that have lifted millions out of poverty around the world, that have brought nations closer together, broken down barriers and improved standards of living and consumer choice, forces that underpin the rules-based international system that is key to our global prosperity and security, are somehow at risk of being undermined. And as we meet here this morning, across Europe parties of the Far Left and the Far Right are seeking to exploit this opportunity, gathering support by feeding off an underlying and keenly felt sense among some people   often those on modest to low incomes living in relatively rich countries around the west   that these forces are not working for them. And those parties   who embrace the politics of division and despair; who offer easy answers; who claim to understand people’s problems and always know what and who to blame   feed off something else too: the sense among the public that mainstream political and business leaders have failed to comprehend their legitimate concerns for too long. This morning, I want to set out a manifesto for change that responds to these concerns and shows that the politics of the mainstream can deliver the change people need. I want to show how, by taking a new approach that harnesses the good of what works and changes what does not, we can maintain   indeed we can build   support for the rules-based international system. And I want to explain how, as we do so, the United Kingdom   a country that has so often been at the forefront of economic and social change   will step up to a new leadership role as the strongest and most forceful advocate for business, free markets and free trade anywhere in the world. Brexit For that is the unique opportunity that Britain now has. I speak to you this morning as the Prime Minister of a country that faces the future with confidence. For a little over 6 months ago, millions of my fellow citizens upset the odds by voting, with determination and quiet resolve, to leave the European Union and embrace the world. Let us not underestimate the magnitude of that decision. It means Britain must face up to a period of momentous change. It means we must go through a tough negotiation and forge a new role for ourselves in the world. It means accepting that the road ahead will be uncertain at times, but believing that it leads towards a brighter future for our country’s children, and grandchildren too. So while it would have been easy for the British people to shy away from taking such a path, they fixed their eyes on that brighter future and chose a bold, ambitious course instead. They chose to build a truly Global Britain. I know that this, and the other reasons Britain took such a decision, is not always well understood internationally, particularly among our friends and allies in Europe. Some of our European partners feel that we have turned our back on them. And I know many fear what our decision means for the future of the EU itself. But as I said in my speech earlier this week, our decision to leave the European Union was no rejection of our friends in Europe, with whom we share common interests and values and so much else. It was no attempt to become more distant from them, or to cease the co-operation that has helped to keep our continent secure and strong. And nor was it an attempt to undermine the European Union itself. It remains overwhelmingly and compellingly in Britain’s national interest that the EU as an organisation should succeed. It was simply a vote to restore, as we see it, our parliamentary democracy and national self-determination. A vote to take control and make decisions for ourselves. And, crucially, to become even more global and internationalist in action and in spirit too. Because that is who we are as a nation. Britain’s history and culture is profoundly internationalist. We are a European country, and proud of our shared European heritage, but we are also a country that has always looked beyond Europe to the wider world. That is why we are among the most racially diverse countries in Europe, one of the most multicultural members of the European Union, and why   whether we are talking about India, Pakistan, Bangladesh, America, Australia, Canada, New Zealand, countries in Africa, Asia or those that are closer to home in Europe   so many of us have close friends and relatives from across the world. And it is why we are by instinct a great, global, trading nation that seeks to trade with countries not just in Europe but beyond Europe too. So at the heart of the plan I set out earlier this week, is a determination to pursue a bold and ambitious free trade agreement between the UK and the European Union. But, more than that, we seek the freedom to strike new trade deals with old friends and new allies right around the world as well. I am pleased that we have already started discussions on future trade ties with countries like Australia, New Zealand and India. While countries including China, Brazil, and the Gulf States have already expressed their interest in striking trade deals with us. That is why, as I said in my speech on Tuesday, I want the UK to emerge from this period of change as a truly Global Britain   the best friend and neighbour to our European partners, but a country that reaches beyond the borders of Europe too; a country that gets out into the world to build relationships with old friends and new allies alike. And that is exactly what we are going to do. Global Britain We are going to be a confident country that is in control of its own destiny once again. And it is because of that that we will be in a position to act in this global role. Because a country in control of its destiny is more, not less able to play a full role in underpinning and strengthening the multilateral rules-based system A Global Britain is no less British because we are a hub for foreign investment. Indeed, our biggest manufacturer, Tata, is Indian   and you still can’t get more British than a Jaguar or a Land Rover. Britain is no less British because it is home to people from around the world. In fact, we derive so much of our strength from our diversity   we are a multi-racial, multi-ethnic, multi-faith democracy, and we’re proud of it. And Britain is no less British because we have led the way in multilateral organisations like the UN, NATO, IMF and the World Bank over many years. Membership of these bodies magnifies all their members’ ability to advance the common goods of peace, prosperity and security. I believe strongly in a rules based global order. The establishment of the institutions that give effect to it in the mid-20th century was a crucial foundation for much of the growing peace and prosperity the world has enjoyed since. And the tragic history of the first half of the last century reminds us of the cost of those institutions’ absence. The litany of follies of that time are mistakes that we should never forget and never repeat. So we must uphold the institutions that enable the nations of the world to work together. And we must continue to promote international co-operation wherever we can. One example of that is modern slavery   a scourge of our world, which we can only defeat if we work together, changing attitudes, rooting out such abhorrent practices and prosecuting the perpetrators. That is why at Davos this year I have convened a high-level panel discussion to continue our co-ordinated effort to save those many lives which are, tragically, being stolen. International co-operation is vital. But we must never forget that our first responsibility as governments it to serve the people. And it is my firm belief that we   as governments, international institutions, businesses and individuals   need to do more to respond to the concerns of those who feel that the modern world has left them behind. Economic reform So in Britain, we have embarked on an ambitious programme of economic and social reform that aims to ensure that, as we build this Global Britain, we are able to take people with us. A programme that aims to show how a strong Britain abroad can be a better Britain at home. Because talk of greater globalisation can make people fearful. For many, it means their jobs being outsourced and wages undercut. It means having to sit back as they watch their communities change around them. And in their minds, it means watching as those who prosper seem to play by a different set of rules, while for many life remains a struggle as they get by, but don’t necessarily get on. And these tensions and differences are increasingly exposed and exploited through the expansion of new technologies and the growth of social media. But if we are to make the case for free markets, free trade and globalisation, as we must, those of us who believe in them must face up to and respond to the concerns people have. And we must work together to shape new policies and approaches that demonstrate their capacity to deliver for all of the people in our respective countries. I believe this challenge demands a new approach from government. And it requires a new approach from business too. For government, it means not just stepping back and, as the prevailing orthodoxy in many countries has argued for so many years, not just getting out of the way. Not just leaving businesses to get on with the job and assuming that problems will just fix themselves. It means stepping up to a new, active role that backs businesses and ensures more people in all corners of the country share in the benefits of its success. And for business, it means doing even more to spread those benefits to more people. It means playing by the same rules as everyone else when it comes to tax and behaviour, because in the UK trust in business runs at just 35% among those in the lowest income brackets. And it means putting aside short-term considerations and investing in people and communities for the long-term. These are all things that I know the vast majority of businesses do already. Not just by creating jobs, supporting smaller businesses, training and developing people, but also by working to give something back to communities and supporting the next generation. Businesses large and small are the backbone of our economies, and enterprise is the engine of our prosperity. That is why Britain is   and will always be   open for business: open to investment in our companies, infrastructure, universities and entrepreneurs. Open to those who want to buy our goods and services. And open to talent and opportunities, from the arts to technology, finance to manufacturing. But, at the same time as promoting this openness, we must heed the underlying feeling that there are some companies, particularly those with a global reach, who are playing by a different set of rules to ordinary, working people. So it is essential for business to demonstrate leadership. To show that, in this globalised world, everyone is playing by the same rules, and that the benefits of economic success are there for all our citizens. This work is absolutely crucial if we are to maintain public consent for a globalised economy and the businesses that operate within it. That is why I have talked a great deal about our country delivering yet higher standards of corporate governance, to help make the UK the best place to invest of any major economy. That means several things. It means businesses paying their fair share of tax, recognising their obligations and duties to their employees and supply chains, and trading in the right way; companies genuinely investing in   and becoming part of   the communities and nations in which they operate, and abiding by the responsibilities that implies; and all of us taking steps towards addressing executive pay and accountability to shareholders. And that is why I welcome the World Economic Forum’s ‘Compact for Responsive and Responsible Leadership’ that businesses are being asked to sign up to at this conference. It is this change   setting clear rules for businesses to operate by, while embracing the liberalism and free trade that enable them to thrive   which will allow us to conserve the ultimate good that is a globalised economy. I have no doubt at all about the vital role business plays, not just in the economic life of a nation, but in society too. But to respond to that sense of anxiety people feel, I believe we   business and government working together   need to do even more to make the case. That is why in Britain, we are developing a new Modern Industrial Strategy. The term ‘industrial strategy’ has fallen into something approaching disrepute in recent years, but I believe such a strategy   that addresses the long-standing and structural weaknesses in our economy   is essential if we are to promote the benefits of free markets and free trade as we wish. Our strategy is not about propping up failing industries or picking winners, but creating the conditions where winners can emerge and grow. It is about backing those winners all the way to encourage them to invest in the long-term future of Britain. And about delivering jobs and economic growth to every community and corner of the country. We can’t leave all this to international market forces alone, or just rely on an increase in overall prosperity. Instead, we have to be practical and proactive   in other words, we have to step up and take control   to ensure free trade and globalisation work for everyone. Social reform At the same time, we have embarked on an ambitious agenda of social reform that embraces the same principles. Active, engaged government that steps up and works for everyone. Because if you are someone who is just managing, just getting by, you don’t need a government that will get out of the way. You need an active government that will step up and champion the things that matter to you. Governments have traditionally been good at identifying, if not always addressing, the problems and challenges faced by the least disadvantaged in our societies. However, the mission I have laid out for the government I lead   to make Britain a country that works for everyone   goes further. It is to build something that I have called the shared society   one that doesn’t just value our individual rights but focuses rather more on the responsibilities we have to one another. That respects the bonds that people share   the bonds of family, community, citizenship and strong institutions. And that recognises the obligations we have as citizens   obligations that make our society work. It is these bonds and obligations that make our society strong and answer our basic human need for definition and identity. And I am absolutely clear that it is the job of government to encourage and nurture the relationships, networks and institutions that provide that definition, and to correct the injustice and unfairness that divides us wherever it is found. Too often today, the responsibilities we have to one another have been forgotten as the cult of individualism has taken hold, and globalisation and the democratisation of communications has encouraged people to look beyond their own communities and immediate networks in the name of joining a broader global community. To say this is not to argue against globalisation   nor the benefits it brings   from modern travel and modern media to new products in our shops and new opportunities for British companies to export their goods to millions of consumers all around the world. But just as we need to act to address the deeply felt sense of economic inequality that has emerged in recent years, so we also need to recognise the way in which a more global and individualistic world can sometimes loosen the ties that bind our society together, leaving some people feeling locked out and left behind. Conclusion I am determined to make sure that centre-ground, mainstream politics can respond to the concerns people have today. I am determined to stand up for free markets, free trade and globalisation, but also to show how these forces can work for everyone. And to do so, I turn to the words of the 18th century philosopher Edmund Burke who said a state without the means of some change is without the means of its own conservation. That great conservative principle   change in order to conserve   is more important than ever in today’s complex geopolitical environment. And I feel it is of huge relevance to those of us here in Davos this week. And it is the principle that guides me as I lead Britain through this period of change. As we build a new, bold, confident Global Britain and shape a new era of globalisation that genuinely works for all. As we harness the forces of globalisation so that the system works for everyone, and so maintain public support for that system for generations to come. I want that to be the legacy of our time. To use this moment to provide responsive, responsible leadership that will bring the benefits of free trade to every corner of the world; that will lift millions more out of poverty and towards prosperity; and that will deliver security, prosperity and belonging for all of our people."
#data = "PM Narendra Modi on Independence Day: My dear fellow citizens, Greetings from the ramparts of the Red Fort on the auspicious occasion of Independence Day. Nation is celebrating the Independence Day with the festival of Janamashtmi today . I can see a number of Bal Kanahiyas here. We are fortunate to have in our cultural and historical heritage from a Sudarshan Chakradhari Mohan to Charkhadhari Mohan. From the ramparts of Red Fort, on behalf of our 125 crore countrymen, I bow and pay respects to all those men and women who have sacrificed their lives, who have undergone immense sufferings and made sacrifices for the independence, glory and pride of the country. Sometimes, natural calamities are a big challenge for us. A good monsoon contributes immensely to the prosperity of the country. However, due to climate change, sometimes it turns into a natural calamity. Several parts of the country faced natural calamities recently. Besides our innocent children lost their lives in a hospital. Our 125 crore countrymen stand shoulder to shoulder with them in this hour of crisis and grief. I assure the countrymen that in this hour of crisis we shall leave no stone unturned to ensure the wellbeing of all. My dear countrymen, this year is a special year for independent India. We commemorated 75 years of Quit India Movement last week. This year we are also commemorating the centenary of Champaran Satyagrah and the Sabarmati Ashram. The centenary of Lokmanya Tilak’s call Swaraj is my birth right falls this year as well. The year marks the 125th Anniversary of the Ganesh Utsav which led to the rise of community awakening through the celebrations. It inspires us to dedicate ourselves to a cause for the country. A collective resolve of the citizens was visible during 1942 to 1947 across the country which forced the British to Quit India within a span of five years. We will have to exhibit the same resolve from now on in the 70th year of independence to 2022, the 75th year of independence. We now have five years before we reach the 75th year of our independence. Our united resolve, strength and determination to work hard remembering the memory of our great patriots will help us to build an India of their dreams by 2022. Therefore, we have to take the country forward with a pledge to build a New India. We are aware of the power of the collective resolve, hard work, sacrifice and devotion of our 125 crore citizens. Lord Krishna was immensely powerful, but it is only when the milkmen came out with sticks in his support that they could lift the Govardhan parvat. When Lord Rama had to go to Lanka, the monkeys of vanar sena came to his help, Ramsetu was built and Lord Rama could reach Lanka. Then there was Mohandas Karamchand Gandhi, who empowered his countrymen to weave the fabric of independence with cotton and spinning wheel. The collective resolve and strength of the people got freedom for our country. No one is small or big. We have in our memory the story of squirrel which became an agent of change. That’s why we must remember that no one among the 125 billion people is either small or big   everyone is equal. If each one of us, irrespective of where he belongs to, strives with a new resolve, a new energy, a new strength, we can change the face of the country with our combined strength in the 75th year of our independence in 2022. It will be the New India   a secure, prosperous and strong nation. A New India where there is equal opportunity for all; where modern science and technology play an important role in bringing glory for the nation in the global arena. Our freedom movement is linked to our sentiments. We know it well that at the time of the freedom movement, the teacher who was engaged in teaching, the farmer who was tilling his land, the laborer who was working   all of them knew in their hearts that whatever they were doing, was contributing towards the independence of the country. This idea is a great source of strength. In the family, food is prepared everyday but it becomes ‘prasad’ only when it is offered to the deity. We are working, but if we do so in the spirit of working for the glory of mother India, for the divinity of mother India, if we do it to rid our countrymen of poverty, if we do it to properly weave our social fabric, if we discharge our duties with feelings towards the country, if we do it with a sense of devotion to country, if we do our work by dedicating it to the country, then the achievements will be much more. That is why we should move forward with this spirit. January 1, 2018 will not be an ordinary day   those born in this century will start turning 18. For these people, this is a decisive year of their lives. They are going to be the creator of the destiny (Bhagya Vidhata) of our nation in the 21st century. I heartily welcome all these youth, honour them and offer my respects to them. You have an opportunity to shape the destiny of our country. A proud nation invites you to become a part of its developmental journey. My dear countrymen, When Arjuna was posing a lot of queries to Lord Krishna in the battlefield of Kurukshetra, then Krishna told Arjuna that you achieve goals as per your thinking and beliefs. We have a strong determination, we are committed for a brighter India. We, who have grown up with a state of hopelessness, have to reject the feeling of despair and must now march ahead with confidence. We have to leave this ‘Chalta Hai’ attitude. We have to think of ‘Badal Sakta Hai’- this attitude will help us as a nation. We should have this confidence that with sacrifice and hard work and a resolve to do something, we will get necessary resources and the ability to do it and then a big transformation will happen, and our resolve will convert into accomplishment. Brothers and sisters, It is natural for our countrymen to think of our safety and security. Our country, our military, our brave-hearts, our uniformed forces, be it any force, not just the army, air force or navy, all the uniformed forces, whenever they were called upon to do so, they showed their valour, their strength; our brave-hearts never turned their back in making the supreme sacrifice. Be it Left-Wing-Extremism, be it terrorism, be it infiltrators, be it the elements fomenting trouble inside our country- the uniformed forces of our country have made extreme sacrifices. And when the surgical strike was carried out, then the world had to acknowledge India’s capability and strength. My dear countrymen, India’s security is our priority. Be it our coast line or our borders, the space or the cyberspace, India is capable of ensuring its own security and strong enough to ward off any threats against the country. My beloved countrymen, Those who have looted the nation and looted the poor are not able to sleep peacefully today. Due to this, the confidence of the hardworking and honest person is increasing. An honest man now feels that his honesty matters. Today, we are celebrating a festival of honesty and there is no place for dishonesty. This gives us new hope. The law against Benami property was lying in limbo for years. Now, we have come up with a law against Benami Property. In such a short time, the government has confiscated Benami properties worth over Rs.800 crores. When these things happen, the common man develops faith that the country is for the honest persons. The ‘One Rank   One Pension’ policy for our defence forces was stuck up for 30-40 years. Our government implemented it. When we fulfill the aspirations of our soldiers, then their morale goes high and their resolve to protect the nation grows manifold. The country has many states and a Central Government. GST has shown the spirit of cooperative federalism and has given competitive cooperative federalism a new strength. The success of GST can be attributed to the hard work that has been put in to make it a success. Technology has made it look like a miracle. The global community is surprised how we were able to roll out GST in such a short time. It is a reflection of our capability and helps in building the confidence and faith of the future generations. New systems are emerging. Today roads are being built at twice the speed. The railway tracks are being laid at double the speed. More than 14 thousand villages that were hitherto shrouded in darkness even after Independence, have been provided with electricity. Bank accounts of 29 crore people have been opened, more than 9 crore farmers have got Soil Health Card. More than 2 crore poor mothers and sisters are no longer using the fuel wood and using LPG gas stove NOW. The poor tribals have gained faith in the system. The person at the last mile of development now joins the mainstream and the nation is marching ahead. Loans worth more than 8 crore have been sanctioned without guarantee to the youth for self-employment. The interest on bank loans have been lowered. Inflation is under control. If a person belonging to the middle-class wants to build his house, then he gets a loan at a lower rate of interest. In this way, the country is moving forward and people are joining the movement. The times have changed. The government is determined to do all it says, like doing away with the interview process. In the labour sector alone, even a small businessman had to fill up to 50-60 forms, but we have now made it more convenient by reducing it to just 5-6 forms. I can provide many such instances of Good Governance, simplifying the process of Governance. By reiterating this we have enforced faster decision making. And that is why 125 crore countrymen have been able to repose faith in our governance. Dear Countrymen, India today has gained in stature across the globe. You will be happy to know that we are not alone in our fight against terrorism. Many nations are proactively supporting us. Be it Hawala, or any inputs related to terrorism, the global community is supporting us with critical information. We have joined hands with other nations in the combined fight against terrorism. I extend my heartfelt greetings to all those countries who are standing in solidarity with us and acknowledge our prowess. For the development and progress of Jammu & Kashmir, its prosperity, and fulfilment of the aspirations of the citizens, not only the J&K government, but we as responsible citizens also have a responsibility. We are committed to restore the state to its former glory so as to experience the paradise that it was, once again. On Kashmir there is rhetoric and politics. But I am clear in my belief on how to win the war against separatism, which is spread by a handful of people. The problem will be solved neither by abuse or bullets   it will be solved by embracing all Kashmiris. Such is the legacy of 125 crore Indians. Hence, neither by abuse nor by bullets, change will come by embracing all. And we are moving forward with this resolve. We shall take strict steps against terrorism. There is no question of being soft on terrorism or terrorists. We have been asking the extremists to join the mainstream. Democracy provides an equal opportunity and rights to be heard for all. The process of engagement can take place only by their joining the mainstream. I deeply appreciate the efforts of security forces in curbing the Left Wing extremism which mobilized a lot of youth from these regions to surrender and joined the mainstream. Security forces are keeping a tight vigil on our borders. I am delighted to announce that today the Indian Government is launching a website which will give an account of the valour of the Gallantry Award winners. A portal is also being launched which aims to provide the entire details of these brave hearts who have brought pride to the nation. The story of sacrifices made will certainly inspire the younger generations. With the help of technology, we are trying to promote honesty and transparency in the country. Our fight against black money will continue; our fight against corruption will continue. With the intervention of technology, slowly we are trying to link Aadhaar with the system. We have been successful in infusing transparency in the system. People from around the world have appreciated this model and have been studying it. A common man living thousands of kilometres away can now supply his products to the Government. He doesn’t need any middle man. We have launched a portal called GEM. The Government is making procurement through this portal. We have been successful in bringing transparency at various levels. Brothers and Sisters, Implementation of Government schemes are gaining momentum. When a work is delayed, it is not only the project that is delayed. It is not a matter of expenditure of money. When a work is stalled, the poor families are the ones who suffer the most. We can reach the planet Mars within 9 months; we are capable of achieving that. I review Government Projects every month. A specific project came to my notice. It was a 42-year old project. 70-72 kilometres of Railway lines were to be laid, but the project was lying in limbo for the last 42 years. My brothers and sisters, When a country that is capable of reaching Mars within 9 months, how is it possible that it is not able to lay 70-72 Kilometres railway lines for 42 long years. It sets in doubts in the minds of the poor. We have taken care of all these things. We have been striving to bring in the changes through technology. Whether it is Geo-Technology or Space-Technology, we have tried to link all these technologies to bring in transformation. You must have seen that there was a time when tension prevailed between the Centre and the States for Urea and Kerosene. The Centre was like an elder brother while the State was treated like a younger brother. I have been a Chief Minister for a long time and I know the importance of states in the development of the country. I know the importance of the Chief Ministers and the State Governments. Hence , we laid emphasis on Co-operative Federalism. Now we are moving towards a Competitive Co-operative Federalism. You must have seen that we are taking all the decisions together. You may recall that one of our Prime Ministers in his address from the ramparts of Red Fort had talked about the poor state of affairs of the power supply companies of the country. He expressed his concerns over the issue. Today with the UDAY Yojana, we have worked together to resolve the issues with these power companies. It is a concrete example of federalism in its real sense. Whether it is GST or Smart City Project, whether it is Swachh Bharat Abhiyan, or construction of toilets, or ease of doing business, all these are being accomplished by working together shoulder to shoulder with the States. My dear fellow countrymen, In the New India, the biggest strength is democracy. But we have reduced our democracy to mere ballot boxes. However, democracy cannot be confined only to ballot boxes. Hence our resolve is to see to it that we have a democracy in New India, where people are not driven by the system, rather the system is driven by the people. Such a democracy should become the identity of New India and we want to move towards that direction. Lokmanya Tilak Ji had said Swaraj is my birthright. In Independent India our Mantra should be Good Governance is my birthright. ‘Suraja’ or Good Governance should be our collective responsibility. The citizens should fulfil their duties and the Government too should discharge its responsibilities. When we move from Swaraj to Suraja, the citizens do not lag behind. For example, when I called upon the countrymen to give up gas subsidy, the nation responded as a whole. I have spoken about cleanliness. Now people are joining hands from every part of the country to take up this cleanliness drive forward. When demonetization was announced, the world was surprised. People thought that this was the end of Modi. But the way our 125 crore countrymen had shown patience and faith, we were able to take one after another step in our drive against corruption. With this new practice of people’s participation, our effort to take the country forward with people’s involvement will help us to reach our goal. My dear fellow countrymen, Lal Bahadur Shastri had given the slogan Jai Jawan Jai Kisan. Our farmers have not looked back since then. They are now producing record harvests and scaling new heights despite facing natural adversities. There has been a record production of pulses this year. My dear brothers and sisters, India never had the tradition to import pulses and if it had to import on rare occasions, it was only a few thousand tons. This year when they produced 16 lakh tons of pulses to provide nutrition to the poor, the government took the historic step of encouraging them by buying their produce. Pradhan Mantri Fasal Bima Yojna has provided a security cover to our farmers. Three years back, this scheme which was running under a different name earlier, covered only 3.25 crore farmers. Now over a short span of three years more farmers have been brought under its ambit. The number is soon going to touch the 5.75 crore mark. Pradhan Mantri Krishi Sinchai Yojna aims to meet the demand of water of the farmers. If my farmers get water, they can produce rich harvests from their fields. That is why I had made some announcements from the ramparts of Red Fort on the last Independence Day. Out of those, we have completed 21 projects and remaining 50 would be completed soon. I have resolved to complete 99 big projects. By completing those 99 big projects before 2019, we shall fulfill our commitment. We cannot change the lot of our farmers until we hand hold him from procuring seeds to ensure that his produce reaches the market. For this we need infrastructure and supply chain. Every year vegetables, fruits and grains worth lakhs of crores of rupees are wasted. In order to change the situation, the government has encouraged Foreign Direct Investment in food processing sector. Government of India has launched ‘Pradhan Mantri Kisan Sampada Yojna’ to encourage infrastructure building. Due to this, systems shall be put in place that will ensure hand-holding of the farmers from supplying seeds to marketing his produce. Such arrangements will usher in a new change in the lives of crores of our farmers. With the changes in demand and technology, the nature of jobs in our country has been changing. The Government has launched several new initiatives in the employment related schemes and also in the manner in which the training is imparted for the development of human resource according to the needs of the 21st century. We have launched a massive program to provide collateral free loans to the youth. Our youth should become independent, he should get the employment, he should become the provider of employment. Over the past three years, ‘Pradhanmantri Mudra Yojana’ has led to millions and millions of youth becoming self-dependent. It’s not just that, one youth is providing employment to one, two or three more people. In the education sector, we have taken an important step to grant the universities freedom from restrictions to make them the world class universities. We have asked 20 universities to decide their own destiny. The government will not interfere in their functioning. Besides, the government is willing to provide funds up to Rs. 1,000 crores. We have appealed to them and I am confident that the education institutions of our country will certainly come forward and make it a success. Over the last three years, we have set up 6 IITs, 7 new IIMs and 8 new IIITs and we have also done the groundwork for linking the education with job opportunities. My mothers and sisters, the women in the families seek employment in large numbers. And therefore we have taken a very important step to reform the labour laws to provide them the employment opportunities in the night also. Our mothers and sisters are an integral unit of our families. Their contribution in making our future is extremely important. And that is why we have decided to increase the paid maternity leave to 26 weeks from the earlier 12 weeks. In the context of women empowerment, I’d like to honour those sisters who have been forced to lead a very difficult life due to ‘Triple Talaq’. There is no recourse for them, and such victims of ‘Triple Talaq’ have launched a massive movement in the country. They shook the conscience of the intellectual class of the country, the media of the country also helped them and a movement against ‘Triple Talaq’ was started in the country. I heartily commend those sisters who have launched this movement, who have been fighting against the ‘Triple Talaq’ and I’m confident that the country will help them in this struggle. The country will help these mothers and sisters in securing this right. India will fully support them and they will ultimately achieve success in this very important step in the direction of women empowerment; I am fully confident about this. My dear countrymen, Sometimes in the name of faith, some people due to lack of patience end up destroying the social fabric. The country is governed by peace, harmony and unity. The poison of casteism and communalism can never benefit the country. This is the land of Gandhi and Buddha, we have to move forward taking everyone along. This is a part of our country’s culture and tradition. We have to successfully carry it forward, and that is why in the name of faith, violence cannot be allowed. If something happens to a patient in a hospital and the hospital is burnt down; some accident happens and the vehicles are burnt down; people launch a movement and the public property is burnt down; is it for a free India? It is the property of 125 crore Indians. Whose cultural legacy is this? This is our cultural legacy, the legacy of 125 crore people. Whose faith is this? This is our faith, the faith of 125 crore people; and that is why the path of violence in the name of faith can never succeed in this country. The country will never accept this. And that is why I’d like to appeal to all the countrymen that at that time our motto was ‘Bharat Chodo’ and today’s slogan is ‘Bharat Jodo’. We have to take everyone and every section of the society along to take the country forward. In order to build a prosperous India, we need a strong economy, balanced development and next generation infrastructure. Only then, we can realise the India of our dreams. Brothers and sisters, We have taken innumerable decisions over the past three years. Some have been noticed, while the rest may not have been noticed. But one thing is important   when you move towards big changes, you will face hurdles. But look at the working style of this government; when a train crosses a railway station and changes the track, the speed needs to be reduced from 60 to 30. The speed of the train decreases while changing the track. We are trying to put the whole country on a new track without decreasing the speed. We have maintained the speed. We may have brought a number of new laws and systems such as the GST, but we have successfully accomplished the task and the work goes on. We have laid emphasis on infrastructure. We have invested lot of funds for building the infrastructure   from modernization of the railway station in a small town, to build an Airport, to expand water-ways or roadways, to form a gas-grid or a water-grid or to lay down an optical fibre network. We are emphasising on all types of modern infrastructure. My dear countrymen, The prosperity of Eastern India is required for India’s forward movement into the 21st century. It has tremendous potential, rich human resources, immense natural wealth, work force and it has the power to change lives. We are devoting significant attention to eastern India- Bihar, Assam, West Bengal, Odisha, and the Northeast. These parts have to grow further. These are abundant in natural resources and are striving hard to take the country to new heights. Brothers and sisters, Making India corruption-free is an important task and we are trying to provide impetus to it. After forming the government, our first task was to constitute an SIT. Today, after three years, I want to tell the countrymen proudly that we have confiscated black money worth Rs. 1.25 lakh crores, the culprits would be brought to book and forced to surrender. Demonetization followed this move. We have achieved several milestones through demonetization. The hidden black money has been brought into the formal economy. You must have seen that we used to extend the days from 7 days to 10 days to 15 days; sometimes we allowed the old notes at the petrol pump, at the chemist shops, and sometimes at the railway stations, because our intention was to bring all money into the formal banking system and we achieved success in completing the task. According to the research conducted by outside experts, about Rs.3 lakh crores that had never come into the banking system before, has been brought into the system after the demonetization. More than Rs. 1.75 lakh crores deposited in the banks is under the scanner. Black money worth Rs. 2 lakh crore had to be deposited in the banks and this system has forced them to be accountable. It has also stopped the flow of black money. The number of new taxpayers filing income tax returns from April 01 to August 05 is 56 lakhs while in the same period last year only 22 lakh filed the returns.. In a way it has more than doubled. This is the result of our fight against black money. Over 18 lakh people have been identified, whose income is much higher than what their declared income and therefore they have to clarify this. Around 4.5 lakh people have now come forward and are trying to trade the right path after accepting their mistakes. One lakh people, who had neither heard of Income Tax nor paid Income Tax, have now been forced to do so. Brothers and sisters, In our country, we tend to indulge in endless debates and discussions following the shutdown of companies. People start speculating about economic meltdown and what not. You will be surprised to know that black marketers used to own shell companies. Post demonetization, the reports from data mining astonishingly revealed that there are 3 lakh shell companies dealing in Hawala transactions. Can anyone imagine? Out of these 3 lakh shell companies, registration of 1.75 lakh companies were cancelled. Even if five companies shut shop in India, there is huge public outcry. And here, we have closed one lakh Seventy five thousand companies. The looters of nation’s wealth will have to answer. We have accomplished this task. You will be surprised to know that there are some shell companies operating from a single address. As many as 400 companies have been found to be operating from one address. There was no one to question them. There was a total collusion. Therefore, Brothers and sisters, I have waged a big war against corruption and black money. We are fighting corruption   for the bright future of India and the wellbeing of our people. Brothers and sisters, We have taken several steps and I am sure that post GST, it will increase further and bring in transparency. Today a truck driver on an average is able to save 30% of his travel time from one point to another after GST was introduced. Hundreds of crores of rupees have been saved due to removal of check posts. This amounts to 30% increase in his efficiency. Can you imagine what it means to accomplish 30% more efficiency in the Indian transportation sector? GST has brought this revolutionary change. My dear countrymen, Today due to demonetization, banks have enough liquidity. Banks are reducing their interest rates. Even a common man is able to access inflow of funds through Mudra. He is getting opportunities to stand on his feet. The middle class and the underprivileged who aspire to have their own homes someday, are getting loans from the banks at low interest rates. Such moves are giving a fillip to the economic growth of the country. My dear countrymen, Time has changed. We are in the 21st Century. Our country boasts of the world’s largest youth population. India is known in the world for its prowess in IT and Digital World. Should we still remain in the old mind-set? There was a time when leather coins were in vogue, but gradually they ceased to exist. Today we have paper-currency. Slowly this paper-currency will be replaced by the digital currency. We should lead from the front to make a move towards Digital transactions. We should adopt BHIM App for transactions and make it a part of our economic activities. We should also work through prepaid systems. I am happy that there has been an increase in Digital transactions. It has increased by 34% compared to last year while the prepaid transactions have gone up by 44%. We should move towards a less-cash economy. My Dear Countrymen, some schemes of the Government are meant for ensuring savings for the common man. If you use LED bulbs, you can save Rs.2000 to Rs. 5000 per year. If we succeed in Swachh Bharat, poor will be able to save Rs. 7000 on medicines that they would otherwise have spent. Curbing inflation has helped people in saving money in a way. Cheap medicines through Jan Aushadhi Kendras are a boon to the poor. There used to be a lot of expenditure on operations and stents. We are striving to make it possible for knee operations as well. We are striving to minimize this expenditure for the poor and the middle class. Earlier there used to be Dialysis only at the State capitals. Now we have decided to open Dialysis Centres at district level. We have already opened this facility in 350 to 400 districts where free Dialysis services are available for the poor. We can take pride in the fact that we have developed various systems to showcase to the world. We have been able to develop the ‘NAVIC Navigation System’ through GPS. We have helped the neighbouring countries by successfully launching SAARC Satellite. We asserted our supremacy in the world by introducing Tejas aircraft. BHIM Aadhaar App for digital transaction has come as a surprise to the world. Crores of RuPay Cards are now available in the country. When all the cards will be operational, it will be the largest number of cards in the world. My dear countrymen, I shall urge you to take up the New India Pledge and move ahead. Our scriptures say,  It implies that, if we don’t accomplish a work within the stipulated time then we shall not be able to get the desired results. So for the ‘Team India’, for the 125 crore countrymen, we have to take the pledge to achieve the goal by 2022. We will do it with dedication to see a great, majestic India by 2022.   So we will together build such an India where the poor will have Pucca House with electricity and water. Together we will build such an India where the farmers can sleep without worry. They will earn double by 2022 of what they are earning today. Together we will build such an India where the youth, the women will have enough opportunities to fulfil their dreams. Together we will build such an India which will be free from terrorism, communalism and casteism. Together we will build such an India where nobody will compromise with corruption and nepotism. Together we will build such an India which will be clean and healthy, and where the dream of Su-Raj will be fulfilled. And that is why my dear fellow countrymen, we will together strive to move ahead in this march towards development. Let all of us march together with the dream of a building an India of grandeur and majesty as we wait for the 75th year of independence after completing 70 years of Independence in the next five years. With this thought in mind I once again bow my head before the heroes of our freedom movement. I bow my head before the new confidence and passion of my 125 crore countrymen and I call upon Team India to march on with this new pledge. With this thought I convey my hearty congratulations to all of you"
#data = "PM Netanyahu at UN: We’re in the midst of a great revolution, a revolution in Israel’s standing among the nations. This is happening because so many countries around the world have finally woken up to what Israel can do for them. Those countries now recognize what brilliant investors like Warren Buffet and great companies like Google and Intel, what they’ve recognized and known for years: that Israel is the innovation nation   the place for cutting-edge technology in agriculture, in water, in cyber security, in medicine, in autonomous vehicles   you name it, we’ve got it. Those countries now also recognize Israel’s exceptional capabilities in fighting terrorism. In recent years, Israel has provided intelligence that has prevented dozens of major terrorist attacks around the world. We have saved countless lives. You may not know this, but your governments do, and they are working closely together with Israel to keep your countries safe and your citizens safe. I stood here last year on this podium and I spoke about this profound change in Israel’s standing in the world and just look at what has happened since, in one year: hundreds of presidents, prime ministers, foreign ministers and other leaders have visited Israel, many for the first time. Of these many visits, two were truly historic. In May, President Trump became the first American president to include Israel in his first visit abroad. President Trump stood at the Western Wall, at the foot of the Temple Mount, where the Jewish people’s Temples stood for nearly a thousand years. When the president touched those ancient stones, he touched our hearts forever. In July, Prime Minister Modi became the first Indian prime minister to visit Israel. You may have seen the pictures. We were on a beach in Hadera. We rode together in a jeep outfitted with a portable desalination device that some thriving Israeli entrepreneur invented. We took off our shoes, waded into the Mediterranean and drank seawater that had been purified only a few minutes earlier. We imagined the endless possibilities for Israel, India, for all humanity. In the past year, Israel hosted so many world leaders, and I had the honor of representing my country on six different continents. One year. Six continents. I went to Africa, where I saw Israeli innovators increasing crop yields, turning air into water, fighting AIDS. I went to Asia, where we deepened our relations with China and with Singapore, and expanded our cooperation with our Muslim friends in Azerbaijan and Kazakhstan. I went to Europe, where in London and Paris, Thessaloniki and Budapest, we enhanced our security and economic ties. I went to Australia, becoming the first Israeli prime minister to visit our great allies Down Under. And just last week, I went to South America, visiting Argentina and Colombia, and then I went on to Mexico, becoming, if you can believe it, the first Israeli prime minister ever to visit Latin America. After 70 years, the world is embracing Israel, and Israel is embracing the world. One year. Six continents. Now, it’s true. I haven’t yet visited Antarctica, but one day I want to go there too because I’ve heard that penguins are also enthusiastic supporters of Israel. You laugh, but penguins have no difficulty recognizing that some things are black and white, are right and wrong. Unfortunately, when it comes to UN decisions about Israel, that simple recognition is too often absent. It was absent last December when the Security Council passed an anti-Israel resolution that set back the cause of peace. It was absent last May, when the World Health Organization adopted   you have to listen to this: the World Health Organization adopted a Syrian-sponsored resolution that criticized Israel for health conditions on the Golan Heights. As the great John McEnroe would say, You can-not be serious! I mean, this is preposterous. Syria has barrel-bombed, starved, gassed and murdered hundreds of thousands of its own citizens and wounded millions more, while Israel has provided lifesaving medical care to thousands of Syrian victims of that very same carnage. Yet who does the World Health Organization criticize? Israel. So is there no limit to the UN’s absurdities when it comes to Israel? Well, apparently not, because in July, UNESCO declared the Tomb of the Patriarchs in Hebron a Palestinian World Heritage site. That’s worse than fake news. That’s fake history. Mind you, it’s true that Abraham, the father of both Ishmael and Isaac, is buried there, but so too are Isaac, Jacob, Sarah, Rebecca   Sarah is a Jewish name, by the way   and Leah, who just happen to be patriarchs and matriarchs of the Jewish people. You won’t read about that in the latest UNESCO report. But if you want to, you can read about that in a somewhat weightier publication   it’s called the Bible. I highly recommend it. I hear it even got 4 ½ out of 5 stars on Amazon. And it’s a great read. I read it every week. Ladies and gentlemen, a moment to be serious: Despite the absurdities, despite the repetition of these farcical events, there is change. Slowly but surely, there are signs of positive change, even at the United Nations.  Mr. Secretary General, I very much appreciate your statement that denying Israel’s right to exist is anti-Semitism, pure and simple. Now, that’s important, because for too long the epicenter of global anti-Semitism has been right here at the UN. And while it may take many years, I am absolutely confident that the revolution in Israel’s ties with individual nations will ultimately be reflected in this hall of nations. I say that because there is also a marked change in the position of some of our key friends. Thanks to President Trump’s unequivocal support for Israel in this body, that positive change is gathering force. So, thank you, President Trump. Thank you for supporting Israel at the UN. And thank you for your support, Ambassador Nikki Haley. Thank you for speaking the truth about Israel. But, ladies and gentlemen, here at the UN, we must also speak the truth about Iran, as President Trump did so powerfully this morning. Now, you know I’ve been ambassador to the UN and I’m a long-serving Israeli prime minister, so I’ve listened to countless speeches in this hall, but I can say this: none were bolder, none more courageous and forthright than the one delivered by President Trump today. President Trump rightly called the nuclear deal with Iran, he called it an embarrassment. Well, I couldn’t agree with him more. And here’s why: Iran vows to destroy my country every day, including by its chief of staff the other day. Iran is conducting a campaign of conquest across the Middle East and Iran is developing ballistic missiles to threaten the entire world. Two years ago, I stood here and explained why the Iranian nuclear deal not only doesn’t block Iran’s path to the bomb, Iran’s nuclear program has what’s called a sunset clause. Let me explain what that term means: It means that in a few years, those restrictions will be automatically removed   not by a change in Iran’s behavior, not by a lessening of its terror or its aggression. They’ll just be removed by a mere change in the calendar. And I warned that when that sunset comes, a dark shadow will be cast over the entire Middle East and the world, because Iran will then be free to enrich uranium on an industrial scale, placing it on the threshold of a massive arsenal of nuclear weapons. That’s why I said two years ago that the greater danger is not that Iran will rush to a single bomb by breaking the deal, but that Iran will be able to build many bombs by keeping the deal. Now, in the last few months, we’ve all seen how dangerous even a few nuclear weapons can be in the hands of a small rogue regime. Now imagine the danger of hundreds of nuclear weapons in the hands of a vast Iranian Islamist empire, with the missiles to deliver them anywhere on earth. I know there are those who still defend the dangerous deal with Iran, arguing that it will block Iran’s path to the bomb. Ladies and gentlemen, That’s exactly what they said about the nuclear deal with North Korea, and we all know how that turned out. Unfortunately, if nothing changes, this deal will turn out exactly the same way. That’s why Israel’s policy regarding the nuclear deal with Iran is very simple: Change it or cancel it, fix it or nix it. Nixing the deal means restoring massive pressure on Iran, including crippling sanctions, until Iran fully dismantles its nuclear weapons capability. Fixing the deal requires many things, among them inspecting military and any other site that is suspect, and penalizing Iran for every violation. But above all, fixing the deal means getting rid of the sunset clause. And beyond fixing this bad deal, we must also stop Iran’s development of ballistic missiles and roll back its growing aggression in the region. I remember we had these debates. As you know, I took a fairly active role in them. And many supporters of the nuclear deal naively believed that it would moderate Iran. It would make it a responsible member, so they said, of the international community. Well as you know, I strongly disagreed. I warned that when the sanctions on Iran would be removed, Iran would behave like a hungry tiger unleashed, not joining the community of nations, but devouring nations, one after the other. And that’s precisely what Iran is doing today. From the Caspian Sea to the Mediterranean, from Tehran to Tartus, an Iranian curtain is descending across the Middle East. Iran spreads this curtain of tyranny and terror over Iraq, Syria, Lebanon and elsewhere, and it pledges to extinguish the light of Israel. Today, I have a simple message for Ayatollah Khamenei, the dictator of Iran: The light of Israel will never be extinguished. Those who threaten us with annihilation put themselves in mortal peril. Israel will defend itself with the full force of our arms and the full power of our convictions. We will act to prevent Iran from establishing permanent military bases in Syria for its air, sea and ground forces. We will act to prevent Iran from producing deadly weapons in Syria or in Lebanon for use against us. And we will act to prevent Iran from opening new terror fronts against Israel along our northern border. As long as Iran’s regime seeks the destruction of Israel, Iran will face no fiercer enemy than Israel. But I also have a message today for the people of Iran: You are not our enemy; you are our friends. Shomaah doosteh mah hasteed [You are our friends]. One day, my Iranian friends, you will be free from the evil regime that terrorizes you, hangs gays, jails journalists, tortures political prisoners, and shoots innocent women like Neda Sultan, leaving her choking on her own blood on the streets of Tehran. I have not forgotten Neda. I am sure you haven’t too. And when that day of liberation finally comes, the friendship between our two ancient peoples will surely flourish once again. Ladies and gentlemen, Israel knows that in confronting the Iranian regime, we are not alone. We stand shoulder to shoulder with those in the Arab world who share our hopes for a brighter future. We’ve made peace with Jordan and Egypt, whose courageous President Abdel-Fattah el-Sissi I met here last night. I appreciate President el-Sissi’s support for peace, and I hope to work closely with him and other leaders in the region to advance peace. Israel is committed to achieving peace with all our Arab neighbors, including the Palestinians. Yesterday, President Trump and I discussed this, all of this, at great length. I appreciate President Trump’s leadership, his commitment to stand by Israel’s side, his commitment to advance a peaceful future for all. Together, we can seize the opportunities for peace and together we can confront the great dangers of Iran. The remarkable alliance between the United States and Israel has never been stronger, never been deeper. Israel is deeply grateful for the support of the Trump administration, the American Congress and the American people. Ladies and gentlemen, in this year of historic visits and historic anniversaries, Israel has so much to be grateful for. A hundred and twenty years ago, Theodor Herzl convened the First Zionist Congress to transform our tragic past into a brilliant future by establishing the Jewish state. One hundred years ago, the Balfour Declaration advanced Herzl’s vision by recognizing the right of the Jewish people to a national home in our ancestral homeland. Seventy years ago, the United Nations further advanced that vision by adopting a resolution supporting the establishment of a Jewish state. And 50 years ago, we reunited our eternal capital Jerusalem, achieving a miraculous victory against those who sought to destroy our state. Theodor Herzl was our modern Moses   and his dream has come true. We’ve returned to the Promised Land, revived our language, in-gathered our exiles and built a modern, thriving democracy. Tomorrow evening, Jews around the world will celebrate Rosh Hashanah, the beginning of our new year. It’s a time of reflection, and we look back with wonder at the miraculous rebirth of our nation, and we look ahead with pride to the remarkable contributions Israel will continue to make to all nations. You look around you and you will see those contributions every day   in the food you eat, the water you drink, the medicines you take, the cars you drive, the cell phones you use, and in so many other ways that are transforming our world. You see it in the smile of an African mother in a remote village, who, thanks to an Israeli innovation, no longer must walk eight hours a day to bring water to her children. You see it in the eyes of an Arab child, who was flown to Israel to undergo a life-saving heart operation. And you see it in the faces of the people in earthquake-stricken Haiti and Nepal who were rescued from the rubble and given new life by Israeli doctors. As the prophet Isaiah said, I have made you a light unto the nations, bringing salvation to the ends of the earth. Today, 2,700 years after Isaiah spoke those prophetic words, Israel is becoming a rising power among the nations. And at long last, its light is shining across the continents, bringing hope and salvation to the ends of the earth. Happy New Year, Shanah Tovah from Israel. Thank you"
#data = "PM Winston Churchill: Sinews of Peace 1946: I am glad to come to Westminster College this afternoon, and am complimented that you should give me a degree. The name  Westminster  is somehow familiar to me. I seem to have heard of it before. Indeed, it was at Westminster that I received a very large part of my education in politics, dialectic, rhetoric, and one or two other things. In fact we have both been educated at the same, or similar, or, at any rate, kindred establishments. It is also an honour, perhaps almost unique, for a private visitor to be introduced to an academic audience by the President of the United States. Amid his heavy burdens, duties, and responsibilities-unsought but not recoiled from-the President has travelled a thousand miles to dignify and magnify our meeting here to-day and to give me an opportunity of addressing this kindred nation, as well as my own countrymen across the ocean, and perhaps some other countries too. The President has told you that it is his wish, as I am sure it is yours, that I should have full liberty to give my true and faithful counsel in these anxious and baffling times. I shall certainly avail myself of this freedom, and feel the more right to do so because any private ambitions I may have cherished in my younger days have been satisfied beyond my wildest dreams. Let me, however, make it clear that I have no official mission or status of any kind, and that I speak only for myself. There is nothing here but what you see. I can therefore allow my mind, with the experience of a lifetime, to play over the problems which beset us on the morrow of our absolute victory in arms, and to try to make sure with what strength I have that what has been gained with so much sacrifice and suffering shall be preserved for the future glory and safety of mankind. The United States stands at this time at the pinnacle of world power. It is a solemn moment for the American Democracy. For with primacy in power is also joined an awe inspiring accountability to the future. If you look around you, you must feel not only the sense of duty done but also you must feel anxiety lest you fall below the level of achievement. Opportunity is here now, clear and shining for both our countries. To reject it or ignore it or fritter it away will bring upon us all the long reproaches of the after-time. It is necessary that constancy of mind, persistency of purpose, and the grand simplicity of decision shall guide and rule the conduct of the English-speaking peoples in peace as they did in war. We must, and I believe we shall, prove ourselves equal to this severe requirement. When American military men approach some serious situation they are wont to write at the head of their directive the words  over-all strategic concept.  There is wisdom in this, as it leads to clarity of thought. What then is the over-all strategic concept which we should inscribe today? It is nothing less than the safety and welfare, the freedom and progress, of all the homes and families of all the men and women in all the lands. And here I speak particularly of the myriad cottage or apartment homes where the wage-earner strives amid the accidents and difficulties of life to guard his wife and children from privation and bring the family up in the fear of the Lord, or upon ethical conceptions which often play their potent part. To give security to these countless homes, they must be shielded from the two giant marauders, war and tyranny. We all know the frightful disturbances in which the ordinary family is plunged when the curse of war swoops down upon the bread-winner and those for whom he works and contrives. The awful ruin of Europe, with all its vanished glories, and of large parts of Asia glares us in the eyes. When the designs of wicked men or the aggressive urge of mighty States dissolve over large areas the frame of civilised society, humble folk are confronted with difficulties with which they cannot cope. For them all is distorted, all is broken, even ground to pulp. When I stand here this quiet afternoon I shudder to visualise what is actually happening to millions now and what is going to happen in this period when famine stalks the earth. None can compute what has been called  the unestimated sum of human pain.  Our supreme task and duty is to guard the homes of the common people from the horrors and miseries of another war. We are all agreed on that. Our American military colleagues, after having proclaimed their  over-all strategic concept  and computed available resources, always proceed to the next step-namely, the method. Here again there is widespread agreement. A world organisation has already been erected for the prime purpose of preventing war, UNO, the successor of the League of Nations, with the decisive addition of the United States and all that that means, is already at work. We must make sure that its work is fruitful, that it is a reality and not a sham, that it is a force for action, and not merely a frothing of words, that it is a true temple of peace in which the shields of many nations can some day be hung up, and not merely a cockpit in a Tower of Babel. Before we cast away the solid assurances of national armaments for self-preservation we must be certain that our temple is built, not upon shifting sands or quagmires, but upon the rock. Anyone can see with his eyes open that our path will be difficult and also long, but if we persevere together as we did in the two world wars-though not, alas, in the interval between them-I cannot doubt that we shall achieve our common purpose in the end. I have, however, a definite and practical proposal to make for action. Courts and magistrates may be set up but they cannot function without sheriffs and constables. The United Nations Organisation must immediately begin to be equipped with an international armed force. In such a matter we can only go step by step, but we must begin now. I propose that each of the Powers and States should be invited to delegate a certain number of air squadrons to the service of the world organisation. These squadrons would be trained and prepared in their own countries, but would move around in rotation from one country to another. They would wear the uniform of their own countries but with different badges. They would not be required to act against their own nation, but in other respects they would be directed by the world organisation. This might be started on a modest scale and would grow as confidence grew. I wished to see this done after the First World War, and I devoutly trust it may be done forthwith. It would nevertheless be wrong and imprudent to entrust the secret knowledge or experience of the atomic bomb, which the United States, Great Britain, and Canada now share, to the world organisation, while it is still in its infancy. It would be criminal madness to cast it adrift in this still agitated and un-united world. No one in any country has slept less well in their beds because this knowledge and the method and the raw materials to apply it, are at present largely retained in American hands. I do not believe we should all have slept so soundly had the positions been reversed and if some Communist or neo-Fascist State monopolised for the time being these dread agencies. The fear of them alone might easily have been used to enforce totalitarian systems upon the free democratic world, with consequences appalling to human imagination. God has willed that this shall not be and we have at least a breathing space to set our house in order before this peril has to be encountered: and even then, if no effort is spared, we should still possess So formidable a superiority as to impose effective deterrents upon its employment, or threat of employment, by others. Ultimately, when the essential brotherhood of man is truly embodied and expressed in a world organisation with all the necessary practical safeguards to make it effective, these powers would naturally be confided to that world organisation. Now I come to the second danger of these two marauders which threatens the cottage, the home, and the ordinary people-namely, tyranny. We cannot be blind to the fact that the liberties enjoyed by individual citizens throughout the British Empire are not valid in a considerable number of countries, some of which are very powerful. In these States control is enforced upon the common people by various kinds of all-embracing police governments. The power of the State is exercised without restraint, either by dictators or by compact oligarchies operating through a privileged party and a political police. It is not our duty at this time when difficulties are so numerous to interfere forcibly in the internal affairs of countries which we have not conquered in war. But we must never cease to proclaim in fearless tones the great principles of freedom and the rights of man which are the joint inheritance of the English-speaking world and which through Magna Carta, the Bill of Rights, the Habeas Corpus, trial by jury, and the English common law find their most famous expression in the American Declaration of Independence. All this means that the people of any country have the right, and should have the power by constitutional action, by free unfettered elections, with secret ballot, to choose or change the character or form of government under which they dwell; that freedom of speech and thought should reign; that courts of justice, independent of the executive, unbiased by any party, should administer laws which have received the broad assent of large majorities or are consecrated by time and custom. Here are the title deeds of freedom which should lie in every cottage home. Here is the message of the British and American peoples to mankind. Let us preach what we practise – let us practise what we preach. I have now stated the two great dangers which menace the homes of the people: War and Tyranny. I have not yet spoken of poverty and privation which are in many cases the prevailing anxiety. But if the dangers of war and tyranny are removed, there is no doubt that science and co-operation can bring in the next few years to the world, certainly in the next few decades newly taught in the sharpening school of war, an expansion of material well-being beyond anything that has yet occurred in human experience. Now, at this sad and breathless moment, we are plunged in the hunger and distress which are the aftermath of our stupendous struggle; but this will pass and may pass quickly, and there is no reason except human folly or sub-human crime which should deny to all the nations the inauguration and enjoyment of an age of plenty. I have often used words which I learned fifty years ago from a great Irish-American orator, a friend of mine, Mr. Bourke Cockran.  There is enough for all. The earth is a generous mother; she will provide in plentiful abundance food for all her children if they will but cultivate her soil in justice and in peace.  So far I feel that we are in full agreement. Now, while still pursuing the method of realising our overall strategic concept, I come to the crux of what I have travelled here to Say. Neither the sure prevention of war, nor the continuous rise of world organisation will be gained without what I have called the fraternal association of the English-speaking peoples. This means a special relationship between the British Commonwealth and Empire and the United States. This is no time for generalities, and I will venture to be precise. Fraternal association requires not only the growing friendship and mutual understanding between our two vast but kindred Systems of society, but the continuance of the intimate relationship between our military advisers, leading to common study of potential dangers, the similarity of weapons and manuals of instructions, and to the interchange of officers and cadets at technical colleges. It should carry with it the continuance of the present facilities for mutual security by the joint use of all Naval and Air Force bases in the possession of either country all over the world. This would perhaps double the mobility of the American Navy and Air Force. It would greatly expand that of the British Empire Forces and it might well lead, if and as the world calms down, to important financial savings. Already we use together a large number of islands; more may well be entrusted to our joint care in the near future. The United States has already a Permanent Defence Agreement with the Do-minion of Canada, which is so devotedly attached to the British Commonwealth and Empire. This Agreement is more effective than many of those which have often been made under formal alliances. This principle should be extended to all British Commonwealths with full reciprocity. Thus, whatever happens, and thus only, shall we be secure ourselves and able to work together for the high and simple causes that are dear to us and bode no ill to any. Eventually there may come-I feel eventually there will come-the principle of common citizenship, but that we may be content to leave to destiny, whose outstretched arm many of us can already clearly see. There is however an important question we must ask ourselves. Would a special relationship between the United States and the British Commonwealth be inconsistent with our over-riding loyalties to the World Organisation? I reply that, on the contrary, it is probably the only means by which that organisation will achieve its full stature and strength. There are already the special United States relations with Canada which I have just mentioned, and there are the special relations between the United States and the South American Republics. We British have our twenty years Treaty of Collaboration and Mutual Assistance with Soviet Russia. I agree with Mr. Bevin, the Foreign Secretary of Great Britain, that it might well be a fifty years Treaty so far as we are concerned. We aim at nothing but mutual assistance and collaboration. The British have an alliance with Portugal unbroken since 1384, and which produced fruitful results at critical moments in the late war. None of these clash with the general interest of a world agreement, or a world organisation; on the contrary they help it.  In my father’s house are many mansions.  Special associations between members of the United Nations which have no aggressive point against any other country, which harbour no design incompatible with the Charter of the United Nations, far from being harmful, are beneficial and, as I believe, indispensable. I spoke earlier of the Temple of Peace. Workmen from all countries must build that temple. If two of the workmen know each other particularly well and are old friends, if their families are inter-mingled, and if they have  faith in each other’s purpose, hope in each other’s future and charity towards each other’s shortcomings -to quote some good words I read here the other day-why cannot they work together at the common task as friends and partners? Why cannot they share their tools and thus increase each other’s working powers? Indeed they must do so or else the temple may not be built, or, being built, it may collapse, and we shall all be proved again unteachable and have to go and try to learn again for a third time in a school of war, incomparably more rigorous than that from which we have just been released. The dark ages may return, the Stone Age may return on the gleaming wings of science, and what might now shower immeasurable material blessings upon mankind, may even bring about its total destruction. Beware, I say; time may be short. Do not let us take the course of allowing events to drift along until it is too late. If there is to be a fraternal association of the kind I have described, with all the extra strength and security which both our countries can derive from it, let us make sure that that great fact is known to the world, and that it plays its part in steadying and stabilising the foundations of peace. There is the path of wisdom. Prevention is better than cure. A shadow has fallen upon the scenes so lately lighted by the Allied victory. Nobody knows what Soviet Russia and its Communist international organisation intends to do in the immediate future, or what are the limits, if any, to their expansive and proselytising tendencies. I have a strong admiration and regard for the valiant Russian people and for my wartime comrade, Marshal Stalin. There is deep sympathy and goodwill in Britain-and I doubt not here also-towards the peoples of all the Russias and a resolve to persevere through many differences and rebuffs in establishing lasting friendships. We understand the Russian need to be secure on her western frontiers by the removal of all possibility of German aggression. We welcome Russia to her rightful place among the leading nations of the world. We welcome her flag upon the seas. Above all, we welcome constant, frequent and growing contacts between the Russian people and our own people on both sides of the Atlantic. It is my duty however, for I am sure you would wish me to state the facts as I see them to you, to place before you certain facts about the present position in Europe. From Stettin in the Baltic to Trieste in the Adriatic, an iron curtain has descended across the Continent. Behind that line lie all the capitals of the ancient states of Central and Eastern Europe. Warsaw, Berlin, Prague, Vienna, Budapest, Belgrade, Bucharest and Sofia, all these famous cities and the populations around them lie in what I must call the Soviet sphere, and all are subject in one form or another, not only to Soviet influence but to a very high and, in many cases, increasing measure of control from Moscow. Athens alone-Greece with its immortal glories-is free to decide its future at an election under British, American and French observation. The Russian-dominated Polish Government has been encouraged to make enormous and wrongful inroads upon Germany, and mass expulsions of millions of Germans on a scale grievous and undreamed-of are now taking place. The Communist parties, which were very small in all these Eastern States of Europe, have been raised to pre-eminence and power far beyond their numbers and are seeking everywhere to obtain totalitarian control. Police governments are prevailing in nearly every case, and so far, except in Czechoslovakia, there is no true democracy. Turkey and Persia are both profoundly alarmed and disturbed at the claims which are being made upon them and at the pressure being exerted by the Moscow Government. An attempt is being made by the Russians in Berlin to build up a quasi-Communist party in their zone of Occupied Germany by showing special favours to groups of left-wing German leaders. At the end of the fighting last June, the American and British Armies withdrew westwards, in accordance with an earlier agreement, to a depth at some points of 150 miles upon a front of nearly four hundred miles, in order to allow our Russian allies to occupy this vast expanse of territory which the Western Democracies had conquered. If now the Soviet Government tries, by separate action, to build up a pro-Communist Germany in their areas, this will cause new serious difficulties in the British and American zones, and will give the defeated Germans the power of putting themselves up to auction between the Soviets and the Western Democracies. Whatever conclusions may be drawn from these facts-and facts they are-this is certainly not the Liberated Europe we fought to build up. Nor is it one which contains the essentials of permanent peace. The safety of the world requires a new unity in Europe, from which no nation should be permanently outcast. It is from the quarrels of the strong parent races in Europe that the world wars we have witnessed, or which occurred in former times, have sprung. Twice in our own lifetime we have seen the United States, against their wishes and their traditions, against arguments, the force of which it is impossible not to comprehend, drawn by irresistible forces, into these wars in time to secure the victory of the good cause, but only after frightful slaughter and devastation had occurred. Twice the United States has had to send several millions of its young men across the Atlantic to find the war; but now war can find any nation, wherever it may dwell between dusk and dawn. Surely we should work with conscious purpose for a grand pacification of Europe, within the structure of the United Nations and in accordance with its Charter. That I feel is an open cause of policy of very great importance. In front of the iron curtain which lies across Europe are other causes for anxiety. In Italy the Communist Party is seriously hampered by having to Support the Communist-trained Marshal Tito’s claims to former Italian territory at the head of the Adriatic. Nevertheless the future of Italy hangs in the balance. Again one cannot imagine a regenerated Europe without a strong France. All my public life I have worked for a Strong France and I never lost faith in her destiny, even in the darkest hours. I will not lose faith now. However, in a great number of countries, far from the Russian frontiers and throughout the world, Communist fifth columns are established and work in complete unity and absolute obedience to the directions they receive from the Communist centre. Except in the British Commonwealth and in the United States where Communism is in its infancy, the Communist parties or fifth columns constitute a growing challenge and peril to Christian civilisation. These are sombre facts for anyone to have to recite on the morrow of a victory gained by so much splendid comradeship in arms and in the cause of freedom and democracy; but we should be most unwise not to face them squarely while time remains. The outlook is also anxious in the Far East and especially in Manchuria. The Agreement which was made at Yalta, to which I was a party, was extremely favourable to Soviet Russia, but it was made at a time when no one could say that the German war might not extend all through the summer and autumn of 1945 and when the Japanese war was expected to last for a further 18 months from the end of the German war. In this country you are all so well-informed about the Far East, and such devoted friends of China, that I do not need to expatiate on the situation there. I have felt bound to portray the shadow which, alike in the west and in the east, falls upon the world. I was a high minister at the time of the Versailles Treaty and a close friend of Mr. Lloyd-George, who was the head of the British delegation at Versailles. I did not myself agree with many things that were done, but I have a very Strong impression in my mind of that situation, and I find it painful to contrast it with that which prevails now. In those days there were high hopes and unbounded confidence that the wars were over, and that the League of Nations would become all-powerful. I do not see or feel that same confidence or even the same hopes in the haggard world at the present time. On the other hand I repulse the idea that a new war is inevitable; still more that it is imminent. It is because I am sure that our fortunes are still in our own hands and that we hold the power to save the future, that I feel the duty to speak out now that I have the occasion and the opportunity to do so. I do not believe that Soviet Russia desires war. What they desire is the fruits of war and the indefinite expansion of their power and doctrines. But what we have to consider here to-day while time remains, is the permanent prevention of war and the establishment of conditions of freedom and democracy as rapidly as possible in all countries. Our difficulties and dangers will not be removed by closing our eyes to them. They will not be removed by mere waiting to see what happens; nor will they be removed by a policy of appeasement. What is needed is a settlement, and the longer this is delayed, the more difficult it will be and the greater our dangers will become. From what I have seen of our Russian friends and Allies during the war, I am convinced that there is nothing they admire so much as strength, and there is nothing for which they have less respect than for weakness, especially military weakness. For that reason the old doctrine of a balance of power is unsound. We cannot afford, if we can help it, to work on narrow margins, offering temptations to a trial of strength. If the Western Democracies stand together in strict adherence to the principles of the United Nations Charter, their influence for furthering those principles will be immense and no one is likely to molest them. If however they become divided or falter in their duty and if these all-important years are allowed to slip away then indeed catastrophe may overwhelm us all. Last time I saw it all coming and cried aloud to my own fellow-countrymen and to the world, but no one paid any attention. Up till the year 1933 or even 1935, Germany might have been saved from the awful fate which has overtaken her and we might all have been spared the miseries Hitler let loose upon mankind. There never was a war in all history easier to prevent by timely action than the one which has just desolated such great areas of the globe. It could have been prevented in my belief without the firing of a single shot, and Germany might be powerful, prosperous and honoured to-day; but no one would listen and one by one we were all sucked into the awful whirlpool. We surely must not let that happen again. This can only be achieved by reaching now, in 1946, a good understanding on all points with Russia under the general authority of the United Nations Organisation and by the maintenance of that good understanding through many peaceful years, by the world instrument, supported by the whole strength of the English-speaking world and all its connections. There is the solution which I respectfully offer to you in this Address to which I have given the title  The Sinews of Peace.  Let no man underrate the abiding power of the British Empire and Commonwealth. Because you see the 46 millions in our island harassed about their food supply, of which they only grow one half, even in war-time, or because we have difficulty in restarting our industries and export trade after six years of passionate war effort, do not suppose that we shall not come through these dark years of privation as we have come through the glorious years of agony, or that half a century from now, you will not see 70 or 80 millions of Britons spread about the world and united in defence of our traditions, our way of life, and of the world causes which you and we espouse. If the population of the English-speaking Commonwealths be added to that of the United States with all that such co-operation implies in the air, on the sea, all over the globe and in science and in industry, and in moral force, there will be no quivering, precarious balance of power to offer its temptation to ambition or adventure. On the contrary, there will be an overwhelming assurance of security. If we adhere faithfully to the Charter of the United Nations and walk forward in sedate and sober strength seeking no one’s land or treasure, seeking to lay no arbitrary control upon the thoughts of men; if all British moral and material forces and convictions are joined with your own in fraternal association, the high-roads of the future will be clear, not only for us but for all, not only for our time, but for a century to come."
#data = "President World Bank Jim Yong Kim at annual meeting: Chairman Fakhoury, Madame Lagarde, Governors, my dear friends Jim and Elaine Wolfensohn, Ministers, Friends, It’s an honor to address you, five years after I first stood before you in Tokyo. Each year, we gather for these Annual Meetings to address the most critical challenges that affect the lives of billions around the world. And each year, we chart the course of our collective efforts to make development work for everyone, to ensure a dignified life for all. This is a critical time for the work of the World Bank Group. The good news is that global growth is robust   2.7 percent this year. The second quarter of last year saw the highest quarterly growth rates since 2010. Trade is picking up. But investment remains weak, and we’re concerned that downside risks such as a rise in protectionism, policy uncertainty, or possible financial market turbulence could derail this fragile recovery. That’s why now is the time for all countries to take action on the needed reforms to grow their economies and compete in what will surely be a more complex, demanding, and digitized future. We’re also meeting again at a time when multiple crises are in full swing, or looming: Conflict, pandemics, climate change, and famine are impacting people all over the world, and they are contributing to an historic number of people being forcibly displaced; Countries in nearly every region are turning inward; International and home-grown terrorism affects every corner of the world. It often feels like our increasingly interconnected world is in fact falling apart and countries and peoples are pulling away from each other. Amidst this turbulence, organizations like the World Bank Group must step forward and help to build new foundations for human solidarity. We are part of the post-1945 world order that was predicated on the notion that what affects one city, one country, one region can have immediate and lasting impacts on us all. These new foundations for human solidarity must move us from the old construct of donors and recipients into a new development compact between stakeholders. In 2013, we announced our goals to end extreme poverty by 2030, and boost shared prosperity among the poorest 40 percent around the world. And a year ago, I explained the three ways we will get there: by accelerating inclusive, sustainable economic growth; by building resilience to shocks and threats; and by investing more   and more effectively   in people. For us, this is an especially important time for tackling global poverty, because we have more room to take bold action to grow the economy, protect countries from major overlapping crises, and invest in people. Before giving you examples of our progress on all of these fronts, I want to thank the dedicated staff of the World Bank Group, who work tirelessly to reach our ambitious goals. And I’d like to applaud our clients and shareholders, who have made tremendous efforts under adversity to achieve their highest aspirations. The first pillar of our strategy to end poverty and boost shared prosperity is to accelerate inclusive, sustainable economic growth. We know that official development assistance will not be enough to meet the 4 trillion dollars per year needed to achieve the sustainable development goals, and meet the world’s rising aspirations. Last April, ahead of the Spring Meetings, I called for a new approach where we maximize finance for development by systematically crowding in private sector investment and making it work for developing countries and poor people. We developed the Joint Principles for Crowding in Private Sector Finance, and the G20 endorsed those principles last summer. This is a great example of close collaboration between management and the Board, to launch a fundamental shift in our approach to development finance. Maximizing Finance for Development isn’t based on ideology, nor is it a panacea for all development challenges. It’s an evidence-based approach where we ask a very straightforward question: How can we maximize the resources that developing countries have to do the things they need to do for their people, while minimizing the burden of public debt?   Maximizing Finance for Development means finding win-win solutions, where investors get a good return, and countries utilize these resources to meet their development goals. We’re putting this approach to work with teams from across the World Bank Group, and we’ve already seen great results. Three years ago in Egypt, subsidies for energy reached 6.6 percent of GDP   more than the government spent on health, education, and social protection combined. When Egypt wanted to reform its energy sector, the World Bank Group designed a comprehensive package: IBRD contributed technical assistance and analytical experts, and a 3 billion dollar loan over 3 years for policy reforms; IFC lent 645 million dollars to the private sector; And with 210 million dollars of risk insurance from MIGA, the joint efforts of IFC and MIGA mobilized 2 billion dollars in private investment in Egypt’s Photovoltaic Solar Feed-in Tariff program. The policy reforms, initial private investment, and risk insurance helped attract 15 banks and 20 different investors for a large Photovoltaic Solar Park Project, and many of these investors will contribute to future projects. The effort helped attract more than 15 billion dollars of private investment in Egypt’s gas sector. As a result of their efforts to reduce fossil fuel subsidies and other reforms, the Egyptian government has increased their fiscal space by around 14 billion dollars a year. This has allowed the government to roll out two new cash transfer programs that reach 1.7 million poor Egyptians; food subsidies for the poorest have increased by 300 percent; and the government expanded its school lunch program. We’ve learned some valuable lessons from this effort in Egypt, most importantly, that we could be more proactive in creating markets   and not just simply wait for them to appear on their own. And we’ve shown that it can work to finance sectors other than infrastructure, such as helping transform the health sector in Turkey. In 2002, the infant mortality rate in Turkey was far higher than other OECD countries, and the life expectancy was years lower. So the government started an ambitious overhaul of the entire health system, which included advice and loans from IBRD. Turkey launched a public-private partnership in 2010; IBRD provided a 134 million dollar loan and technical support; IFC invested 241 million dollars, mobilizing an additional 540 million dollars in private investment; and MIGA provided political risk guarantees. One of the flagship projects was the Elazig Integrated Health Campus   a 400 million euro, thousand-bed hospital that was financed by the first project bond issuance under the Turkish PPP program. MIGA and the EBRD worked with Rönesans Holding   a Turkish construction group, and Meridiam   a French infrastructure investor to develop an innovative credit-enhancement application that enabled the project to be financed in the bond market. They secured a Moody’s investment grade rating two notches above Turkish sovereign bonds. These investments were relatively small, but very important parts of a large program that transformed Turkey’s health sector, considerably increasing access and improving public health. In 2002, less than two-thirds of Turkey’s population had health insurance, and just over half had regular access to health care. The Health Transformation Program has led to nearly universal access   98 percent of people in Turkey are covered by affordable health insurance. The program has also led to dramatic improvements in health across Turkey. Infant mortality rates were cut in half, life expectancy increased from 71 to 74 years, and the mortality of children under 5 fell 55 percent. These are just two examples of many that we are now scaling and spreading around the world. There’s never been a better time to crowd in private sector investment. Right now, there’s more than 10 trillion dollars invested in negative interest rate bonds; 24 trillion dollars in low-yield government securities; and 5 trillion dollars sitting in cash, waiting for better investment with higher returns. Last year at Davos, President Xi Jinping of China said that the global market system is the ocean we all swim in and cannot escape from. He said, Any attempt to channel the waters in the ocean back into isolated lakes and creeks is simply not possible. Maximizing Finance for Development is our best chance to make the global market system work for everyone. We need to embrace the notion that our greatest moral responsibility is to create equality of opportunity. The Maximizing Finance for Development approach gives countries the resources to build bridges, solar parks, and hospitals. It frees up funds for schools, job training, and social safety nets, and creates win-win solutions that can grow economies and give everyone opportunity   for a good education, for a stable job, for a chance to achieve their highest aspirations. The second pillar of our strategy is to build resilience to overlapping shocks and crises, and one of the most critical is climate change. When it comes to climate change, we’re running out of time   2016 was once again the hottest year since record keeping began. The last three years each broke the previous record. Worldwide, more extreme natural disasters force 26 million people into poverty every year. We have to reduce our carbon footprint and help countries adapt to natural disasters, like the recent hurricanes in the Caribbean and devastating floods in South Asia. Last year, the World Bank Group loaned 12.8 billion dollars for climate investments, 22 percent of our portfolio. We’re now the largest funder of climate-related investments in the developing world, and we’re on track to meet our goal of having 28 percent of our portfolio deliver climate benefits by 2020. We’re also using our convening power to scale up climate action by bringing the public and private sectors together. There’s 23 trillion dollars of potential investments just in the Paris commitments of 21 emerging economies   including green buildings, sustainable transport, renewable energy, and energy efficiency. We need climate investments in the trillions, not billions of dollars. Around 90 trillion dollars will be invested in infrastructure over the next 15 years, just to replace aging infrastructure in advanced economies and meet expected growth in emerging economies. Right now, infrastructure investment tops out at around 3.4 trillion dollars annually, but the need is more like 6 trillion dollars a year. And all of this must be climate-smart, low-carbon, resilient infrastructure. With the UN and France, we’re co-hosting the Paris Climate Summit on December 12, to connect investors with climate-smart investment opportunities. Along with climate change, we have to do more to help refugees   and to help the countries and people who are providing the world with a global public good by hosting them. A year ago, in the midst of the refugee crisis, we established a special fund that has provided 200 million dollars in grants and unlocked more than 1 billion dollars in concessional financing for Jordan and Lebanon. Lebanon, which is hosting more than 1.5 million displaced Syrians, has the highest per-capita ratio of refugees in the world. And Jordan, as the Chairman said, hosts more refugees than any country in the world. In both countries, 81 percent of Syrian refugees are under age 35, and 70 percent are poor. This instrument, now known as the Global Concessional Financing Facility, is making a significant impact. In Jordan, the GCFF has helped provide 50,000 formal working permits to Syrian refugees. In Lebanon, this fund is financing roads and employment programs, creating more than a million days’ worth of labor in direct and indirect jobs. It is also getting kids out of the streets and into schools, to provide education and to prevent the tragedy of a lost generation. This year, we also took a large step to break the cycle of panic and neglect around pandemics. Too often, we neglect infectious disease outbreaks until they become a global threat   and then we quickly forget about them after the threat subsides. With the Pandemic Emergency Financing Facility, for the first time, we have actual pandemic insurance   a 450 million dollar policy that will automatically disburse funds to the poorest countries when an epidemic reaches a critical stage.   This is the first time that pandemic risk in low-income countries is being shared with the financial markets. Why can’t we do the same thing for famine, or other humanitarian crises? I believe we can, and we’re working on it right now.   The third pillar of our strategy is to invest more   and more effectively   in people. For most of my adult life, I worked to provide care to people suffering from terrible diseases in some of the poorest parts of the world. And for most of my adult life, my colleagues and I have been advocating for more investments in people. Usually we made a moral argument   that everyone deserves a chance to reach their highest aspirations. Providing health, education, and social protection is one of the most powerful ways of giving everyone a chance. We always knew that investing in people is the right thing to do; now we’re learning that, economically, it may very well be the smartest thing to do. Over the past year, we’ve done several different analyses, and we’re finding that investments in human beings   especially in their health, education, and social protection   are far more powerfully correlated with economic growth than we ever thought. Some of our economists at the World Bank Group are working with Chris Murray and the Institute for Health Metrics and Evaluation, a group at the University of Washington that works closely with the Gates Foundation. We asked them to use their powerful analytic tools to look at the relationship between economic growth and improvements in human capital. I’m using the term human capital because I want to make the point to the ministers of finance who are here today: when you invest in human beings, you’re putting in place the capital you need to grow your economies. Too often, we still hear from leaders, First we’ll grow our economies, then we’ll invest in our people. Investing in people is investing in economic growth. Briefly, here’s what we found: If you look at the top quartile   the top 25 percent of countries that have improved human capital the most, and you compare them with the bottom 25 percent   countries that have improved human capital the least   the difference is enormous. We looked at the 25 years between 1991 and 2016   the difference in economic growth was 1.25 percent of GDP each year over 25 years. We need to do more work and more research, but this suggests that looking backwards, investments in human beings have had a huge impact on economic growth. And looking forward, surely investing in people will become more important in the increasingly digital economy of the future. Some studies estimate that as many as 65 percent of primary school children today will work in jobs or fields that don’t yet exist. We know that countries have had to make tough choices on how to spend scarce public funds. But we believe, and the evidence suggests, that the more   and more effectively   you invest in health, education, and social protection, the better you’ll do as an economy. This idea has been around for some time. But with better data, greater transparency in sharing that data, and new, more powerful analytic tools, we’re now understanding that the relationship between human capital and economic growth could be far more profound than we ever imagined. Soon, we’ll publish a document called The Changing Wealth of Nations, and for the first time, we’re looking at human capital as part of the overall wealth of nations. It turns out that more than 65 percent of the wealth of all nations in the world is in human capital. In rich countries, human capital is a much larger proportion of overall wealth; in low-income countries, it’s a much smaller proportion. So developing countries have a long way to go. We desperately need these investments now, because we’re facing several human capital crises: Globally, 155 million children are stunted; 400 million people around the world lack access to essential health services; 100 million each year fall into poverty because of catastrophic health expenditures; Only one-third of the world’s poor are covered by social safety nets. Unless we act decisively, by 2030, 167 million children will still be living in extreme poverty. We have to fundamentally change the way world invests in building human capital. Last week, I announced the Human Capital Project   an accelerated effort to help countries invest more   and more effectively   in their people. The Human Capital Project will involve three efforts:We will accelerate innovative and results-based financing for human capital investments. We will continue to dig into the data linking investments in people to economic growth: we’re looking for new patterns and new answers. And we’re forming a broad coalition that includes all stakeholders   other MDBs, international financial institutions, governments, civil society organizations, and the private sector. Eventually, the Project will include rankings: one focusing on the stock of human capital, and another where we measure the flow   the investments countries are making today to build human capital. We’re trying to create conditions where heads of state and finance ministers cannot resist investing in their people. We’re trying to create an environment where investing in people is not only the moral thing to do, but in fact it’s something they absolutely must do   with great urgency to drive economic growth.  This will be controversial. But I feel that we have a moral responsibility to reveal to our shareholders the powerful relationship between investing in people and economic growth. And more importantly, we’re ready to help every single country rapidly accelerate the quality and quantity of their investments in people. To accomplish all of these things   to deliver what countries need at the scale you expect of us   we need more resources. Over the years, we have proven our exceptional value for money   19 billion dollars of total paid-in capital for IBRD and IFC, since the beginning of this institution, has yielded: More than 900 billion dollars of financing; 50 billion dollars of reserves; And 28 billion dollars in transfers to IDA and other programs.   With enormous development needs and rising aspirations, demand is overwhelming. Since the 2008 financial crisis, IBRD has almost doubled its lending portfolio. IFC grew its loan portfolio threefold, and its equity portfolio fivefold, over the last 10 years. If we don’t enhance our financial capacity, IBRD would have to dramatically shrink its annual lending commitments by one-third. IFC would have to cut back its investments in IDA countries and fragile and conflict-affected states, as well as its equity investments, making the ability to create markets in the world's most difficult settings unattainable. We have done a lot of good work with all of you. Many countries have evolved, progressed, and grown their way to new levels of development. We need to recognize that. And we need to open a dialogue with all of you, so that the resources of the World Bank Group keep getting channeled to where they make the most difference in achieving our twin goals. And with a capital increase, we can fulfill our commitment to having a financially strong World Bank Group that possesses the ability to provide finance at a level that will meet the aspirations of our client countries.   I feel a tremendous sense of urgency, not just because there are enormous needs in the world. Aspirations are rising. And aspirations, linked to opportunity, can breed dynamism and inclusive, sustainable economic growth. But if there’s no opportunity to achieve one’s aspirations, rising frustration may very well lead countries down the path of fragility, conflict, violence, extremism, and eventually migration. We can play a critical role in finding win-win solutions, where we maximize financing for development, and create opportunities for the owners of capital to make higher returns. We can play a critical role, using our full range of financial tools to protect countries from overlapping crises. And we can play a critical role, by helping countries make more   and more effective   investments in their people. In five years as president of the World Bank Group, despite all of the troubles in the world, I’ve never been more optimistic that we can help people lift themselves out of poverty; that we can build new foundations of human solidarity. But we must act, as the great Martin Luther King said, with the fierce urgency of now, knowing that, as he said, there is such a thing as being too late. If we commit to these pillars, If we invest the right resources, If we act with the fierce urgency that these times require   I believe that we can be the first generation in history to end poverty on the face of the earth."
#data = "President Juncker EU State of the Union: Mr President, Honourable Members of the European Parliament, When I stood before you this time last year, I had a somewhat easier speech to give. It was plain for all to see that our Union was not in a good state. Europe was battered and bruised by a year that shook our very foundation. We only had two choices. Either come together around a positive European agenda or each retreat into our own corners. Faced with this choice, I argued for unity. I proposed a positive agenda to help create   as I said last year   a Europe that protects, a Europe that empowers, a Europe that defends. Over the past twelve months, the European Parliament has helped bring this agenda to life. We continue to make progress with each passing day. Just last night you worked to find an agreement on trade defence instruments and on doubling our European investment capacity. And you succeeded. Thank you for that. I also want to thank the 27 leaders of our Member States. Days after my speech last year, they welcomed my agenda at their summit in Bratislava. In doing so they chose unity. They chose to rally around our common ground. Together, we showed that Europe can deliver for its citizens when and where it matters. Ever since, we have been slowly but surely gathering momentum. It helped that the economic outlook swung in our favour. We are now in the fifth year of an economic recovery that really reaches every single Member State. Growth in the European Union has outstripped that of the United States over the last two years. It now stands above 2% for the Union as a whole and at 2.2% for the monetary area. Unemployment is at a nine year low. Almost 8 million jobs have been created during this mandate so far. With 235 million people at work, more people are in employment in the European Union than ever before. The European Commission cannot take the credit for this alone. Though I am sure that had 8 million jobs been lost, we would have taken the blame. But Europe's institutions played their part in helping the wind change. We can take credit for our European Investment Plan which has triggered €225 billion worth of investment so far. It has granted loans to 450,000 small firms and more than 270 infrastructure projects. We can take credit for the fact that, thanks to determined action, European banks once again have the capital firepower to lend to companies so that they can grow and create jobs. And we can take credit for having brought public deficits down from 6.6% to 1.6%. This is thanks to an intelligent application of the Stability and Growth Pact. We ask for fiscal discipline but are careful not to kill growth. This is in fact working very well across the Union   despite the criticism. Ten years since crisis struck, Europe's economy is finally bouncing back. And with it, our confidence. Our 27 leaders, the Parliament and the Commission are putting the Europe back in our Union. And together we are putting the Union back in our Union. In the last year, we saw all 27 leaders walk up the Capitoline Hill in Rome, one by one, to renew their vows to each other and to our Union. All of this leads me to believe: the wind is back in Europe's sails. We now have a window of opportunity but it will not stay open forever. Let us make the most of the momentum, catch the wind in our sails. For this we must do two things: First, we should stay the course set out last year. We still have 16 months in which real progress can be made by Parliament, Council and Commission. We must use this time to finish what we started in Bratislava and deliver on our own positive agenda. Secondly, we should chart the direction for the future. As Mark Twain wrote   I am quoting   years from now we will be more disappointed by the things we did not do, than by those we did. Now is the time to build a more united, a stronger, a more democratic Europe for 2025. STAYING COURSE Mr President, Honourable Members, As we look to the future, we cannot let ourselves be blown off course. We set out to complete an Energy Union, a Security Union, a Capital Markets Union, a Banking Union and a Digital Single Market. Together, we have already come a long way. As the Parliament testified, 80% of the proposals promised at the start of the mandate have already been put forward by the Commission. We must now work together to turn proposals into law, and law into practice. As ever, there will be a degree of give and take. The Commission's proposals to reform our Common Asylum System and strengthen rules on the Posting of Workers have caused controversy, I know. Achieving a good result will need all sides to do their part so they can move towards each other. I want to say today: as long as the outcome is the right one for our Union and is fair to all its Member States, the Commission will be open to compromise We are now ready to put the remaining 20% of initiatives on the table by May 2018. This morning, I sent a Letter of Intent to the President of the European Parliament and to the Prime Minister of Estonia   whose strong work for Europe I would like to praise   outlining the priorities for the year ahead. I will not and I cannot list all these proposals here, but let me mention five which are particularly important. Firstly, I want us to strengthen our European trade agenda. Yes, Europe is open for business. But there must be reciprocity. We have to get what we give. Trade is not something abstract. Trade is about jobs, creating new opportunities for Europe's businesses big and small. Every additional €1 billion in exports supports 14,000 extra jobs in Europe. Trade is about exporting our standards, be they social or environmental standards, data protection or food safety requirements. Europe has always been an attractive place to do business. But over the last year, partners across the globe are lining up at our door to conclude trade agreements with us. With the help of this Parliament, we have just secured a trade agreement with Canada that will provisionally apply as of next week. We have a political agreement with Japan on a future economic partnership. And by the end of the year, we have a good chance of doing the same with Mexico and South American countries. Today, we are proposing to open trade negotiations with Australia and New Zealand. I want all of these agreements to be finalised by the end of this mandate. And I want them negotiated in the greatest transparency. Open trade must go hand in hand with open policy making. The European Parliament will have the final say on all trade agreements. So its Members, like members of national and regional parliaments, must be kept fully informed from day one of the negotiations. The Commission will make sure of this. From now on, the Commission will publish in full all draft negotiating mandates we propose to the Council. Citizens have the right to know what the Commission is proposing. Gone are the days of no transparency. Gone are the days of rumours, of incessantly questioning the Commission's motives. I call on the Council to do the same when it adopts the final negotiating mandates. Let me say once and for all: we are not naïve free traders. Europe must always defend its strategic interests. This is why today we are proposing a new EU framework for investment screening. If a foreign, state-owned, company wants to purchase a European harbour, part of our energy infrastructure or a defence technology firm, this should only happen in transparency, with scrutiny and debate. It is a political responsibility to know what is going on in our own backyard so that we can protect our collective security if needed. Secondly, the Commission wants to make our industry stronger and more competitive. This is particularly true for our manufacturing base and the 32 million workers that form its backbone. They make the world-class products that give us our edge, like our cars. I am proud of our car industry. But I am shocked when consumers are knowingly and deliberately misled. I call on the car industry to come clean and make it right. Instead of looking for loopholes, they should be investing in the clean cars of tomorrow Honourable Members, the new Industrial Policy Strategy we are presenting today will help our industries stay, or become, the number one in innovation, digitisation and decarbonisation. Third: I want Europe to be the leader when it comes to the fight against climate change. Last year, we set the global rules of the game with the Paris Agreement ratified here, in this very House. Set against the collapse of ambition in the United States, Europe must ensure we make our planet great again. It is the shared heritage of all of humanity. The Commission will shortly present proposals to reduce the carbon emissions of our transport sector. Fourth priority for the year ahead: I want us to better protect Europeans in the digital age. Over the past years, we have made marked progress in keeping Europeans safe online. New rules, put forward by the Commission, will protect our intellectual property, our cultural diversity and our personal data. We have stepped up the fight against terrorist propaganda and radicalisation online. But Europe is still not well equipped when it comes to cyber-attacks. Cyber-attacks can be more dangerous to the stability of democracies and economies than guns and tanks. Last year alone there were more than 4,000 ransomware attacks per day and 80% of European companies experienced at least one cyber-security incident. Cyber-attacks know no borders and no one is immune. This is why, today, the Commission is proposing new tools, including a European Cybersecurity Agency, to help defend us against such attacks. Fifth: migration must stay on our radar. In spite of the debate and controversy around this topic, we have managed to make solid progress   though admittedly insufficient in many areas. We are now protecting Europe's external borders more effectively. Over 1,700 officers from the new European Border and Coast Guard are now helping Member States' 100,000 national border guards patrol in places like Greece, Italy, Bulgaria and Spain. We have common borders but Member States that by geography are the first in line cannot be left alone to protect them. Common borders and common protection must go hand in hand. We have managed to stem irregular flows of migrants, which were a cause of great anxiety for many. We have reduced irregular arrivals in the Eastern Mediterranean by 97% thanks our agreement with Turkey. And this summer, we managed to get more control over the Central Mediterranean route with arrivals in August down by 81% compared to the same month last year. In doing so, we have drastically reduced the loss of life in the Mediterranean. I cannot talk about migration without paying strong tribute to Italy for their tireless and noble work. Over the summer months, the Commission worked in perfect harmony with the Prime Minister of Italy, my friend Paolo Gentiloni, and his government to improve the situation. We did so - and we will continue to do so - because Italy is saving Europe's honour in the Mediterranean. We must also urgently improve migrants' living conditions in Libya. I am appalled by the inhumane conditions in detention or reception centres. Europe has a responsibility   a collective responsibility   and the Commission will work in concert with the United Nations to put an end to this scandalous situation that cannot be made to last. Even if it saddens me to see that solidarity is not yet equally shared across all our Member States, Europe as a whole has continued to show solidarity. Last year alone, our Member States resettled or granted asylum to over 720,000 refugees   three times as much as the United States, Canada and Australia combined. Europe, contrary to what some say, is not a fortress and must never become one. Europe is and must remain the continent of solidarity where those fleeing persecution can find refuge. I am particularly proud of the young Europeans volunteering to give language courses to Syrian refugees or the thousands more young people who are serving in our new European Solidarity Corps. These young people are bringing life and colour to European solidarity. But we now need to redouble our efforts. At the end of the month, the Commission will present a new set of proposals with an emphasis on returns, solidarity with Africa and opening legal pathways. When it comes to returns, I would like to repeat that people who have no right to stay in Europe must be returned to their countries of origin. When only 36% of irregular migrants are returned, it is clear we need to significantly step up our work. This is the only way Europe will be able to show solidarity with refugees in real need of protection. Solidarity cannot be an exclusively intra-European affair. We must also showsolidarity withAfrica. Africa is a noble continent, a young continent, the cradle of humanity. Our €2.7 billion EU-Africa Trust Fund is creating employment opportunities across the continent. The EU budget fronted the bulk of the money, but all our Member States combined have still only contributed €150 million. The Fund is currently reaching its limits. We know   or we should know   the dangers of a lack of funding   in 2015 many migrants headed towards Europe when the UN's World Food Programme ran out of funds. I call on all Member States to now match their actions with their words and ensure the Africa Trust Fund does not meet the same fate. The risk is high. We will also work on opening up legal pathways. Irregular migration will only stop if there is a real alternative to perilous journeys. We are close to having resettled 22,000 refugees from Turkey, Jordan and Lebanon and I support UN High Commissioner for Refugees' call to resettle a further 40,000 refugees from Libya and the surrounding countries. At the same time, legal migration is an absolute necessity for Europe as an ageing continent. This is why the Commission made proposals to make it easier for skilled migrants to reach Europe with a Blue Card. I would like to thank the Parliament for its support on this. SETTING SAIL Dear Mr President, Ladies and Gentlemen, Honourable Members, I have mentioned just a few of the initiatives we want and must deliver over the next 16 months. But this alone will not be enough to regain the hearts and minds of Europeans. Now is the time to chart the direction for the future. In March, the Commission presented our White Paper on the future of Europe, with five scenarios for what Europe could look like by 2025. These scenarios have been discussed, sometimes superficially, sometimes violently. They have been scrutinised and partly ripped apart. That is good   they were conceived for exactly this purpose. I wanted to launch a process in which Europeans determined their own path and their own future. The future of Europe cannot be decided by decree. It has to be the result of democratic debate and, ultimately, broad consensus. This House contributed actively, through the three ambitious resolutions on Europe's future which I would like to particularly thank the rapporteurs for. And I want to thank all the colleagues that participated in the more than 2,000 public events across Europe that the Commission organised since March. Now is the time to draw first conclusions from this debate. Time to move from reflection to action. From debate to decision. Today I would like to present you my view: my own 'sixth scenario', if you will. This scenario is rooted in decades of first-hand experience. I have lived, fought and worked for the European project my entire life. I have seen and lived through good times and bad. I have sat on many different sides of the table: as a Minister, as Prime Minister, as President of the Eurogroup, and now as President of the Commission. I was there in Maastricht, Amsterdam, Nice and Lisbon as our Union evolved and enlarged. I have always fought for Europe. At times I have suffered because of Europe. And even despaired for Europe. Through thick and thin, I have never lost my love of Europe. But there is, as we know, rarely love without pain. Love for Europe because Europe and the European Union have achieved something unique in this fraying world: peace within and peace outside of Europe. Prosperity for many if not yet for all. This is something we have to remember during the European Year of Cultural Heritage. 2018 must be a celebration of cultural diversity. A UNION OF VALUES Our values are our compass. For me, Europe is more than just a single market. More than money, more than a currency, more than the euro. It was always about values. That is why, in my sixth scenario, there are three fundamentals, three unshakeable principles: freedom, equality and the rule of law. Europe is first of all a Union of freedom. Freedom from the kind of oppression and dictatorship our continent knows all too well   sadly none more than central and Eastern European countries. Freedom to voice your opinion, as a citizen and as a journalist   a freedom we too often take for granted. It was on these freedoms that our Union was built. But freedom does not fall from the sky. It must be fought for. In Europe and throughout the world. Second, Europe must be a Union of equality and a Union of equals. Equality between its Members, big or small, East or West, North or South. Make no mistake, Europe extends from Vigo to Varna. From Spain to Bulgaria. East to West: Europe must breathe with both lungs. Otherwise our continent will struggle for air. In a Union of equals, there can be no second class citizens. It is unacceptable that in 2017 there are still children dying of diseases that should long have been eradicated in Europe. Children in Romania or Italy must have the same access to measles vaccines as children in other European countries. No ifs, no buts. This is why we are working with all Member States to support national vaccination efforts. Avoidable deaths must not occur in Europe. In a Union of equals, there can be no second class workers. Workers should earn the same pay for the same work in the same place. This is why the Commission proposed new rules on posting of workers. We should make sure that all EU rules on labour mobility are enforced in a fair, simple and effective way by a new European inspection and enforcement body. It is absurd to have a Banking Authority to police banking standards, but no common Labour Authority for ensuring fairness in our single market. We will create such an Authority. In a Union of equals, there can be no second class consumers either. I cannot accept that in some parts of Europe, in Central and Eastern Europe,people are sold food of lower quality than in other countries, despite the packaging and branding being identical. Slovaks do not deserve less fish in their fish fingers. Hungarians less meat in their meals. Czechs less cacao in their chocolate. EU law outlaws such practices already. And we must now equip national authorities with stronger powers to cut out these illegal practices wherever they exist. Third, in Europe the strength of the law replaced the law of the strong. The rule of law means that law and justice are upheld by an independent judiciary. Accepting and respecting a final judgement is what it means to be part of a Union based on the rule of law. Our Member States gave final jurisdiction to the European Court of Justice. The judgements of the Court have to be respected by all. To undermine them, or to undermine the independence of national courts, is to strip citizens of their fundamental rights. The rule of law is not optional in the European Union. It is a must. Our Union is not a State but it must be a community of law. A MORE UNITED UNION These three principles   freedom, equality and the rule of law   must remain the foundations on which we build a more united, stronger and more democratic Union. When we talk about the future, experience tells me new Treaties and new institutions are not the answer people are looking for. They are merely a means to an end, nothing more, nothing less. They might mean something to us here in Strasbourg or in Brussels. They do not mean a lot to anyone else. I am only interested in institutional reforms if they lead to more efficiency in our European Union. Instead of hiding behind calls for Treaty change   which is in any case inevitable we must first change the mind-set that for some to win others must lose. Democracy is about compromise. And the right compromise makes winners out of everyone in the long run. A more united Union should see compromise, not as something negative, but as the art of bridging differences. Democracy cannot function without compromise. Europe cannot function without compromise. A more united Union also needs to become more inclusive. If we want to protect our external borders and rightly so strengthen them even more, then we need to open the Schengen area of free movement to Bulgaria and Romania immediately. We should also allow Croatia to become a full Schengen member once all the criteria are met. If we want the euro to unite rather than divide our continent, then it should be more than the currency of a select group of countries. The euro is meant to be the single currency of the European Union as a whole. All but two of our Member States are required and entitled to join the euro once they fulfil the conditions. Member States that want to join the euro must be able to do so. This is why I am proposing to create a Euro-accession Instrument, offering technical and even financial assistance. If we want banks to operate under the same rules and under the same supervision across our continent, then we should encourage all Member States to join the Banking Union. We need to reduce the remaining risks in the banking systems of some of our Member States. Banking Union can only function if risk-reduction and risk-sharing go hand in hand. As everyone well knows, this can only be achieved if the conditions, as proposed by the Commission in November 2015, are met. There can only be a common deposit insurance scheme once everyone will have done their national homework. And if we want to avoid social fragmentation and social dumping in Europe, then Member States should agree on the European Pillar of Social Rights as soon as possible and at the latest at the Gothenburg summit in November. National social systems will still remain diverse and separate for a long time. But at the very least, we should agree on a European Social Standards Union in which we have a common understanding of what is socially fair in our single market. I remain convinced: Europe cannot work if it shuns workers. Ladies and Gentlemen, if we want more stability in our neighbourhood, then we must also maintain a credible enlargement perspective for the Western Balkans. It is clear that there will be no further enlargement during the mandate of this Commission and this Parliament. No candidate is ready. But thereafter the European Union will be greater than 27 in number. Accession candidates must give the rule of law, justice and fundamental rights utmost priority in the negotiations. This rules out EU membership for Turkey for the foreseeable future. Turkey has been taking giant strides away from the European Union for some time. Journalists belong in newsrooms not in prisons. They belong where freedom of expression reigns. The call I make to those in power in Turkey is this: Let our journalists go. And not only ours. Stop insulting our Member States by comparing their leaders to fascists and Nazis. Europe is a continent of mature democracies. But deliberate insults create roadblocks. Sometimes I get the feeling Turkey is deliberately placing these roadblocks so that it can blame Europe for any breakdown in accession talks. As for us, we will always keep our hands stretched out towards the great Turkish people and all those who are ready to work with us on the basis of our values.  A STRONGER UNION Ladies and Gentlemen, I want our Union to be stronger and for this we need a stronger single market. When it comes to important single market questions, I want decisions in the Council to be taken more often and more easily by qualified majority   with the equal involvement of the European Parliament. We do not need to change the Treaties for this. There are so-called passerelle clauses in the current Treaties which allow us to move from unanimity to qualified majority voting in certain cases   provided the European Council decides unanimously to do so. I am also strongly in favour of moving to qualified majority voting for decisions on the common consolidated corporate tax base, on VAT, on fair taxes for the digital industry and on the financial transaction tax. Europe has to be able to act quicker and more decisively, and this also applies to theEconomic and Monetary Union. The euro area is more resilient now than in years past. We now have the European Stabilisation Mechanism (ESM). I believe the ESM should now progressively graduate into a European Monetary Fund which, however, must be firmly anchored in the European Union's rules and competences. The Commission will make concrete proposals for this in December. We need a European Minister of Economy and Finance: a European Minister that promotes and supports structural reforms in our Member States. He or she can build on the work the Commission has been doing since 2015 with our Structural Reform Support Service. The new Minister should coordinate all EU financial instruments that can be deployed if a Member State is in a recession or hit by a fundamental crisis. I am not calling for a new position just for the sake of it. I am calling for efficiency. The Commissioner for economic and financial affairs   ideally also a Vice-President   should assume the role of Economy and Finance Minister. He or she should also preside the Eurogroup. The European Economy and Finance Minister must be accountable to the European Parliament. We do not need parallel structures. We do not need a budget for the Euro area but a strong Euro area budget line within the EU budget. I am also not fond of the idea of having a separate euro area parliament. The Parliament of the euro area is this European Parliament. The European Union must also be stronger in fighting terrorism. In the past three years, we have made real progress. But we still lack the means to act quickly in case of cross-border terrorist threats. This is why I call for setting up a European intelligence unit that ensures data concerning terrorists and foreign fighters are automatically shared among intelligence services and with the police. I also see a strong case for tasking the new European Public Prosecutor with prosecuting cross-border terrorist crimes. I want our Union to become a stronger global actor. In order to have more weight in the world, we must be able to take foreign policy decisions quicker. This is why I want Member States to look at which foreign policy decisions could be moved from unanimity to qualified majority voting. The Treaty already provides for this, if all Member States agree to do it. We need qualified majority decisions in foreign policy if we are to work efficiently. And I want us to dedicate further efforts to defence matters. A new European Defence Fund is in the offing. As is a Permanent Structured Cooperation in the area of defence. By 2025 we need a fully-fledged European Defence Union. We need it. And NATO wants it. Last but not least, I want our Union to have a stronger focus on things that matter, building on the work this Commission has already undertaken. We should not meddle in the everyday lives of European citizens by regulating every aspect. We should be big on the big things. We should not march in with a stream of new initiatives or seek ever growing competences. We should give back competences to Member States where it makes sense. This is why this Commission has sought to be big on big issues and small on the small ones and has done so, putting forward less than 25 new initiatives a year where previous Commissions proposed well over 100. To finish the work we started, I am setting up a Subsidiarity and Proportionality Task Force as of this month to take a very critical look at all policy areas to make sure we are only acting where the EU adds value. The First Vice-President, my friend, Frans Timmermans, who has a proven track record on better regulation, will head this Task Force. The Timmermans Task Force should include Members of this Parliament as well as Members of national Parliaments. It should report back in a years' time. A MORE DEMOCRATIC UNION Honourable Members, Mr President, Our Union needs to take a democratic leap forward. I would like to see European political parties start campaigning for the next European elections much earlier than in the past. Too often Europe-wide elections have been reduced to nothing more than the sum of national campaigns. European democracy deserves better. Today, the Commission is proposing new rules on the financing of political parties and foundations. We should not be filling the coffers of anti-European extremists. We should be giving European parties the means to better organise themselves. I also have sympathy for the idea of having transnational lists in European elections   though I am aware this is an idea more than a few of you disagree with. I will seek to convince the President of my parliamentary Group to follow me in this ambition which will bring Europe democracy and clarity. I also believe that, over the months to come, we should involve national Parliaments and civil society at national, regional and local level more in the work on the future of Europe. Over the last three years, as we promised, Members of the Commission have visited national Parliaments more than 650 times. They also debated in more than 300 interactive Citizens' Dialogues in more than 80 cities and towns across 27 Member States. This is why I support President Macron's idea of organising democratic conventions across Europe in 2018. As the debate gathers pace, I will personally pay particular attention to Estonia, to Latvia, to Lithuania and to Romania in 2018. This is the year they will celebrate their 100th anniversary. Those who want to shape the future of our continent should well understand and honour our common history. This includes these four countries   the European Union would not be whole without them. The need to strengthen democracy and transparency also has implications for the European Commission. Today, I am sending the European Parliament a new Code of Conduct for Commissioners. The new Code first of all makes clear that Commissioners can be candidates in European Parliament elections under the same conditions as everyone else. The new Code will of course strengthen the integrity requirements for Commissioners both during and after their mandate. If you want to strengthen European democracy, then you cannot reverse the small democratic progress seen with the creation of lead candidates   'Spitzenkandidaten'. I would like the experience to be repeated. More democracy means more efficiency. Europe would function better if we were to merge the Presidents of the European Council and the European Commission. This is nothing against my good friend Donald, with whom I have worked intimately and seamlessly together since the beginning of my mandate. This is nothing against Donald or against me. Europe would be easier to understand if one captain was steering the ship. Having a single President would simply better reflect the true nature of our European Union as both a Union of States and a Union of citizens. OUR ROADMAP My dear colleagues, The vision of a more united, stronger and more democratic Europe I am outlining today combines elements from all of the scenarios I set out to you in March. But our future cannot remain a simple scenario, a sketch, an idea amongst others. We have to prepare the Union of tomorrow, today. This morning I sent a Roadmap to President Tajani, President Tusk as well as to the holders of the rotating Presidencies of the Council between now and March 2019, outlining where we should go from here. An important element will be the budgetary plans the Commission will present in May 2018. Here again we have a choice: either we pursue the European Union's ambitions in the strict framework of the existing budget, or we increase the European Union's budgetary capacity so that it might better reach its ambitions. I am for the second option. On 29 March 2019, the United Kingdom will leave the European Union. This will be both a sad and tragic moment. We will always regret it. But we have to respect the will of the British people. We will advance, we must advance because Brexit is not everything. Because Brexit is not the future of Europe. On 30 March 2019, we will be a Union of 27. I suggest that we prepare for this moment well, amongst the 27 and within the EU institutions. European Parliament elections will take place just a few weeks later, in May 2019. Europeans have a date with democracy. They need to go to the polls with a clear understanding of how the European Union will develop over the years to come. This is why I call on President Tusk and Romania, the country holding the Presidency in the first half of 2019, to organise a Special Summit in Romania on 30 March 2019. My wish is that this summit be held in the beautiful city of Sibiu, also known as Hermannstadt. This should be the moment we come together to take the decisions needed for a more united, stronger and democratic Europe. My hope is that on 30 March 2019, Europeans will wake up to a Union where we stand by all our values. Where all Member States respect the rule of law without exception. Where being a full member of the euro area, the Banking Union and the Schengen area has become the norm for all. Where we have shored up the foundations of our Economic and Monetary Union so that we can defend our single currency in good times and bad, without having to call on external help. Where our single market will be fairer towards workers from the East and from the West. I want Europeans to wake up to a Europe where we have managed to agree on a strong pillar of social standards. Where profits will be taxed where they were made. Where terrorists have no loopholes to exploit. Where we have agreed on a proper European Defence Union. Where eventually a single President leads the work of the Commission and the European Council, having been elected after a democratic Europe-wide election campaign. Mr President, if our citizens wake up to this Union on 30 March 2019, then the European Union will be a Union able to meet their legitimate expectations. CONCLUSION Honourable Members, Europe was not made to stand still. It must never do so. Helmut Kohl and Jacques Delors, whom I had the honour to know, taught me that Europe only moves forward when it is bold. The single market, Schengen and the single currency: these were all ideas that were written off as pipe dreams before they happened. And yet these three ambitious projects are now a part of our daily reality. Now that Europe is doing better, people tell me I should not rock the boat. But now is not the time to err on the side of caution. We started to fix the European roof. But today and tomorrow we must patiently, floor by floor, moment by moment, inspiration by inspiration, continue to add new floors to the European House. We must complete the European House now that the sun is shining and whilst it still is. Because when the next clouds appear on the horizon   and they will appear one day   it will be too late. So let's throw off the bowlines. Sail away from the harbour. And catch the trade winds in our sails."
#data = "President Trump at UN Assembly: Mr. Secretary General, Mr. President, world leaders, and distinguished delegates, welcome to New York. It is a profound honor to stand here in my home city as a representative of the American people to address the people of the world. As millions of our citizens continue to suffer the effects of the devastating hurricanes that have struck our country, I want to begin by expressing my appreciation to every leader in this room who has offered assistance and aid. The American people are strong and resilient, and they will emerge from these hardships more determined than ever before. Fortunately, the United States has done very well since Election Day last November 8. The stock market is at an all-time high, a record. Unemployment is at its lowest level in 16 years, and because of our regulatory and other reforms, we have more people working in the United States today than ever before. Companies are moving back, creating job growth, the likes of which our country has not seen in a very long time, and it has just been announced that we will be spending almost $700 billion on our military and defense. Our military will soon be the strongest it has ever been. For more than 70 years, in times of war and peace, the leaders of nations, movements, and religions have stood before this assembly. Like them, I intend to address some of the very serious threats before us today, but also the enormous potential waiting to be unleashed. We live in a time of extraordinary opportunity. Breakthroughs in science, technology, and medicine are curing illnesses and solving problems that prior generations thought impossible to solve. But each day also brings news of growing dangers that threaten everything we cherish and value. Terrorists and extremists have gathered strength and spread to every region of the planet. Rogue regimes represented in this body not only support terror but threaten other nations and their own people with the most destructive weapons known to humanity. Authority and authoritarian powers seek to collapse the values, the systems, and alliances, that prevented conflict and tilted the word toward freedom since World War II. International criminal networks traffic drugs, weapons, people, force dislocation and mass migration, threaten our borders and new forms of aggression exploit technology to menace our citizens. To put it simply, we meet at a time of both immense promise and great peril. It is entirely up to us whether we lift the world to new heights or let it fall into a valley of disrepair. We have it in our power, should we so choose, to lift millions from poverty, to help our citizens realize their dreams, and to ensure that new generations of children are raised free from violence, hatred, and fear. This institution was founded in the aftermath of two world wars, to help shape this better future. It was based on the vision that diverse nations could cooperate to protect their sovereignty, preserve their security, and promote their prosperity. It was in the same period exactly 70 years ago that the United States developed the Marshall Plan to help restore Europe. Those these beautiful pillars, they are pillars of peace, sovereignty, security, and prosperity. The Marshall Plan was built on the noble idea that the whole world is safer when nations are strong, independent, and free. As president, Truman said in his message to Congress at that time, our support of European recovery is in full accord with our support of the United Nations. The success of the United Nations depends upon the independent strength of its members. To overcome the perils of the present, and to achieve the promise of the future, we must begin with the wisdom of the past. Our success depends on a coalition of strong and independent nations that embrace their sovereignty, to promote security, prosperity, and peace, for themselves and for the world. We do not expect diverse countries to share the same cultures, traditions, or even systems of government, but we do expect all nations to uphold these two core sovereign duties, to respect the interests of their own people and the rights of every other sovereign nation. This is the beautiful vision of this institution, and this is the foundation for cooperation and success. Strong sovereign nations let diverse countries with different values, different cultures, and different dreams not just coexist, but work side by side on the basis of mutual respect. Strong sovereign nations let their people take ownership of the future and control their own destiny. And strong sovereign nations allow individuals to flourish in the fullness of the life intended by God. In America, we do not seek to impose our way of life on anyone, but rather to let it shine as an example for everyone to watch. This week gives our country a special reason to take pride in that example. We are celebrating the 230th anniversary of our beloved Constitution, the oldest constitution still in use in the world today. This timeless document has been the foundation of peace, prosperity, and freedom for the Americans and for countless millions around the globe whose own countries have found inspiration in its respect for human nature, human dignity, and the rule of law. The greatest in the United States Constitution is its first three beautiful words. They are We the people. Generations of Americans have sacrificed to maintain the promise of those words, the promise of our country and of our great history. In America, the people govern, the people rule, and the people are sovereign. I was elected not to take power, but to give power to the American people where it belongs. In foreign affairs, we are renewing this founding principle of sovereignty. Our government's first duty is to its people, to our citizens, to serve their needs, to ensure their safety, to preserve their rights, and to defend their values. As president of the United States, I will always put America first. Just like you, as the leaders of your countries, will always and should always put your countries first. All responsible leaders have an obligation to serve their own citizens, and the nation state remains the best vehicle for elevating the human condition. But making a better life for our people also requires us to with work together in close harmony and unity, to create a more safe and peaceful future for all people. The United States will forever be a great friend to the world and especially to its allies. But we can no longer be taken advantage of or enter into a one-sided deal where the United States gets nothing in return. As long as I hold this office, I will defend America's interests above all else, but in fulfilling our obligations to our nations, we also realize that it's in everyone's interests to seek the future where all nations can be sovereign, prosperous, and secure. America does more than speak for the values expressed in the United Nations charter. Our citizens have paid the ultimate price to defend our freedom and the freedom of many nations represented in this great hall. America's devotion is measured on the battlefields where our young men and women have fought and sacrificed alongside of our allies. From the beaches of Europe to the deserts of the Middle East to the jungles of Asia, it is an eternal credit to the American character that even after we and our allies emerge victorious from the bloodiest war in history, we did not seek territorial expansion or attempt to oppose and impose our way of life on others. Instead, we helped build institutions such as this one to defend the sovereignty, security, and prosperity for all. For the diverse nations of the world, this is our hope. We want harmony and friendship, not conflict and strife. We are guided by outcomes, not ideologies. We have a policy of principled realism, rooted in shared goal, interests, and values. That realism forces us to confront the question facing every leader and nation in this room, it is a question we cannot escape or avoid. We will slide down the path of complacency, numb to the challenges, threats, and even wars that we face, or do we have enough strength and pride to confront those dangers today so that our citizens can enjoy peace and prosperity tomorrow. If we desire to lift up our citizens, if we aspire to the approval of history, then we must fulfill our sovereign duties to the people we faithfully represent. We must protect our nations, their interests and their futures. We must reject threats to sovereignty from the Ukraine to the South China Sea. We must uphold respect for law, respect for borders, and respect for culture, and the peaceful engagement these allow. And just as the founders of this body intended, we must work together and confront together those who threatens us with chaos, turmoil, and terror. The score of our planet today is small regimes that violate every principle that the United Nations is based. They respect neither their own citizens nor the sovereign rights of their countries. If the righteous many do not confront the wicked few, then evil will triumph. When decent people and nations become bystanders to history, the forces of destruction only gather power and strength. No one has shown more contempt for other nations and for the well-being of their own people than the depraved regime in North Korea. It is responsible for the starvation deaths of millions of North Koreans. And for the imprisonment, torture, killing, and oppression of countless more. We were all witness to the regime's deadly abuse when an innocent American college student, Otto Warmbier, was returned to America, only to die a few days later. We saw it in the assassination of the dictator's brother, using banned nerve agents in an international airport. We know it kidnapped a sweet 13-year-old Japanese girl from a beach in her own country, to enslave her as a language tutor for North Korea's spies. If this is not twisted enough, now North Korea's reckless pursuit of nuclear weapons and ballistic missiles threatens the entire world with unthinkable loss of human life. It is an outrage that some nations would not only trade with such a regime, but would arm, supply, and financially support a country that imperils the world with nuclear conflict. No nation on Earth has an interest in seeing this band of criminals arm itself with nuclear weapons and missiles. The United States has great strength and patience, but if it is forced to defend itself or its allies, we will have no choice but to totally destroy North Korea. Rocket Man is on a suicide mission for himself and for his regime. The United States is ready, willing, and able, but hopefully this will not be necessary. That's what the United Nations is all about. That's what the United Nations is for. Let's see how they do. It is time for North Korea to realize that the denuclearization is its only acceptable future. The United Nations Security Council recently held two unanimous 15-0 votes adopting hard-hitting resolutions against North Korea, and I want to thank China and Russia for joining the vote to impose sanctions, along with all of the other members of the Security Council. Thank you to all involved. But we must do much more. It is time for all nations to work together to isolate the Kim regime until it ceases its hostile behavior. We face this decision not only in North Korea; it is far past time for the nations of the world to confront another reckless regime, one that speaks openly of mass murder, vowing death to America, destruction to Israel, and ruin for many leaders and nations in this room. The Iranian government masks a corrupt dictatorship behind the false guise of a democracy. It has turned a wealthy country, with a rich history and culture, into an economically depleted rogue state whose chief exports are violence, bloodshed, and chaos. The longest-suffering victims of Iran's leaders are, in fact, its own people. Rather than use its resources to improve Iranian live, its oil profits go to fund Hezbollah and other terrorists that kill innocent Muslims and attack their peaceful Arab and Israeli neighbors. This wealth, which rightly belongs to Iran's people, also goes to shore up Bashar al-Assad's dictatorship, fuel Yemen's civil war, and undermine peace throughout the entire Middle East. We cannot let a murderous regime continue these destabilizing activities while building dangerous missiles, and we cannot abide by an agreement if it provides cover for the eventual construction of a nuclear program. The Iran deal was one of the worst and most one-sided transactions the United States has ever entered into. Frankly, that deal is an embarrassment to the United States, and I don't think you've heard the last of it. Believe me. It is time for the entire world to join us in demanding that Iran's government end its pursuit of death and destruction. It is time for the regime to free all Americans and citizens of other nations that they have unjustly detained. Above all, Iran's government must stop supporting terrorists, begin serving its own people, and respect the sovereign rights of its neighbors. The entire world understands that the good people of Iran want change, and, other than the vast military power of the United States, that Iran's people are what their leaders fear the most. This is what causes the regime to restrict internet access, tear down satellite dishes, shoot unarmed student protesters, and imprison political reformers. Oppressive regimes cannot endure forever, and the day will come when the people will face a choice. Will they continue down the path of poverty, bloodshed, and terror, or will the Iranian people return to the nation's proud roots as a center of civilization, culture, and wealth, where their people can be happy and prosperous once again? The Iranian regime's support for terror is in stark contrast to the recent commitments of many of its neighbors to fight terrorism and halt its finance, and in Saudi Arabia early last year, I was greatly honored to address the leaders of more than 50 Arab and Muslim nations. We agreed that all responsible nations must work together to confront terrorists and the Islamic extremism that inspires them. We will stop radical islamic terrorism because we cannot allow it to tear up our nation and, indeed, to tear up the entire world. We must deny the terrorists safe haven, transit, funding, and any form of support for their vile and sinister ideology. We must drive them out of our nation. It is time to expose and hold responsible those countries whose support and fi   who support and finance terror groups like al-Qaeda, Hezbollah, the Taliban, and others that slaughter innocent people. The United States and our allies are working together throughout the Middle East to crush the loser terrorists and stop the reemergence of safe havens they use to launch attacks on all of our people. Last month I announced a new strategy for victory in the fight against this evil in Afghanistan. From now on, our security interests will dictate the length and scope of military operation, not arbitrary benchmarks and timetables set up by politicians. I have also totally changed the rules of engagement in our fight against the Taliban and other terrorist groups. In Syria and Iraq, we have made big gains toward lasting defeat of ISIS. In fact, our country has achieved more against ISIS in the last eight months than it has in many, many years combined. We seek the deescalation of the Syrian conflict, and a political solution that honors the will of the Syrian people. The actions of the criminal regime of Bashar al-Assad, including the use of chemical weapons against his own citizens, even innocent children, shock the conscience of every decent person. No society could be safe if banned chemical weapons are allowed to spread. That is why the United States carried out a missile strike on the airbase that launched the attack. We appreciate the efforts of the United Nations agencies that are providing vital humanitarian assistance in areas liberated from ISIS, and we especially thank Jordan, Turkey, and Lebanon for their role in hosting refugees from the Syrian conflict. The United States is a compassionate nation and has spent billions and billions of dollars in helping to support this effort. We seek an approach to refugee resettlement that is designed to help these horribly treated people and which enables their eventual return to their home countries to be part of the rebuilding process. For the cost of resettling one refugee in the United States, we can assist more than 10 in their home region. Out of the goodness of our hearts, we offer financial assistance to hosting countries in the region and we support recent agreements of the G20 nations that will seek to host refugees as close to their home countries as possible. This is the safe, responsible, and humanitarian approach. For decades the United States has dealt with migration challenges here in the Western Hemisphere. We have learned that over the long term, uncontrolled migration is deeply unfair to both the sending and the receiving countries. For the sending countries, it reduces domestic pressure to pursue needed political and economic reform and drains them of the human capital necessary to motivate and implement those reforms. For the receiving countries, the substantial costs of uncontrolled migration are born overwhelmingly by low-income citizens whose concerns are often ignored by both media and government. I want to salute the work of the United Nations in seeking to address the problems that cause people to flee from their home. The United Nations and African Union led peacekeeping missions to have invaluable contributions in stabilizing conflict in Africa. The United States continues to lead the world in humanitarian assistance, including famine prevention and relief, in South Sudan, Somalia, and northern Nigeria and Yemen. We have invested in better health and opportunity all over the world through programs like PEPFAR, which funds AIDS relief, the President’s Malaria Initiative, the Global Health Security Agenda, the Global Fund to End Modern Slavery, and the Women Entrepreneur's Finance Initiative, part of our commitment to empowering women all across the globe. We also thank   we also thank the secretary general for recognizing that the United Nations must reform if it is to be an effective partner in confronting threats to sovereignty, security, and prosperity. Too often the focus of this organization has not been on results, but on bureaucracy and process. In some cases, states that seek to subvert this institution's noble end have hijacked the very systems that are supposed to advance them. For example, it is a massive source of embarrassment to the United Nations that some governments with egregious human rights records sit on the UN Human Rights Council. The United States is one out of 193 countries in the United Nations, and yet we pay 22 percent of the entire budget and more. In fact, we pay far more than anybody realizes. The United States bears an unfair cost burden, but to be fair, if it could actually accomplish all of its stated goals, especially the goal of peace, this investment would easily be well worth it. Major portions of the world are in conflict, and some, in fact, are going to hell, but the powerful people in this room, under the guidance and auspices of the United Nations, can solve many of these vicious and complex problems. The American people hope that one day soon the United Nations can be a much more accountable and effective advocate for human dignity and freedom around the world. In the meantime, we believe that no nation should have to bear a disproportionate share of the burden, militarily or financially. Nations of the world must take a greater role in promoting secure and prosperous societies in their own region. That is why in the Western Hemisphere the United States has stood against the corrupt, destabilizing regime in Cuba and embraced the enduring dream of the Cuban people to live in freedom. My administration recently announced that we will not lift sanctions on the Cuban government until it makes fundamental reforms. We have also imposed tough calibrated sanctions on the socialist Maduro regime in Venezuela, which has brought a once thriving nation to the brink of total collapse. The socialist dictatorship of Nicolás Maduro has inflicted terrible pain and suffering on the good people of that country. This corrupt regime destroyed a prosperous nation   prosperous nation, by imposing a failed ideology that has produced poverty and misery everywhere it has been tried. To make matters worse, Maduro has defied his own people, stealing power from their elected representatives, to preserve his disastrous rule. The Venezuelan people are starving, and their country is collapsing. Their democratic institutions are being destroyed. The situation is completely unacceptable, and we cannot stand by and watch. As a responsible neighbor and friend, we and all others have a goal   that goal is to help them regain their freedom, recover their country, and restore their democracy. I would like to thank leaders in this room for condemning the regime and providing vital support to the Venezuelan people. The United States has taken important steps to hold the regime accountable. We are prepared to take further action if the government of Venezuela persists on its path to impose authoritarian rule on the Venezuelan people. We are fortunate to have incredibly strong and healthy trade relationships with many of the Latin American countries gathered here today. Our economic bond forms a critical foundation for advancing peace and prosperity for all of our people and all of our neighbors. I ask every country represented here today to be prepared to do more to address this very real crisis. We call for the full restoration of democracy and political freedoms in Venezuela. The problem in Venezuela is not that socialism has been poorly implemented, but that socialism has been faithfully implemented. From the Soviet Union to Cuba to Venezuela, wherever true socialism or communism has been adopted, it has delivered anguish and devastation and failure. Those who preach the tenets of these discredited ideologies only contribute to the continued suffering of the people who live under these cruel systems. America stands with every person living under a brutal regime. Our respect for sovereignty is also a call for action. All people deserve a government that cares for their safety, their interests, and their well-being, including their prosperity. In America, we seek stronger ties of business and trade with all nations of goodwill, but this trade must be fair and it must be reciprocal. For too long the American people were told that mammoth, multinational trade deals, unaccountable international tribunals, and powerful global bureaucracies were the best way to promote their success. But as those promises flowed, millions of jobs vanished and thousands of factories disappeared. Others gamed the system and broke the rules, and our great middle class, once the bedrock of American prosperity, was forgotten and left behind, but they are forgotten no more and they will never be forgotten again. While America will pursue cooperation and commerce with other nations, we are renewing our commitment to the first duty of every government, the duty of our citizens. This bond is the source of America's strength and that of every responsible nation represented here today. If this organization is to have any hope of successfully confronting the challenges before us, it will depend, as President Truman said some 70 years ago, on the independent strength of its members. If we are to embrace the opportunities of the future and overcome the present dangers together, there can be no substantive for strong, sovereign, and independent nations, nations that are rooted in the histories and invested in their destiny, nations that seek allies to befriend, not enemies to conquer, and most important of all, nations that are home to men and women who are willing to sacrifice for their countries, their fellow citizens, and for all that is best in the human spirit. In remembering the great victory that led to this body's founding, we must never forget that those heroes who fought against evil, also fought for the nations that they love. Patriotism led the Poles to die to save Poland, the French to fight for a free France, and the Brits to stand strong for Britain. Today, if we do not invest ourselves, our hearts, our minds, and our nations, if we will not build strong families, safe communities, and healthy societies for ourselves, no one can do it for us. This is the ancient wish of every people and the deepest yearning that lives inside every sacred soul. So let this be our mission, and let this be our message to the world. We will fight together, sacrifice together, and stand together for peace, for freedom, for justice, for family, for humanity, and for the almighty God who made us all. Thank you, God bless you, God bless the nations of the world, and God bless the United States of America. Thank you very much."
#data = "Editor of Guardian Katharine Viner Role of Media: A mission for journalism in a time of crisis  In a turbulent era, the media must define its values and principles, writes Guardian editor-in-chief Katharine Viner  Thu 16 Nov 2017 19.00 GMT Last modified on Wed 14 Feb 2018 16.55 GMT  Shares 8,056  ‘No former period, in the history of our Country, has been marked by the agitation of questions of a more important character than those which are now claiming the attention of the public. So began the announcement, nearly 200 years ago, of a brand-new newspaper to be published in Manchester, England, which proclaimed that the spirited discussion of political questions and the accurate detail of facts were particularly important at this juncture.  Now we are living through another extraordinary period in history: one defined by dazzling political shocks and the disruptive impact of new technologies in every part of our lives. The public sphere has changed more radically in the past two decades than in the previous two centuries – and news organisations, including this one, have worked hard to adjust. A mission for journalism in a time of crisis – podcast  But the turbulence of our time may demand that we do more than adapt. The circumstances in which we report, produce, distribute and obtain the news have changed so dramatically that this moment requires nothing less than a serious consideration of what we do and why we do it.  The Scott Trust, which owns the Guardian, stated a very clear purpose when it was established in 1936: to secure the financial and editorial independence of the Guardian in perpetuity and to safeguard the journalistic freedom and liberal values of the Guardian free from commercial or political interference. As an editor, it’s hard to imagine a finer mission for a proprietor: our sole shareholder is committed only to our journalistic freedom and longterm survival.  But if the mission of the Scott Trust is to ensure that Guardian journalism will exist for ever, it is still left to us to define what the mission of that journalism will be. What is the meaning and purpose of our work? What role do we play in society?  After working at the Guardian for two decades, I feel I know instinctively why it exists. Most of our journalists and our readers do, too – it’s something to do with holding power to account, and upholding liberal values. We know what defines a Guardian story, what feels like a Guardian perspective, what makes something very Guardian (for better and for worse).  In my own work as editor of the Guardian in Australia, and then as the editor of the Guardian in the US, I tried to conceptualise the Guardian with a different accent – to identify the essential qualities of Guardian journalism and bring them to new audiences. Now, as the editor-in-chief of the Guardian and the Observer, I believe our time requires something deeper. It is more urgent than ever to ask: who are we, fundamentally?  The answer to this question is in our past, our present and our future. I want to lead a Guardian that relates to the world in a way that reflects our history, engages deeply with this disorientating global moment, and is sustainable for ever.  The history of the Guardian begins on 16 August 1819, when John Edward Taylor, a 28-year-old English journalist, attended an enormous demonstration for parliamentary reform in Manchester. In St Peter’s Field, a popular radical speaker, Henry Hunt, addressed a crowd estimated to contain 60,000 people – more than half the population of the Manchester area at the time, dressed in their Sunday best and packed in so tightly that their hats were said to be touching. Lose yourself in a great story: Sign up for the long read email Read more  At the time, the mood in the country was insurrectionary. The French revolution, three decades earlier, had spread throughout the world the seismic idea that ordinary people could face down the powerful and win – a revelation for the masses and a fright for those in power. After Britain’s victory at Waterloo and the end of the Napoleonic wars, the country was mired in economic depression and high unemployment, while the Corn Laws, which kept the price of grain artificially high, brought mass hunger. There were protests and riots throughout the country, from handloom weavers trashing newly invented factory machinery to anti-slavery campaigners boycotting sugar.  There was also a growing campaign for the vote: the big, densely populated city of Manchester had no member of parliament – while Old Sarum, a prosperous hamlet in southern England, with just one voter, had two MPs to represent him. The city’s businessmen were demanding an overhaul of this rotten system – and working men (and, for the first time, women) wanted their own chance to vote.  The combination of economic depression, political repression and the politicisation of workers with economic need was combustible. As the essayist William Hazlitt wrote one year earlier, nothing that was established was to be tolerated … the world was to be turned topsy-turvy. Print showing the Peterloo massacre, published by Richard Carlile in 1819. The Peterloo massacre of 1819. Photograph: Courtesy of Manchester Libraries, Information and Archives Advertisement  As most of Manchester gathered in St Peter’s Field on 16 August, the city’s magistrates, intimidated by the size of the crowd and their demands, ordered armed cavalry to charge into the crowd – to break up the meeting and arrest Hunt and other speakers on the podium. The troops stormed through the people, hacking with their sabres and cutting at everyone they could reach. Eleven people were killed on the day, seven men and four women, and many hundreds were injured. It became known as the Peterloo massacre or the Battle of Peterloo, and its impact was huge: the historian AJP Taylor said that Peterloo began the breakup of the old order in England.  John Edward Taylor was in the crowd that day, reporting for a weekly paper, the Manchester Gazette. When a reporter for the daily Times of London was arrested, Taylor was concerned that the people of the capital might not get an accurate report of the massacre – he correctly feared that without the account of a journalist on the scene, Londoners would instead get only the official version of events, which would protect the magistrates who had caused the bloodshed.  So Taylor rushed a report on to the night coach to London, got it into the Times, and thus turned a Manchester demonstration into a national scandal. Taylor exposed the facts, without hysteria. By reporting what he had witnessed, he told the stories of the powerless, and held the powerful to account.  But Taylor did not stop there. After the massacre, he spent months reporting on the fate of the wounded, documenting the injuries of more than 400 survivors. Quick guide What was the Peterloo massacre? Advertisement  Taylor’s relentless effort to tell the full story of Peterloo strengthened his own reformist political views – and he became determined to agitate for fair representation in parliament. He decided to start his own newspaper, the Manchester Guardian, with the financial backing of other middle-class radicals (10 put up £100 each, and an 11th contributed £50). The first edition was published on 5 May 1821, devoted to enlightenment values, liberty, reform and justice. It was launched with great confidence and optimism, by a man who believed that, in spite of Peterloo and police spies, reason was great and would prevail.  The Manchester Guardian was founded in a mood of great hope, and faith in ordinary people. The manifesto that Taylor produced before the paper’s launch speaks powerfully of the great diffusion of Education that was taking place, and the greatly increased interest which political subjects excite, and the immense extension of the circle within which they are discussed. It is of the utmost importance that this increased interest should be turned to beneficial account.  It is a powerful document, and one whose ideals still shape the Guardian – a celebration of more people getting educated, of more people engaging in politics, from different walks of life, from poorer communities. And it is a document that articulates a sense of responsibility to the public – that the Manchester Guardian could engage with the people who were starting to become involved in politics, giving them the information they need to take action. It is a wholly uncynical and unsnobbish document. It is on people’s side.  In the decades following Taylor’s death in 1844, the Manchester Guardian began to drift from the political ideals that had inspired its founding. It was highly profitable, but in becoming so it got too close to the Manchester cotton merchants who paid for the advertising that supported the paper. It even sided with the slave-owning south in the American civil war: the paper demanded that the Manchester cotton workers who starved in the streets because they refused to touch cotton picked by American slaves should be forced back into work. (Abraham Lincoln wrote to the working men of Manchester in 1863 to thank them for their sublime Christian heroism, which has not been surpassed in any age or in any country.) Advertisement  This period of complacency for the Guardian was dramatically ended by the appointment as editor of CP Scott, who transformed the paper and helped establish the political commitments that have been so important to its identity ever since.  Scott was made editor in 1872, at the age of 25. He was a radical Liberal and party activist who cared greatly about social justice and pacifism. Scott faced two big ideological challenges during the 57 years of his editorship; and his response to both helped form the Guardian as it is today.  The first was the question of Irish Home Rule: on the most contentious issue of the time, which split the Liberal party in the 1880s, Scott campaigned for self-government in Ireland – marking the moment, according to the historian David Ayerst, when the Guardian most clearly became a paper of the Left. At the end of the 19th century, Scott took the Guardian to an even more controversial anti-colonial position. During the second Boer war, from 1899 to 1902, Britain was rampantly jingoistic; anyone who opposed the war was cast as a traitor. The Guardian stood against it and ran a campaign for peace, while the brilliant Guardian reporter Emily Hobhouse exposed the concentration camps for the Boers run by the British. The prospectus published in advance of the first edition of the Manchester Guardian in 1821. Facebook Twitter Pinterest The prospectus published in advance of the first edition of the Manchester Guardian in 1821. Photograph: University of Manchester Library  The paper’s stance was so controversial that it lost advertisers and one-seventh of its sales. One rival paper, confident that the Guardian was on the verge of collapse, sent a brass band to stand outside our offices in Cross Street, Manchester to play Handel’s mournful Dead March from Saul.  Scott’s courageous position nearly did kill the Guardian. But in standing up to the prevailing political mood of the day, Scott turned the newspaper into the dominant expression of radical thinking among educated men and women, as Ayerst wrote. Clearly this was a paper that could not be bought.  As Scott orientated the paper towards a more radical position – away from laissez-faire liberalism to what was known as New Liberalism, concerned with social justice and welfare – he set the Guardian on the progressive path it has maintained, with a few missteps, ever since.  One of those missteps came in 1948. Surprising as it may seem today, the Manchester Guardian disparaged the foundation of Britain’s National Health Service. While supporting the changes as a great step forward, the Guardian feared that the state providing welfare risks an increase in the proportion of the less gifted. Three years later, the paper went further and backed the Conservatives at the 1951 general election. (Historians believe that these decisions came about because the editor at the time, AP Wadsworth, loathed Nye Bevan, the passionate Labour politician behind the welfare state.)  Making sense of a political moment when you’re in the midst of it is difficult – even if you avoid commercial and personal conflicts, it can still be hard to see it and understand it. A news organisation might often get things wrong – it needs some core values and principles to stick to in order to try to get it right. Quick guide A Guardian timeline Advertisement  Many of these core values were laid out by Scott on the 100th birthday of the Guardian, with his justly celebrated centenary essay of 1921. It was here that Scott introduced the famous phrase comment is free, but facts are sacred, and decreed that the voice of opponents no less than that of friends has a right to be heard. It was here that he laid out the values of the Guardian: honesty, cleanness [integrity], courage, fairness, a sense of duty to the reader and a sense of duty to the community.  CP Scott’s essay, like John Edward Taylor’s foundational prospectus, is both powerful and hopeful; as Scott writes, the newspaper has a moral as well as a material existence.  Our moral conviction, as exemplified by Taylor and codified by Scott, rests on a faith that people long to understand the world they’re in, and to create a better one. We believe in the value of the public sphere; that there is such a thing as the public interest, and the common good; that we are all of equal worth; that the world should be free and fair.  These inspiring ideas have always been at the heart of the Guardian at its best – whether the paper is called the Manchester Guardian or the Guardian, the name it adopted in 1959 – and they are enshrined in our independent ownership structure, in which the Guardian is owned solely by the Scott Trust. Any money made must be spent on journalism. (The Observer, of course, has its own distinct and honourable history and perspective – and as part of the same company, we are close siblings, but not twins.)  This is the mission that has motivated so many of the great moments in Guardian history, from our independent reporting of the Spanish civil war to the dramatic Edward Snowden revelations; from taking an anti-colonial position in the Suez crisis to standing up to Rupert Murdoch, the police and politicians in the phone-hacking scandal; from sending Jonathan Aitken to jail to the Panama and Paradise Papers.  These values, beliefs and ideas are well-established and enduring. They do not, by themselves, tell us how to meet the moral urgency of this new era. The world we knew has been pulled out of shape, and we must ask what it means to uphold these values now – as journalists and as citizens – and how they will inform our journalism and purpose.  Almost 200 years have passed since the public meeting that sparked Peterloo. But the past three decades – since the invention of the world wide web in 1989 – have transformed our idea of the public in ways that John Edward Taylor and CP Scott could not have imagined.  This technological revolution was exciting and inspiring. After 600 years of the Gutenberg era, when mass communication was dominated by established and hierarchical sources of information, the web felt like a breath of fresh air: open, creative, egalitarian. As its creator, Tim Berners-Lee, put it, this is for everyone. At first, it felt like the beginning of a thrilling new era of hyper-connectivity, with all the world’s knowledge at our fingertips and every person empowered to participate – as if the internet was one big town square where all our problems could be solved and everyone helped each other.  While many news organisations saw the internet as a threat to the old hierarchies of authority, forward-looking editors like Alan Rusbridger, who led the Guardian from 1995 until 2015, embraced this hopeful new future for journalism, by investing in digital expansion – hiring engineers and product managers – and by understanding that journalists, in this new world, must be open to challenge and debate from their audience. From making the Guardian the first British news organisation to employ a readers’ editor to launching an opinion site that inverted the traditional model of top-down newspaper commentary, he put the Guardian at the forefront of digital innovation and the changed relationships of this new era. As I wrote four years ago in my essay The Rise of the Reader, the open web created genuinely new possibilities for journalism – and journalists who resisted the technological revolution would damage both their own interests and the interests of good journalism.  But it has become clear that the utopian mood of the early 2000s did not anticipate all that technology would enable. Our digital town squares have become mobbed with bullies, misogynists and racists, who have brought a new kind of hysteria to public debate. Our movements and feelings are constantly monitored, because surveillance is the business model of the digital age. Facebook has become the richest and most powerful publisher in history by replacing editors with algorithms – shattering the public square into millions of personalised news feeds, shifting entire societies away from the open terrain of genuine debate and argument, while they make billions from our valued attention.  This shift presents big challenges for liberal democracy. But it presents particular problems for journalism.  The transition from print to digital did not initially change the basic business model for many news organisations – that is, selling advertisements to fund the journalism delivered to readers. For a time, it seemed that the potentially vast scale of an online audience might compensate for the decline in print readers and advertisers. But this business model is currently collapsing, as Facebook and Google swallow digital advertising; as a result, the digital journalism produced by many news organisations has become less and less meaningful.  Publishers that are funded by algorithmic ads are locked in a race to the bottom in pursuit of any audience they can find – desperately binge-publishing without checking facts, pushing out the most shrill and most extreme stories to boost clicks. But even this huge scale can no longer secure enough revenue.  On some sites, journalists who learned in training that news is something that someone, somewhere doesn’t want published churn out 10 commodified stories a day without making a phone call. Where once we had propaganda, press releases, journalism, and advertising, the academic Emily Bell has written, we now have ‘content’. Readers are overwhelmed: bewildered by the quantity of news they see every day, nagged by intrusive pop-up ads, confused by what is real and what is fake, and confronted with an experience that is neither useful nor enjoyable. How technology disrupted the truth Read more  Many people get most of their news from Facebook, which means that information arrives in one big stream – which may contain fact-based independent journalism from transparent sources alongside invented stories from a click farm, or content funded by malevolent actors to influence an election. The Richmond Standard, a website in California’s Bay Area, describes itself as a community-driven daily news source. If you see one of its headlines in your news feed, you couldn’t possibly know that it is owned by the multinational oil giant Chevron – which, according to the Financial Times, also owns the Richmond refinery that in August 2012 caught fire, spewing plumes of black smoke over the city and sending more than 15,000 residents to hospital for medical help. Such arrangements are no longer remarkable: the Australian Football League employs about 30 journalists to write friendly stories. Many free local newspapers in the UK are funded by the very councils they should be holding to account. It is asking a lot of individuals to sift the real from the fake when they are bombarded by information – how do they know who to trust?  Trust in all kinds of established institutions – including the media – is at an historic low. This is not a blip, and it should not be a surprise, when so many institutions have failed the people who trusted them and responded to criticism with contempt. As a result, people feel outraged but powerless – nothing they do seems to stop these things happening, and nobody seems to be listening to their stories.  This has created a crisis for public life, and particularly for the press, which risks becoming wholly part of the same establishment that the public no longer trusts. At a moment when people are losing faith in their ability to participate in politics and make themselves heard, the media can play a critical role in reversing that sense of alienation.  If mistrust in institutions is changing how people participate in civics, news organisations might need to change as well, the MIT professor Ethan Zuckerman has argued. We could rethink our role as journalists as helping people … find the places where they, individually and collectively, can be the most effective and powerful.  To do this well, journalists must work to earn the trust of those they aim to serve. And we must make ourselves more representative of the societies we aim to represent. Members of the media are increasingly drawn from the same, privileged sector of society: this problem has actually worsened in recent decades. According to the government’s 2012 report on social mobility in the UK, while most professions are still dominated by a social elite, journalism lags behind medicine, politics and even law in opening its doors to people from less well-off backgrounds. Indeed, the report concludes, journalism has had a greater shift towards social exclusivity than any other profession. Grenfell Tower in west London. Facebook Twitter Pinterest Grenfell Tower in west London. Photograph: Rick Findler/PA  This matters because people from exclusive, homogenous backgrounds are unlikely to know anyone adversely affected by the crises of our era, or to spend time in the places where they are happening. Media organisations staffed largely by people from narrow backgrounds are less likely to recognise the issues that people notice in their communities every day as news; the discussions inside such organisations will inevitably be shaped by the shared privilege of the participants.  After 71 people died in the devastating Grenfell Tower fire in west London – which residents had forewarned for years – the Channel 4 News presenter Jon Snow said that the failure to attend to these warnings showed that the media was comfortably with the elite, with little awareness, contact or connection with those not of the elite. As Gary Younge, the Guardian editor-at-large, has put it: ‘They’ are not ‘us’ – ‘their’ views are not often heard in newsrooms, and they know it.  If journalists become distant from other people’s lives, they miss the story, and people don’t trust them. The Guardian is not at all exempt from these challenges, and our staff is not diverse enough. Because of our history, values and purpose, we are committed to addressing these issues – but there is still a long way to go.  Meanwhile, those in power have exploited distrust of the media to actively undermine the role of journalism in the public interest in a democracy – from Donald Trump calling the fake news media the enemy of the American people to a British cabinet member suggesting that broadcasters should be patriotic in their Brexit reporting. All over the world – in Turkey, Russia, Poland, Egypt, China, Hungary, Malta and many other countries – powerful interests are on the march against free speech. Journalists are undermined, attacked, even murdered.  In these disorientating times, championing the public interest – which has always been at the heart of the Guardian’s mission – has become an urgent necessity. People are understandably anxious in the face of crises that are global, national, local and personal. At the global level, these crises are overwhelming: climate change, the refugee crisis, the rise of a powerful super-rich who bestride the global economy. It is easy to feel that humanity is facing a great shift, about which we were not consulted. Overwhelming technological, environmental, political and social change has precipitated what the philosopher Timothy Morton memorably describes as a traumatic loss of co-ordinates for all of us.  These global upheavals have plainly destabilised national politics, producing the shocks and surprises of the past two years: the unexpected result of the Brexit referendum, which leaves Britain facing a deeply uncertain future; the stunning election of Donald Trump; the collapse of support for traditional parties across Europe, and the unexpected rise of Emmanuel Macron. These events confounded the experts and the insiders who confidently declared them impossible. In the UK, Jeremy Corbyn appeared to have torn up the rulebook that had governed electoral politics for two decades – finding a surge of support in the June snap election, particularly with young people, by promoting socialist ideas that had long been dismissed. Bernie Sanders tapped into a similar mood in the US Democratic primary.  Skyrocketing inequality between the rich and poor has bred resentment at the political and economic establishment. In October it was revealed that the world’s super-rich now hold the greatest concentration of wealth for 120 years – many of them taking elaborate steps to avoid tax in the process, as the Paradise Papers showed. Paradise Papers leak reveals secrets of the world elite's hidden wealth Read more  What is becoming clear is that the way things have been run is unsustainable. We are at a turning point in which, in writer Naomi Klein’s words, the spell of neoliberalism has been broken, crushed under the weight of lived experience and a mountain of evidence. (Klein defines neoliberalism as shorthand for an economic project that vilifies the public sphere.) Perhaps the markets don’t have all the answers after all. The Financial Times columnist Martin Wolf, who says that many had not understood how radical the implications of worsening inequality would be, suggests that the political backlash to globalisation could possibly produce a fundamental transformation of the world – at least as significant as the one that brought about the first world war and the Russian revolution.  In many local areas, in our neighbourhoods and our communities, we see the collapse of civic life, from public space sold off cheaply to developers to the closing of libraries to the underfunding of schools and hospitals. It is not hard to imagine what has produced the growing tide of resentment that has shaken our politics. It is painful to see the rich getting away with it in the big cities while you’re struggling in your small town. Older people lament the loss of community life; younger people are unlikely to be able to find a good job or afford somewhere decent to live.  All of these dislocations have led to another set of crises at a personal level. This year, the World Health Organisation announced that cases of depression had ballooned in the past decade, making it the leading cause of disability worldwide. Loneliness is now being recognised as an epidemic throughout the west.  Our lives are increasingly atomised, but you can see the pleasure that comes from communal or civic participation. People long to help each other, to be together, to share experiences, to be part of a community, to influence the powers that control their lives. But in everyday life, such togetherness is hard to achieve: workplaces in the era of the gig economy no longer offer a solid place to gather; religion has declined; technology means that we often communicate via screens rather than face-to-face.  This is a dangerous moment: these are fertile grounds for authoritarianism and fascistic movements, and it’s no surprise that people feel anxious and confused. The desire to belong can just as easily find a home in dark places; new ways of participating can just as easily be used to foster hate.  But it is the presence of all these crises that recalls AJP Taylor’s remark that Peterloo began the breakup of the old order – and I cannot help wondering if this is another such moment. After the fever of Peterloo, amid mass demands for the vote, the Manchester Guardian caught the mood of the people, and found a way to respond – not to deny what was happening or minimise it, but to acknowledge it, contextualise it, analyse it, try to understand it, to turn it to beneficial account.  The urgent question now, then, is how the Guardian should do that today.  One response to this crisis is despair and escapism: to bury our heads in our phones, or watch some dystopian TV. Another is to declare that the whole system is broken, and everything must be torn down – a view whose popularity may partly explain our recent political tremors.  But despair is just another form of denial. People long to feel hopeful again – and young people, especially, yearn to feel the hope that previous generations once had.  Hope does not mean naively denying reality, as Rebecca Solnit explains in her inspiring book Hope in the Dark. Hope is an embrace of the unknown and the unknowable, an alternative to the certainty of both optimists and pessimists, Solnit writes. It’s a belief that actions have meaning and that what we do matters. Authentic hope, she says, requires clarity and imagination. American football players Eli Harold, Colin Kaepernick and Eric Reid kneeling during the US national anthem to protest against police brutality American football players Eli Harold, Colin Kaepernick and Eric Reid kneeling during the US national anthem to protest against police brutality Photograph: John G Mabanglo/EPA  Hope, above all, is a faith in our capacity to act together to make change. To do this, we need to be bold. Not everything that is faced can be changed, James Baldwin wrote in 1962. But nothing can be changed until it is faced. We need to accept the limits of the old kind of power, and work out what the new kinds will be. We need to be engaged with the world, uncynical, unsnobbish, on people’s side: just like the 1821 manifesto that established the Guardian.  Because people are not powerless to change things, and they are finding ways to act – new ways to get involved, to be citizens – it’s just that it might not be the kind of civic action that we’re used to. It might be American football players taking a knee to protest against police violence; it might be the people of Iceland, swarming around their parliament building banging pots and pans to bring down the government that bungled their country’s financial crash. It might be university students demanding divestment from fossil fuels, or the spread of small-scale renewable energy projects across the developing world. It might be digital activists building powerful new encryption tools in the wake of the Snowden revelations. This kind of action might not look like politics as we know it – but it’s politics all the same. These are new forms of engagement and participation, new ways to be an engaged citizen.  If people long to understand the world, then news organisations must provide them with clarity: facts they can trust, information that they need, reported and written and edited with care and precision.  If people long to create a better world, then we must use our platform to nurture imagination – hopeful ideas, fresh alternatives, belief that the way things are isn’t the way things need to be. We cannot merely criticise the status quo; we must also explore the new ideas that might displace it. We must build hope.  To do this, the Guardian will embrace as wide a range of progressive perspectives as possible. We will support policies and ideas, but we will not give uncritical backing to parties or individuals. We will also engage with and publish voices from the right. In an age of tumultuous change, nobody has a monopoly on good ideas.  But our guiding focus, especially in countries such as Britain, the US and Australia, will be to challenge the economic assumptions of the past three decades, which have extended market values such as competition and self-interest far beyond their natural sphere and seized the public realm. We will explore other principles and avenues through which to organise society for the common good. Neoliberalism: the idea that swallowed the world Read more  In doing this, we want nuance and knowledge, surprises and context and history, because power and influence might not reside where they used to; as identities change, the political assumptions of the recent past should not dictate our perspective on the present. We should be guided by curiosity, not certainty. We like experts, but that’s not enough; we must also ask why more people don’t.  This kind of journalism, which champions the public interest, requires a deep understanding of the changes taking place, so we will continually find the best ways to listen to people, even – perhaps especially – those who don’t read us. This is why it is essential that we have a staff that is representative of the society to which we all belong. We need to ensure our journalists will find and hear different stories, have different instincts, gain different insights, make different connections, give voice to the silenced, cover areas and topics that are neglected – in other words, make our journalism better. Advertisement  We do this by taking people seriously and treating our subjects, sources and readers with respect. Our relationship with our readers is not transactional: it is about sharing a sense of purpose and a commitment to understand and illuminate our times.  Supporting the Guardian – via subscriptions, contributions or membership – is only one way to participate in our mission. We are inviting our readers to be part of a community, whether that means reading and listening to and watching and sharing our work, or responding to it, or by sending us anonymous information or participating in a reporting project. We will also collaborate with news organisations – and others – who are doing work in the public interest.  We must embrace the new ways that people are engaging in the world, not long for a lost past when the ballot box and a handful of powerful media was the end of the story. As Ethan Zuckerman says, if news organisations can help make citizens feel powerful, like they can make effective civic change, they’ll develop a strength and loyalty they’ve not felt in years.  The Guardian is now funded more by our readers than by our advertisers. This is not only a new business model. It is an opportunity to focus on what readers value in Guardian journalism: serious reporting that takes time and effort, carefully uncovers the facts, holds the powerful to account, and interrogates ideas and arguments – work that speaks to the urgency of the moment, but lasts for more than a day. Being funded by our readers means we must focus on the stories that are most meaningful. It also means that we must spend money carefully, trying to produce – as one writer described CP Scott’s ambition for the Guardian a century ago – a great paper without any of the airs of a great paper.  Of course, in a serious age, the appetite for thoughtful, clever features beyond the news is possibly greater than ever. Our readers want to be nourished – by meaningful journalism about technology, economics, science, the arts – not fattened up with junk. They want useful, enjoyable reporting on how we live now, spotting trends, catching the mood, understanding what people are talking about – life-affirming, inspiring, challenging. We can be fun, and we must be funny, but it must always have a point, laughing with our audience, never at them. Their attention is not a commodity to be exploited and sold.  We will give people the facts, because they want and need information they can trust, and we will stick to the facts. We will find things out, reveal new information and challenge the powerful. This is the foundation of what we do. As trust in the media declines in a combustible political moment, people around the world come to the Guardian in greater numbers than ever before, because they know us to be rigorous and fair. If we once emphasised the revolutionary idea that comment is free, today our priority is to ensure that facts are sacred. Our ownership structure means we are entirely independent and free from political and commercial influence. Only our values will determine the stories we choose to cover – relentlessly and courageously. Migrants being rescued as they try to cross the Mediterranean sea from north Africa to Europe Migrants being rescued as they try to cross the Mediterranean sea from north Africa to Europe Photograph: Emilio Morenatti/AP  We will ask the questions that people are asking, and the questions that no one is asking. Honest reporters approach every situation with humility: they find the people who don’t get listened to and really listen to them. They get to know a place. We will get out of the big cities and the big institutions, and stay with stories for the long-term. Our commentary must also be based in facts, but we will keep a clear distinction between news and opinion.  We will provide a worthwhile space in which to read, watch, listen to and debate the issues of the day. We will be at the forefront of emerging new technologies, and will embrace those that truly benefit Guardian journalism and our readers’ experience of it. We must be proud of everything with the Guardian’s logo on it. Rather than overwhelming readers with stuff we demand they consume, we will edit for a meaningful experience. In print and in digital, we will be explanatory, visual, keepable. In recent years, the trend has been to prioritise the platforms on which journalism appears. We must now prioritise the reason for that journalism.  More than 800,000 people now help fund the Guardian, because they think what we do is important – and there are millions more who read us every day. This is inspiring, and it shows us a path towards a secure future for our journalism. We want to make sure that generations to come can read the Guardian, and that requires making our finances sustainable. Advertisement  For now, we cannot predict where this political moment will lead, or what changes await. There is much about the future that we do not know.  But we do know that there are serious questions that must be answered today, and that the Guardian is well placed to do this: because of our unique independent ownership; because of our high-quality journalism, rooted in the facts; because of our progressive perspective; and because our readers believe, as we do, that Guardian journalism should have the biggest possible impact and try to change the world for the better.  To steal Rainer Maria Rilke’s phrase, we must live the questions now: constantly examining our assumptions, our biases, how the world is changing, what it means. To do this, we will follow five principles: we will develop ideas that help improve the world, not just critique it; we will collaborate with readers, and others, to have greater impact; we will diversify, to have richer reporting from a representative newsroom; we will be meaningful in all of our work; and, underpinning it all, we will report fairly on people as well as power and find things out. Together, we are safeguarding the Guardian's independent journalism Read more  This is a challenge: a challenge for us at the Guardian to grab these principles, develop them and use them in all we do; a challenge to Guardian readers, to engage with us, support us if you believe in us, participate, advocate; and a challenge to all media organisations, to find ways to face this moment.  In the two-and-a-half years since I became editor-in-chief, we have experienced a huge number of political and social shocks, a dramatic undermining of the business model for serious journalism, and what many believe is an unprecedented level of disruption to our planet, our nation states, our communities, ourselves. It is a searching time to be an editor, a journalist and a citizen – but also a privilege to be grappling with these questions, with a possibility of helping to turn this era into something better, to turn this moment to beneficial account, as our founding manifesto proclaimed. And to do what has been the mission of the Guardian since 1821: to use clarity and imagination to build hope."
data = "Editor of Wired Mag Nicholas Thompson on Media: The Internet Society: You recently published (and co-wrote) a long feature on Facebook’s difficulties over the past two years, focusing to a large extent on its role in distributing news and misinformation (or fake news) alike. As policy leaders shape future norms in this field, do you think platforms face stricter regulatory measures? How?  Nicholas Thompson: Platforms need to do better. They need to play a better role in helping civil society, not breaking apart civil society. They need to do a better role of helping to build a world that shares the broad values they have, which includes truth and basic human rights. I don’t know whether regulation is the right answer. Platforms certainly need to change their internal policies; they certainly need to look at their algorithms; they certainly need to be much more aware of how people are using them and the effect they are having on civil society. The question with regulation is whether the regulation will actually help more than it harms. And regulating a tech company is hard: you have to understand and dig deep into its layers and know how it works. I wouldn’t trust many governments to do a good job of that. Sometimes the threat of regulation could be equally helpful. In my view there are three things that can happen. First, we could see changes in competition law. If we have more competitive landscapes we’ll probably see better behavior by the big tech companies. Second, we could also see more regulation to specifically limit what the tech companies can do or control. I’m very wary of these two options, which require us to intervene and either break up big tech or regulate tech more closely. Lastly, we could see action being taken internally by tech companies without any regulation. This is my favorite one; one in which tech companies start to look more carefully at themselves; think more carefully about their actions and start to act differently.  Recent research has questioned the notion of echo chambers, misinformation (or so-called ‘fake news’) and other media myths that impact the media’s credibility today. Are you concerned about the media’s role as counterweights to the excesses of power and corruption in the future? I think that echo chambers are a huge problem right now, especially in the current political situation in the US. I don’t think we’ll always have a president like Donald Trump, but even under Obama we had echo chambers and divisiveness and filter bubbles. The issue of filter bubbles and echo chambers might have been overstated at times, but it also is real and I think it’s really important for media to try to gain trust across a broad spectrum of society. I think the fact that serious media   media that actually checks facts and reports across multiple sources   only appeals to a segment of the population with one ideology is a huge problem. You need to have facts for society to work. You need everybody to agree on basic facts and then to have arguments and discussions and negotiations about those facts. And media is very important in providing those basic facts. So the fact that the media has lost the trust of the larger population is a massive problem in the US and elsewhere.  There are often different interpretations of particularly the right to freedom of expression in, for example, Europe and the US. How do these differences impact media and tech companies? American companies’ views of free speech have changed completely. They have gone from massively valuing free speech to not valuing free speech. If you look at many of the biggest trends in the tech industry, they have all been about filtering and limiting free speech. To get rid of comments sections; to make it harder to be anonymous online; to remove people from platforms. The reason why views on free speech have changed is because of what tech companies saw. They saw the abuse on Twitter, for example. If people are allowed to post anything on YouTube, people post videos of putting children in washing machines. The base human emotions lead to some terrible outcomes. And that’s a totally legitimate reason for the crackdown on free speech. Has it gone too far? I’m not sure. There are legitimate reasons why the tech industry’s view on free speech has changed in my view. What will the media landscape look like in three to five years? What/who will be more important producers of news in the future: citizen journalists or algorithms/artificial intelligence? AI is going to play a huge role in the future of the media. You can already see that in sports journalism, for example, where the Washington Post has AI writing sports pages. It’s pretty easy to describe a soccer game   you know, the game was tied until the 63rd minute and somebody scored. You could probably even get AI to write about a UN vote: for example, this country voted that way, etc. Over time, computers will become more sophisticated and should be able to write much larger proportions of stories. They wouldn’t be able to write our story about Facebook, because it required talking to lots of people, but they probably could do all kinds of sophisticated data pattern analysis that humans can’t do now. So, I think that AI will have a huge effect on journalism; both because of reporters using it as a tool to find things, and publishers actually using it as authors. While citizen journalism has become more important, I don’t know whether it’s going to keep growing the way it has grown. There are some really important limits on it. It’s very good for reporting on basic stories like ‘this is what’s happening here’. In fact, if the business model for local journalism goes away, citizen journalism about local reporting will become more important. But in the long term I think AI will have more of an influence. I suspect that while citizen journalism will always play a very significant role in the ecosystem, it has already had its biggest impact. Wired recently instituted a paywall. What made you take this leap, and do you think paywalls are the future for serious journalism online? One of the problems with media is that chasing digital advertisements makes you do things that aren’t great. It makes you pursue sensationalism, for example. Publications that are supported by paywalls and subscriptions and unique business models have a whole different set of incentives. My hope is that maybe some of the problems that have existed now, like misinformation and echo chambers, are caused because of publishers chasing digital advertising. I hope that when publishers start thinking more about paid content in different forms, whether it’s a paywall or other types, some of the problems we created in the last few years will be addressed. The recent feature Facebook article is around 11,000 words. What do Wired’s analytics say about whether readers can focus their attention on reading long pieces of journalism in an age of constant distraction? Our story about Facebook is long and hard to read, but within the first week of it being published it had almost two million readers. There has been a lot of concern about the shortening of attention spans because of the Internet. I think that concern is overstated. It is true, that sometimes it’s hard to concentrate. People start reading a book, they’ll read two pages and they grab their phone. We all do it ourselves. But if you look at the data it does seem like while phones distract us, they also make us smarter and they allow us to have conversations. You spent time reporting from West Africa on the role that technology plays there. What role do you think technology   especially technology from developed regions   has to play in sustainable development in developing regions?   Technology has a huge role to play in development; it could be incredibly important. When I was reporting on it, mostly in 2002 and 2003, I overestimated the role it could play though. I overestimated how easy it would be. It was really interesting for me in those early days of the Internet to watch and to think: oh, if there is an Internet connection suddenly people will be connected to the world. But it turns out that to have an Internet connection you also need to have electricity, and you need to have computer screens, you need to be able to repair them, you need to have a fiber optic network, and to have fiber optic network you need a telcom company, which means you need functioning transactions and economies. I was going to write a long essay about an incredible, brilliant guy in Ghana who was going to set up a technology center a couple of hours outside of the capital. He got Microsoft to sponsor him; he got ahold of computers. But it was hard to teach people how to use computers, as the power kept going out. I think the building may even have flooded. It was hard, it was really tricky. But today we see particularly the mobile telephone revolution that has come to West Africa. You’ve seen massive technology growth there. I think it’s doing all the things I thought it would do, it’s just doing it through mobile and it’s doing it later. And it was all way more complex that what I expected at the time. But it’s on my bucket list is to go back and find all the people I talked to then, and to ask them what role technology has played in their lives the past ten years or so.  What do you think of the role tech companies like Facebook are playing in connecting people to the Internet (or parts of it)? Platforms always have a conflict of interest when it comes to promoting connectivity. What you would really like is for the connectivity to happen separately from the platform providers. You don’t want Facebook providing Internet access because they are going to lock users into using only Facebook. You want the Internet being provided and then you want society to choose whether people spend a lot of time on Facebook or on other things. But Facebook’s business model works really well. And Facebook has all the money in the world, so is gaining access to the Internet via Facebook not better than not having connectivity? Probably. But I do wish that all people can just have a fresh start to the Internet. How has technology’s role in societies changed in the time since you’ve been working in this field? Have you perceived a backlash against the tech industry? Technology has changed since I started writing about it. It has also changed since Wired was started. Most big platforms and Internet companies were started by a bunch of outsiders with revolutionary ideas that were cool and different. Then, they all made billions of dollars. They started as David and became Goliath. I believe, though, that the people who run these companies are genuinely sincere people who are trying to help the world. At Facebook, for example, they want the world to become more open and connected. They are all smart, sincere good-hearted people who aren’t in it to manipulate us to gain power and make money. They happen to have made a lot of money, but I actually think that the top management at companies like Facebook and Google and Microsoft are in it for the right reasons. At the same time, they’ve also created something they can’t control, and they created something with more power than they know how to handle. They have incredible power, but they don’t know what to do with it. I don’t think that the executives are as evil as sometimes portrayed in popular imagination. I think the backlash sometimes goes too far. But they should have more responsibility, they should have thought more seriously of the consequences, they need to recognize the power that they have, which they haven’t. They need to do much better. They don’t know how to solve the problems they created. Wired aims to help its audiences understand how technology is changing our lives and what role it is playing in society. On the whole, what do you think are the biggest technological trends that will impact societies in the next five years? The biggest trend is clearly going to be AI. It has to be to train computers who can think like humans, act like humans, do things that humans did. It’s going to grow massively in the next five years to do the degree that it’s going to be hard to overstate. The biggest unknown is still quantum computing. Will it actually work, what will it do, who will get that power first. Will it be the US, will it be China, will it be Google, will it be Microsoft, will it be a startup. That’s the biggest unknown in the tech industry right now. Those are the two biggest. And then of course robotics, which ties in closely with AI.  What are your hopes for the future of the Internet? What are your fears? My hope is that we have a much more competitive tech industry than we have right now. One of the things that I don’t like is that there’s only five companies that have such a dominating influence. I don’t like that the only two social media platforms that are growing are owned by Facebook (WhatsApp and Instagram). I want there to be much more competition. I want Snapchat to thrive; I want five more alternatives. I want there to be an alternative search engine to Google; an online store alternative to Amazon. I want much more competition   not just competition between the tech companies. Regulation can help that happen, the threat of regulation can help make that happen. Consumers can help make that happen. I am all for regulation to ensure competitive markets. I am less in favor of regulation to either break up the companies. I am all in favor of strict regulation to maintain competition in the market place. And I would prefer regulation to prevent Facebook from ripping off Snapchat, for example. If we have as little competition as we have right now in three to five years’ time, we will all be in trouble. If we have more, we will be in the right place."

#txt = data['users'][0]['news'][6]['full_text']
txt = data

In [6]:
txt

'Editor of Wired Mag Nicholas Thompson on Media: The Internet Society: You recently published (and co-wrote) a long feature on Facebook’s difficulties over the past two years, focusing to a large extent on its role in distributing news and misinformation (or fake news) alike. As policy leaders shape future norms in this field, do you think platforms face stricter regulatory measures? How?  Nicholas Thompson: Platforms need to do better. They need to play a better role in helping civil society, not breaking apart civil society. They need to do a better role of helping to build a world that shares the broad values they have, which includes truth and basic human rights. I don’t know whether regulation is the right answer. Platforms certainly need to change their internal policies; they certainly need to look at their algorithms; they certainly need to be much more aware of how people are using them and the effect they are having on civil society. The question with regulation is whether th

Process the input set according to the same procedure as the seed texts

In [7]:
with open("data/stopwords_standard.txt", "r") as f:
    STOP_WORDS_STANDARD = set(f.read().strip().split("\n"))
print(STOP_WORDS_STANDARD)

with open("data/stopwords_openmaker.txt", "r") as f:
    STOP_WORDS_OPENMAKER = set(f.read().strip().split("\n"))
print(STOP_WORDS_OPENMAKER)

# merging the two list together
STOP_WORDS = STOP_WORDS_STANDARD.union(STOP_WORDS_OPENMAKER)
print(STOP_WORDS)

{'have', "how's", "didn't", "what's", 'having', "wasn't", 'other', 've', "it's", "i'll", 'yourself', "why's", 'if', 'until', 'himself', 'during', 'on', 'you', 'where', 'there', 'm', 'are', "we'd", "aren't", "we'll", 'who', "shouldn't", 'ours ', 'under', "hadn't", 'some', 'more', "here's", 'com', 'her', 'out', 'she', 'each', 'my', 'when', "where's", 'him', "you'll", "she'd", 'hers', 'can', 'then', 'get', 'would', 'myself', 'just', "he'd", 're', 'been', "isn't", "they've", 'were', 'any', 'than', 'because', 'while', 'yourselves', "haven't", 'had', 'only', "wouldn't", 'was', "doesn't", 'which', 'be', 'in', 'should', 'no', 'am', 'into', 'once', 'and', 'again', 'ought', "she's", 'we', 'did', 'by', 'how', 'that', 'both', 'all', "she'll", 'between', 'after', "there's", "they'd", 'those', 'ourselves', 'above', 'its', 'over', 'why', 'few', 's', 'themselves', "can't", "let's", 'has', 'down', 'know', 'own', 'nor', 'their', 'your', 'with', "won't", 'from', 'whom', 'www', 'yours', "mustn't", 'furthe

In [8]:
import nltk
from utils import tokenizer
import nltk
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from numpy import log, mean
import json, csv, re
import pprint as pp

tokens = nltk.word_tokenize(txt)

In [9]:
tokens

['Editor',
 'of',
 'Wired',
 'Mag',
 'Nicholas',
 'Thompson',
 'on',
 'Media',
 ':',
 'The',
 'Internet',
 'Society',
 ':',
 'You',
 'recently',
 'published',
 '(',
 'and',
 'co-wrote',
 ')',
 'a',
 'long',
 'feature',
 'on',
 'Facebook',
 '’',
 's',
 'difficulties',
 'over',
 'the',
 'past',
 'two',
 'years',
 ',',
 'focusing',
 'to',
 'a',
 'large',
 'extent',
 'on',
 'its',
 'role',
 'in',
 'distributing',
 'news',
 'and',
 'misinformation',
 '(',
 'or',
 'fake',
 'news',
 ')',
 'alike',
 '.',
 'As',
 'policy',
 'leaders',
 'shape',
 'future',
 'norms',
 'in',
 'this',
 'field',
 ',',
 'do',
 'you',
 'think',
 'platforms',
 'face',
 'stricter',
 'regulatory',
 'measures',
 '?',
 'How',
 '?',
 'Nicholas',
 'Thompson',
 ':',
 'Platforms',
 'need',
 'to',
 'do',
 'better',
 '.',
 'They',
 'need',
 'to',
 'play',
 'a',
 'better',
 'role',
 'in',
 'helping',
 'civil',
 'society',
 ',',
 'not',
 'breaking',
 'apart',
 'civil',
 'society',
 '.',
 'They',
 'need',
 'to',
 'do',
 'a',
 'bett

In [10]:
token_counts = FreqDist(tokens)
tokenizer.CHARACTERS_TO_SPLIT += '‘'+'’'+'“'+'”'+'.'

In [11]:
for stopword in STOP_WORDS:
    if stopword in token_counts:
        del token_counts[stopword]
        
for punctuation in tokenizer.CHARACTERS_TO_SPLIT:
    if punctuation in token_counts:
        del token_counts[punctuation]


        
pattern_letters = re.compile('[a-z]')
def has_letters(x):
    return(pattern_letters.search(x) is not None)

reduced = {k:v for k,v in token_counts.items() if has_letters(k)}
print("Reduction due to all number matches: ", len(token_counts) - len(reduced))
token_counts = reduced

Reduction due to all number matches:  7


In [12]:
token_counts

{'63rd': 1,
 'Africa': 2,
 'Amazon': 1,
 'American': 1,
 'And': 7,
 'Are': 1,
 'As': 1,
 'At': 2,
 'But': 9,
 'China': 1,
 'Consumers': 1,
 'David': 1,
 'Donald': 1,
 'Editor': 1,
 'Europe': 1,
 'Facebook': 15,
 'First': 1,
 'Ghana': 1,
 'Goliath': 1,
 'Google': 3,
 'Has': 1,
 'Have': 1,
 'He': 1,
 'How': 3,
 'If': 5,
 'In': 2,
 'Instagram': 1,
 'Internet': 12,
 'It': 9,
 'Lastly': 1,
 'Mag': 1,
 'Media': 1,
 'Microsoft': 3,
 'Most': 1,
 'My': 2,
 'Nicholas': 2,
 'Obama': 1,
 'On': 1,
 'One': 2,
 'Our': 1,
 'Over': 1,
 'People': 1,
 'Platforms': 3,
 'Post': 1,
 'Probably': 1,
 'Publications': 1,
 'Recent': 1,
 'Regulation': 1,
 'Second': 1,
 'Snapchat': 2,
 'So': 2,
 'Society': 1,
 'Sometimes': 1,
 'Technology': 2,
 'That': 1,
 'The': 8,
 'Then': 1,
 'There': 4,
 'They': 11,
 'This': 1,
 'Thompson': 2,
 'Those': 1,
 'To': 1,
 'Trump': 1,
 'Twitter': 1,
 'Washington': 1,
 'We': 1,
 'West': 2,
 'What': 8,
 'What/who': 1,
 'WhatsApp': 1,
 'When': 1,
 'While': 1,
 'Will': 2,
 'Wired': 5,
 

In [13]:
reduced = {k:v for k,v in token_counts.items() if len(k) > 1}
print("Reduction due to single characters: ", len(token_counts) - len(reduced))
token_counts = reduced

Reduction due to single characters:  0


In [14]:
token_counts

{'63rd': 1,
 'Africa': 2,
 'Amazon': 1,
 'American': 1,
 'And': 7,
 'Are': 1,
 'As': 1,
 'At': 2,
 'But': 9,
 'China': 1,
 'Consumers': 1,
 'David': 1,
 'Donald': 1,
 'Editor': 1,
 'Europe': 1,
 'Facebook': 15,
 'First': 1,
 'Ghana': 1,
 'Goliath': 1,
 'Google': 3,
 'Has': 1,
 'Have': 1,
 'He': 1,
 'How': 3,
 'If': 5,
 'In': 2,
 'Instagram': 1,
 'Internet': 12,
 'It': 9,
 'Lastly': 1,
 'Mag': 1,
 'Media': 1,
 'Microsoft': 3,
 'Most': 1,
 'My': 2,
 'Nicholas': 2,
 'Obama': 1,
 'On': 1,
 'One': 2,
 'Our': 1,
 'Over': 1,
 'People': 1,
 'Platforms': 3,
 'Post': 1,
 'Probably': 1,
 'Publications': 1,
 'Recent': 1,
 'Regulation': 1,
 'Second': 1,
 'Snapchat': 2,
 'So': 2,
 'Society': 1,
 'Sometimes': 1,
 'Technology': 2,
 'That': 1,
 'The': 8,
 'Then': 1,
 'There': 4,
 'They': 11,
 'This': 1,
 'Thompson': 2,
 'Those': 1,
 'To': 1,
 'Trump': 1,
 'Twitter': 1,
 'Washington': 1,
 'We': 1,
 'West': 2,
 'What': 8,
 'What/who': 1,
 'WhatsApp': 1,
 'When': 1,
 'While': 1,
 'Will': 2,
 'Wired': 5,
 

# Stemming

In [15]:
stemmer = PorterStemmer()
input_wset_stems = {k: stemmer.stem(k) for k in token_counts.keys()}
input_wset_stems

{'63rd': '63rd',
 'Africa': 'africa',
 'Amazon': 'amazon',
 'American': 'american',
 'And': 'and',
 'Are': 'are',
 'As': 'As',
 'At': 'At',
 'But': 'but',
 'China': 'china',
 'Consumers': 'consum',
 'David': 'david',
 'Donald': 'donald',
 'Editor': 'editor',
 'Europe': 'europ',
 'Facebook': 'facebook',
 'First': 'first',
 'Ghana': 'ghana',
 'Goliath': 'goliath',
 'Google': 'googl',
 'Has': 'ha',
 'Have': 'have',
 'He': 'He',
 'How': 'how',
 'If': 'If',
 'In': 'In',
 'Instagram': 'instagram',
 'Internet': 'internet',
 'It': 'It',
 'Lastly': 'lastli',
 'Mag': 'mag',
 'Media': 'media',
 'Microsoft': 'microsoft',
 'Most': 'most',
 'My': 'My',
 'Nicholas': 'nichola',
 'Obama': 'obama',
 'On': 'On',
 'One': 'one',
 'Our': 'our',
 'Over': 'over',
 'People': 'peopl',
 'Platforms': 'platform',
 'Post': 'post',
 'Probably': 'probabl',
 'Publications': 'public',
 'Recent': 'recent',
 'Regulation': 'regul',
 'Second': 'second',
 'Snapchat': 'snapchat',
 'So': 'So',
 'Society': 'societi',
 'Sometim

Catalog contains all the catagory score tables

In [16]:
catalog = []
for i in range(len(DF)):
    catalog.append({u[1].Stem: u[1].Score for u in DF[i].iterrows()})


Rate the example text according to the scores:

The scoring just adds up the word scores but this must be improved

In [17]:
Scores = []
for cat_idx in range(len(fn)):
    print(fn[cat_idx])

    sum_score = 0
    notin_catalog = 0
    for p in token_counts.keys():
        if p in input_wset_stems.keys():
            r = input_wset_stems[p]
        else:
            r = p
        
        #print('.....')
        if p in catalog[cat_idx].keys():
            #sum_score += catalog[cat_idx][p] 
            #print(catalog[cat_idx][p], p,token_counts[p])
            None
        else:
            #print('No Score', p,token_counts[p])
            #notin_catalog+=1
            None
            
        if r in catalog[cat_idx].keys():
            sum_score += catalog[cat_idx][r] 
            print(catalog[cat_idx][r], r)
        else:
            print('No Score', r)
            notin_catalog+=1

    print('Count of Terms not found in the catalog:', notin_catalog)   
    print('Score = ', sum_score)
    print(CATEGORIES[cat_idx])
    print('*'*30)
    
    Scores.append(sum_score)
    

schwartz_Stimulation.csv
No Score editor
No Score wire
No Score mag
No Score nichola
No Score thompson
2.203476173194637 media
No Score the
1.0358098829693214 internet
-0.2578018581999175 societi
No Score you
-0.8244253832927448 recent
-0.1256759471841298 publish
No Score co-wrot
-1.4865860131683482 long
0.7541236761076103 featur
No Score facebook
No Score difficulti
-1.2753942467857355 past
-2.1166121072716955 two
-1.9826759382805856 year
2.7253516331472136 focus
-0.7527276475513889 larg
-0.7429850513525704 extent
0.2293951471726281 role
No Score distribut
-0.2620123907362609 news
No Score misinform
No Score fake
No Score alik
No Score As
-1.222038516118224 polici
No Score leader
No Score shape
-0.4558537913630771 futur
No Score norm
-1.6556327919482243 field
No Score think
No Score platform
No Score face
No Score stricter
2.060375329553964 regulatori
No Score measur
No Score how
No Score platform
-0.17500101374663113 need
No Score better
No Score they
0.006251595858418291 play
-0.460

-0.3137124110958414 free
-0.33957474812766764 speech
-1.2244105278339366 chang
-1.492426805732774 complet
-2.531319859699261 gone
No Score massiv
-0.8354083492613341 valu
-0.9471997552494508 biggest
-1.5977873213906002 trend
-1.7511613021496863 industri
0.3213115702140564 filter
-1.3526648633576148 limit
No Score To
-0.4259028316161647 rid
-1.5068155431848735 comment
-0.6755180241441109 section
-0.8908813246592273 make
-0.6169580683788741 harder
No Score anonym
1.0 onlin
-0.6354201312186095 remov
-1.4172033844951866 reason
-1.6403469358093965 saw
1.879456188260606 abus
No Score twitter
-1.2426639681432867 allow
-0.8255028202719311 post
-1.3670536008097152 anyth
No Score youtub
No Score video
-1.7581299714657794 put
0.2088838873331376 children
No Score wash
-1.4230456601194146 machin
-1.9923233589665743 base
1.6118148666922711 emot
0.17083061927576038 lead
No Score terribl
1.0096394643380526 outcom
-0.8601883782598212 total
0.1514125334186588 legitim
No Score crackdown
No Score ha
-1.39

0.9624207193048256 landscap
-2.0118325225720413 probabl
1.067781234962652 behavior
No Score big
No Score compani
No Score second
0.3118331531636762 specif
-1.1107512093614151 limit
-0.1497357944710413 control
No Score wari
No Score option
-0.8403885861098144 requir
-1.3037967295183457 us
No Score interven
-0.9176313570689076 either
No Score break
No Score regul
-1.6794896792928395 close
No Score lastli
0.0484183179486826 action
-2.0856667845830934 taken
No Score intern
-0.5908757500755065 without
No Score thi
No Score favorit
-1.771946790114758 start
-0.523307538803252 care
0.0484183179486826 action
-0.84586805187444 act
-1.1044420401681505 differ
-1.6346979210901031 recent
-0.2026813106321172 research
-1.1854579839282389 question
0.5569556111966611 notion
No Score echo
No Score chamber
No Score so-cal
1.6808857078490609 media
2.0767813649410747 myth
No Score impact
No Score credibl
-0.9176313570689076 today
No Score are
-1.3199616663717006 concern
No Score counterweight
0.508165447027

No Score build
No Score world
No Score share
No Score broad
No Score valu
No Score includ
No Score truth
No Score basic
No Score human
No Score right
No Score don
No Score whether
No Score regul
No Score right
No Score answer
No Score certainli
No Score chang
No Score intern
No Score polici
No Score look
No Score algorithm
No Score much
No Score awar
No Score peopl
No Score use
No Score effect
No Score question
No Score will
No Score actual
No Score help
No Score harm
No Score and
No Score regul
No Score tech
No Score compani
No Score hard
No Score understand
No Score dig
No Score deep
No Score layer
No Score work
No Score wouldn
No Score trust
No Score govern
No Score good
No Score job
No Score sometim
No Score threat
No Score equal
No Score help
No Score In
No Score view
No Score three
No Score thing
No Score happen
No Score first
No Score see
No Score chang
No Score competit
No Score law
No Score If
No Score competit
No Score landscap
No Score probabl
No Score behavior
No Score big


No Score advertis
-1.6790959895714923 think
-2.189921613337483 paid
0.02049301985690539 content
-1.0330955874372492 form
-0.6656785469666147 type
0.11266347965656255 creat
-2.1889359060127243 last
-0.35302048844465644 address
-1.5145466025233354 recent
0.9622628370320452 articl
-1.3474651434743803 around
-0.4284207045066085 word
No Score analyt
-0.8967902679300013 say
-0.974387986610471 reader
1.2574169243225273 focu
0.16437568070387307 attent
-1.6790959895714923 read
-1.7349764479659489 piec
-0.2103896524502598 age
No Score constant
No Score distract
No Score our
-1.6790959895714923 read
0.053372693976061474 within
-2.5422768961316997 week
No Score million
-0.8534021530495639 lot
-1.137633396343612 concern
No Score shorten
-0.15762685008334915 span
-0.422013359931608 true
-0.6189204077444554 sometim
No Score concentr
0.12264887833675628 peopl
-0.10355962881307353 book
No Score grab
No Score phone
No Score We
-2.259190308850456 seem
No Score phone
No Score distract
No Score smarter
-0.

...

No Score We
-1.586952491794858 seem
-1.5595535176067432 phone
No Score distract
No Score smarter
-0.7256337831657158 allow
-0.3839801878025052 convers
No Score spent
-1.3561785516149565 west
0.009062400307101995 africa
-0.23315729806792154 technolog
-1.5696038534602448 play
-1.0670770325089491 develop
-1.0540049509415963 region
-0.2786196721446789 sustain
-1.0670770325089491 develop
-1.0670770325089491 develop
-0.23315729806792154 technolog
No Score incred
No Score when
-1.3547591049607302 mostli
No Score overestim
-0.5512638672319193 though
-1.4664630945407315 interest
-0.6597920876837988 earli
-2.540382770618469 day
-1.3904771875628092 watch
No Score oh
-1.0670770325089491 connect
No Score suddenli
-1.0670770325089491 connect
-2.3580612138245147 turn
No Score electr
No Score comput
No Score screen
0.9000353241969671 repair
No Score fiber
No Score optic
-0.12446899231742073 network
No Score telcom
-0.7436107595668702 mean
-0.14248749782009892 function
2.1192756006536912 transact
-0.73

-0.4270331190903214 content
-0.605152989030576 form
-0.8795898347323363 type
-0.2634036953085192 creat
-1.874321992913054 last
No Score address
-1.4920691122248266 recent
1.0647830350040906 articl
-1.4019486943119999 around
-1.5998656826805342 word
No Score analyt
-1.8038487362556683 say
-0.4410848725459717 reader
-0.14562065965213586 focu
-1.1741233821332189 attent
-1.8389400560669384 read
No Score piec
-1.0932338626553322 age
-0.24941745333377935 constant
No Score distract
No Score our
-1.8389400560669384 read
-0.18830139076701646 within
-1.6035086739590354 week
-1.3048575701366805 million
-1.5240718433110003 lot
-1.5525343079747622 concern
No Score shorten
No Score span
-2.1223024673742032 true
-0.573970061593888 sometim
-0.8177144310142489 concentr
-0.5510857677603003 peopl
-0.4333027321039168 book
No Score grab
No Score phone
No Score We
-2.4012888328584303 seem
No Score phone
No Score distract
No Score smarter
0.0550500358100154 allow
-0.5919077622805553 convers
-1.10113210467957

2.245974091805179 landscap
-0.4365912029749813 five
No Score what/who
-0.007420756998095203 produc
0.5842343846819441 citizen
0.5113730364170721 journalist
No Score algorithms/artifici
-0.9702315045071432 intellig
-2.1941740344670198 go
-1.1532859305167509 alreadi
-0.9237114888722504 sport
-0.6264599654043187 journal
-0.33538559505439625 washington
-1.3756966126153076 post
-1.5011742704327335 write
-1.0254941831821929 page
No Score It
-2.7343394424046883 pretti
-1.718641363742138 easi
0.7147139604351024 describ
No Score soccer
-0.4594058807411525 game
-1.7078304476379225 tie
No Score 63rd
-2.1764744573676182 minut
-1.9396320616952467 somebodi
-0.9927043603592016 score
-1.5011742704327335 write
-0.022346407214770957 vote
0.34132157221493203 countri
-0.022346407214770957 vote
-0.5768310509236054 way
No Score over
-0.9702315045071432 time
-0.23056430831230498 comput
-0.08123291492099566 becom
-1.8304327697302547 sophist
-0.3457465622686311 abl
-0.9518823658389468 proport
-1.14603838027990

-1.1672458380226305 goe
-2.2902766540388857 away
No Score but
-0.07980684995264598 term
1.5668100657232085 influenc
No Score suspect
-0.8682642103169159 signific
No Score ecosystem
-0.11089743702267704 institut
No Score paywal
-1.8323456060274197 made
-1.0336802929758435 take
No Score leap
No Score paywal
-0.5849017990096516 one
-1.0810779121295488 problem
No Score chase
1.6837817423087131 digit
-0.18516736561047206 advertis
-1.03657853459237 make
No Score aren
-1.7512801533061988 great
0.14333670136156385 pursu
No Score sensation
-1.0914077616311255 public
-0.6675935148547649 support
No Score subscript
-1.1445175869450743 uniqu
0.4520721830111771 model
-1.2592712352498503 whole
-0.6515931735083237 set
0.8364838819215091 incent
No Score My
No Score hope
No Score mayb
-0.7453024991820811 exist
-1.8715663191807008 caus
-0.18516736561047206 advertis
-2.056200015733036 think
-0.18900614191763795 paid
-0.14434537109021722 content
-0.3161956280168763 form
0.22029774249769224 type
-0.37991144

No Score sensation
-0.1137852589080564 public
-0.20378873045736431 support
No Score subscript
-1.2560769474512354 uniqu
0.15819126571106112 model
-1.2549987802308902 whole
-0.82360463793176 set
0.6071414857589644 incent
No Score My
No Score hope
No Score mayb
-0.4426806387397131 exist
0.38399793444475466 caus
No Score advertis
-2.978689592455525 think
-1.1915384263136644 paid
-0.8782437780051572 content
-1.2386852047393664 form
-0.2967267261166335 type
0.13021741366865508 creat
-2.7309977599360544 last
-0.24015637462823905 address
-0.7735818303114708 recent
0.7443426072724495 articl
-1.2935435652914131 around
-1.4224676821729956 word
0.6071414857589645 analyt
-2.5709123445889808 say
No Score reader
1.1523685362422875 focu
-0.30357820106573513 attent
-2.067007163667564 read
No Score piec
-0.6746738053309054 age
-0.6110159535589278 constant
No Score distract
No Score our
-2.067007163667564 read
0.16623680779247904 within
No Score week
-0.6679272402507018 million
-2.0398210233634075 lot
-

Visualize the score of the txt for each category

In [18]:
for i in range(len(CATEGORIES)):
    print(CATEGORIES[i], Scores[i])

Stimulation -147.40034876059056
Hedonism -347.2751665394124
Tradition -287.7607501195769
 0
Conformity -330.128211202443
Self-direction -285.6835041837369
Benevolence -326.26431577169024
Power -327.0207980363774
Universalism -222.35739038531088
Achievement -268.270896846003
Security -278.4551609021719


In [19]:
print(txt)

Editor of Wired Mag Nicholas Thompson on Media: The Internet Society: You recently published (and co-wrote) a long feature on Facebook’s difficulties over the past two years, focusing to a large extent on its role in distributing news and misinformation (or fake news) alike. As policy leaders shape future norms in this field, do you think platforms face stricter regulatory measures? How?  Nicholas Thompson: Platforms need to do better. They need to play a better role in helping civil society, not breaking apart civil society. They need to do a better role of helping to build a world that shares the broad values they have, which includes truth and basic human rights. I don’t know whether regulation is the right answer. Platforms certainly need to change their internal policies; they certainly need to look at their algorithms; they certainly need to be much more aware of how people are using them and the effect they are having on civil society. The question with regulation is whether the